# Работа с метриками по OZON

### Импортруемы библиотеки

In [1]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None
import datetime
import numpy as np
import pyjokes
import xlrd
import glob
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from sys import exit
import sys
link_to_table = 'https://docs.google.com/spreadsheets/d/1A1TyZPeq9r2HiDi-CHfmTfRpuAQYToib1vqyTx4ZyRE/edit#gid=0'
df = pd.read_excel('/'.join(link_to_table.split('/')[:-1])+'/export?format=xlsx')
PATH, df = [dict(df.loc[x]) for x in list(df.index)], None
i = 0
while len([py for py in glob.glob(f"{PATH[i]['path_downloads']}*.xlsx")]) == 0:
    i +=1
path_downloads = PATH[i]['path_downloads']
path_algoritm = PATH[i]['path_algoritm']
sys.path.append(f'{path_algoritm}Модули')
import constants_for_marketplace_metrics_dags as const
import working_with_marketplace_metrics_dags as wwm
import maslow_working_module as mwm

In [2]:
for i in range(1, 20):
    locals()[f"path_ozon_orders_new{i}"] = None
    locals()[f"path_ozon_orders_new_fbs{i}"] = None

#### Константы

In [3]:
# Обработка отправлений (меняется редко)
oz_shipment_processing = 10
# Индекс локализации (Аналитика - География продаж - Общий индекс локализации)
oz_general_localization_index = 76

#### Остатки FBS

In [4]:
## Товары и цены - Список товаров - Скачать - Скачать остатки на моих складах
path_ozon_fbs = wwm.return_path('Остатки на моих складах*.csv')

#### Цены

In [5]:
## Товары и цены - Цены на товары - Все товары - Скачать шаблон
path_price = wwm.return_path('Шаблон_цен*.xlsx')

#### Заказы фбо и фбс

In [6]:
## FBO - Заказы со склада Ozon - Выбрать нужный день - CSV
##FBS - Заказы с моих складов - Все - Экспорт CSV
path_ozon_orders_new1 = path_downloads + 'orders (28).csv'   # (Заказы на озон)
path_ozon_orders_new_fbs1 = path_downloads + 'postings (29).csv'   # (Заказы fbs на озон)

# path_ozon_orders_new2 = path_downloads + 'orders (26).csv'
# path_ozon_orders_new_fbs2 = path_downloads + 'postings (27).csv' 

# path_ozon_orders_new3 = path_downloads + 'orders (27).csv'
# path_ozon_orders_new_fbs3 = path_downloads + 'postings (28).csv' 

#### Остатки

In [7]:
## Аналитика - Отчеты - Все товары и видимость на Ozon - Скачать
path_ozon_stocks_new = wwm.return_path('products*.csv')

#### Оценки IDC

In [8]:
## FBO - Управление остатками - Оборачиваемость - Скачать отчёт
path_in_tur_oz = wwm.return_path('inventory-turnover-report*.xlsx')

#### Остатки на всех складах Озон

In [9]:
## Товары и цены - Список товаров - Скачать - Отчёт по остаткам и товарам в пути на склады Ozon
path_os = wwm.return_path('stocks_and_movement_products-report*.xlsx')

#### Акции

In [10]:
dop_ac_name = [
               'products-1177169 (9).xlsx',
               'products-1177179 (9).xlsx',
               'products-1177210 (4).xlsx',
               'products-1177259 (9).xlsx',
               'products-1177292 (5).xlsx'
              ]

In [11]:
# for row in dop_ac_name:
#     df = pd.read_excel(path_downloads + row)
#     print()
#     print(row)
#     print(df.columns[0].split('Название акции: ')[1])
#     print()

In [12]:
def dt_col(data):
    date_col = []
    for row in data.columns:
        try:
            pd.to_datetime(row)
            date_col.append(row)
        except:
            None
    return date_col 

### Метрики, к которым происходит подгрузка

In [13]:
# файл с метриками общими 
path_ozon_orders_metrics = path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Исторические метрики /ozon_orders_metrics.csv'
# файл с метриками по остаткам
path_stocks_ozon_metrics = path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Остатки по дням /stocks_ozon_metric.csv'
# файл с метриками по объему
path_ozon_size_metrics = path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Метрики по объемам/ozon_size_metrics.csv'
# файл с метриками по ценам
path_ozon_price_metrics = path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Цены по дням /price_ozon_metric.csv'
# файл с метриками оценкам IDC
path_ozon_IDC_metrics = path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Метрики по оценки IDC/ozon_IDC_metrics.csv'

### Метрики, которые каждый раз обновляются

In [14]:
# файл со всеми обработанными признаками
path_OZON_full_features_metric = path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Метрики с расчетами и выгрузкой за день /OZON_full_features_metric.csv'
# Путь для файла "метрики финальные для презентации"
path_OZON_full_df = path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Метрики для презентации /OZON_full_df.csv'

### Пути к подгружаемым и ежедневно не обновляемым файлам

In [15]:
# файл с дополнением по именам (только для ОЗОН)
path_dop_name = path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Дополнение по именам/Дополнение по именам.xlsx'
# файл с себестоимостью (общий)
path_cost_price = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/Себестоимость /cost_price 2.0.xlsx'
# файл с РЦ (общий)
path_rc = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/РЦ/РЦ_обработанные.xlsx'
# файл с признаком "В резерве" (общий)
path_vr = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/В резерве/В резерве_обработанные.xlsx'

In [16]:
path_ozon_fbs_metrics = path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/fbs остатки ozon/ozon_fbs_metrics.csv'

### Обработка цен

In [17]:
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True
wb = xlrd.open_workbook(path_price)
sh = wb.sheet_by_name('Товары и цены')
header = sh.row_values(2, start_colx=0, end_colx=None)
header = header[:38]
price = pd.DataFrame()
for i in range(len(header)):
    price[header[i]] = sh.col_values(i, start_rowx=4, end_rowx=None)
price['Артикул'] = price['Артикул'].astype(str)
if len(price) - len(list(set(price['Артикул']))) != 0:
    print('В price есть неуникальные артикулы')
    exit()
if 'Цена с учетом акции или стратегии, руб.' in price.columns:
    price = price.rename(columns={'Штрихкод': 'Баркод', 
                                  'Объемный вес, кг': 'Объемный вес',
                                  'Размер комиссии FBO, %': 'Комиссия в %',
                                  'Цена с учетом акции или стратегии, руб.': 'Цена',
                                  'Цена до скидки, руб.': 'Цена до скидки', 
                                  'Объем, л': 'Объем товара, л',
                                  'Размер комиссии FBS, %': 'Комиссия в % fbs',
                                  'Комиссия за эквайринг': 'Эквайринг'})
elif 'Цена с учетом акции, руб.' in price.columns:
    price = price.rename(columns={'Ozon SKU ID': 'Баркод', 
                                  'Объемный вес, кг': 'Объемный вес',
                                  'Размер комиссии FBO, %': 'Комиссия в %',
                                  'Цена с учетом акции, руб.': 'Цена',
                                  'Цена до скидки, руб.': 'Цена до скидки', 
                                  'Объем, л': 'Объем товара, л',
                                  'Размер комиссии FBS, %': 'Комиссия в % fbs',
                                  'Комиссия за эквайринг': 'Эквайринг'})
else:
    print('error')
    exit()
for i in range(len(price)):
    if price['Объемный вес'][i] == '0':
        price['Объемный вес'][i] = '0,1'
price['Объемный вес'] = price['Объемный вес'].astype('float')
price['Баркод'] = price['Баркод'].astype('int')
price['Комиссия в %'] = price['Комиссия в %'].astype('int')
price['Цена'] = price['Цена'].astype('int')
for i in range(len(price)):
    if price.loc[i]['Цена до скидки'] < price.loc[i]['Цена'] or price.loc[i]['Цена до скидки'] == 0:
        price.loc[i, ['Цена до скидки']] = price.loc[i]['Цена']
price_dop = price[['Артикул', 'Рыночная цена конкурентов, руб.', 'Моя цена на других площадках, руб.', 'Индекс цен', 'Ценовой индекс товара на Ozon', 'Ценовой индекс товара на рынке', 'Ценовой индекс товара на рынке на мои товары', 'Ссылка на рыночную цену конкурентов']]
price = price[['Артикул', 
       'Баркод',
       'Объемный вес',
       'Объем товара, л',
       'Цена',
       'Цена до скидки',
       'Эквайринг',
       'Комиссия в %',
       'Логистика Ozon, минимум, FBO',
       'Логистика Ozon, максимум, FBO',
       'Комиссия в % fbs',
       'Обработка отправления, минимум FBS',
       'Обработка отправления, максимум FBS',
       'Логистика Ozon, минимум, FBS',
       'Логистика Ozon, максимум, FBS']]

In [18]:
# flag_stop_nac = True
flag_stop_nac = False

reestr = wwm.Reestr()
# False - если нет продаж по fbs
# post = False
post = True

# файл с ценой по дню
path_price = path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Цены новые за каждый день /цены_озон_заказы_обработанные.xlsx'
paths = []
paths_fbs = []
for i in range(1, 20):
    try:
        if locals()[f"path_ozon_orders_new{i}"] != None:
            paths.append(locals()[f"path_ozon_orders_new{i}"])
        if locals()[f"path_ozon_orders_new_fbs{i}"] != None:
            paths_fbs.append(locals()[f"path_ozon_orders_new_fbs{i}"])
    except:
        None

### Обработка файлов с заказами FBS

In [19]:
for i in range(len(paths)):
    date_filter_1 = (pd.to_datetime((pd.read_csv(paths[i], sep=';')['Принят в обработку'])))\
                    .apply(lambda x: x.strftime('%d-%B-%Y')).value_counts()
    date_filter_2 = [x for x in list(date_filter_1.keys()) if date_filter_1[x] == sorted(date_filter_1.values)[-1]][0]
    fbs = pd.read_csv(paths_fbs[i], sep=';')
    fbs = fbs[fbs['Статус']!='Отменен']
    Q =  [x for x in list(fbs.index) if pd.to_datetime(fbs.loc[x]['Принят в обработку']).strftime('%d-%B-%Y') == date_filter_2]
    fbs = fbs.query("index in @Q").reset_index(drop=True)
    Q = None
    fbs.to_csv(paths_fbs[i], index=False, sep=";", encoding='utf-8-sig')
    fbs = None
    date_filter_1 = None
    date_filter_2 = None

### Общий индекс локализации

In [20]:
path_oz_general_localization_index_metrics = path_algoritm\
+'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Индекс локализации/oz_general_localization_index_metrics.xlsx'
oz_general_localization_index_metrics = pd.read_excel(path_oz_general_localization_index_metrics)
date_now = pd.to_datetime(datetime.datetime.now()).strftime('%d-%B-%Y')
if date_now in oz_general_localization_index_metrics.columns:
    oz_general_localization_index_metrics = oz_general_localization_index_metrics.drop(date_now, axis=1)
    oz_general_localization_index_metrics.loc[0, date_now] = oz_general_localization_index
else:
    oz_general_localization_index_metrics.loc[0, date_now] = oz_general_localization_index
oz_general_localization_index_metrics.to_excel(path_oz_general_localization_index_metrics, index=False)
oz_general_localization_index_metrics = None

### Доп. признаки

In [21]:
dop_pr = pd.read_excel(path_downloads + 'Доп. признаки озон.xlsx',
                       dtype={'Номенклaтурa': 'int'})
dop_komm = pd.read_excel(path_downloads + 'озон комменатрий.xlsx',
                       dtype={'Артикул': 'str'})

### Доп. по рейтингу

In [22]:
oz_rank_feature_metrics = pd.read_excel(path_algoritm \
                          + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Признак для ранжирования/oz_rank_feature_metrics.xlsx')

### Доп. категории 

In [23]:
cat = pd.read_excel(path_downloads + 'категории озон.xlsx')
cat['Артикул'] = cat['Артикул'].astype(str)
cat = cat.drop_duplicates().reset_index(drop=True)
if len(cat) - len(list(set(cat['Артикул']))) !=0:
    print('В cat неуникальные артикулы')
    exit()

### Доп. признак для маркетинга

In [24]:
# Спринт WB
path_spr_wb = path_downloads + 'Спринт вб.xlsx'
# Спринт OZON
path_spr_oz = path_downloads + 'Спринт озон.xlsx'

### Доп. признак

In [25]:
one = pd.read_excel(path_downloads + 'озон_отметка.xlsx')
one['Артикул'] = one['Артикул'].astype(str)
if len(one) - len(list(set(one['Артикул']))) !=0:
    print('В one неуникальные артикулы')
    exit()

In [26]:
def ac_redact(ac_name):
    
    for row in ac_name:
        df =pd.read_excel(path_downloads + row, sheet_name='Товары и цены')
        df.columns = list(df.loc[0].values)
        df = df.iloc[2:, :].reset_index(drop=True)
        try:
            df = df[['Артикул', 'Рассчитанная цена для участия в акции, RUB']]
        except:
            df = df[['Артикул', 'Итоговая цена по акции']]
        df.columns = ['Артикул', 'Итоговая цена по акции']
        q = pd.read_excel(path_downloads+row, sheet_name='Описание').columns[0]
        if 'Название акции' == q:
            df.columns = ['Артикул'] + [pd.read_excel(path_downloads+row, sheet_name='Описание').loc[0].values[0] + ' Цена']
            print(pd.read_excel(path_downloads+row, sheet_name='Описание').loc[0].values[0])
        else:
            df.columns = ['Артикул'] + [pd.read_excel(path_downloads+row, sheet_name='Описание').columns[0].split(':')[1][1:] + ' Цена']
            print(pd.read_excel(path_downloads+row, sheet_name='Описание').columns[0].split(':')[1][1:])
        df['Артикул'] = df['Артикул'].astype(str)
        print(row)
        if dop_ac_name.index(row) == 0:
            DF_ac = df.copy()
        if dop_ac_name.index(row) > 0:
            DF_ac = DF_ac.merge(df, on='Артикул', how='outer')
            DF_ac = DF_ac.fillna(0)
    df = None
    return DF_ac

In [27]:
DF_ac = ac_redact(dop_ac_name)

Хиты. Без ограничения срока действия
products-1177169 (9).xlsx
СуперХиты. Без ограничения срока действия
products-1177179 (9).xlsx
СуперХиты со скидкой на комиссию - 5%.Без ограничения срока действия
products-1177210 (4).xlsx
 МегаХиты. Без ограничения срока действия
products-1177259 (9).xlsx
МегаХиты со скидкой на комиссию - 5%. Без ограничения срока действия
products-1177292 (5).xlsx


### Работа с оценками IDC

In [28]:
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True
wb = xlrd.open_workbook(path_in_tur_oz)
sh = wb.sheet_by_name('Оборачиваемость товаров')
header = sh.row_values(2, start_colx=0, end_colx=None)
header = header[:38]
in_tur_oz = pd.DataFrame()
for i in range(len(header)):
    in_tur_oz[header[i]] = sh.col_values(i, start_rowx=1, end_rowx=None)
in_tur_oz_date = pd.to_datetime(in_tur_oz.loc[0]['Название'], format=('%d/%m/%Y')).strftime('%d-%B-%Y')    
in_tur_oz = in_tur_oz[['Артикул', 'IDC', 'Оценка IDC', 'SKU']].iloc[2:, :]
in_tur_oz['Артикул'] = in_tur_oz['Артикул'].astype(str)
in_tur_oz['SKU'] = in_tur_oz['SKU'].astype(int)
products = pd.read_csv(path_ozon_stocks_new, sep=';')
products['FBO OZON SKU ID'] = products['FBO OZON SKU ID'].astype(int)
products = list(set(products['FBO OZON SKU ID']))
in_tur_oz = in_tur_oz.query('SKU in @products').reset_index(drop=True)
in_tur_oz = in_tur_oz.drop('SKU', axis=1)
in_tur_oz = in_tur_oz.query(f"Артикул not in ['NaN', '', ' ', '   ', 'nan', '0', 'Nan', 'None', 'NONE', 'none']")\
                     .reset_index(drop=True)
if len(in_tur_oz) - len(list(set(in_tur_oz['Артикул']))) != 0:
    print('В in_tur_oz неуникальные артикулы')
    exit()
in_tur_oz_for_metrics = in_tur_oz.copy()
in_tur_oz_for_metrics = in_tur_oz_for_metrics.astype(str)
in_tur_oz_for_metrics[in_tur_oz_date] = in_tur_oz_for_metrics['IDC'] + ['; '] + in_tur_oz_for_metrics['Оценка IDC']
in_tur_oz_for_metrics = in_tur_oz_for_metrics.drop(['IDC', 'Оценка IDC'], axis=1)
ozon_IDC_metrics = pd.read_csv(path_ozon_IDC_metrics,
                               sep=';',
                               dtype={'Артикул': 'str'})
if len(ozon_IDC_metrics) - len(list(set(ozon_IDC_metrics['Артикул']))) != 0:
    print('В ozon_IDC_metrics неуникальные артикулы')
    exit()
else:
    L1 = len(ozon_IDC_metrics.columns)
if list(in_tur_oz_for_metrics.columns)[-1] not in list(ozon_IDC_metrics.columns):
    ozon_IDC_metrics = ozon_IDC_metrics.merge(in_tur_oz_for_metrics, on='Артикул', how='outer')
    ozon_IDC_metrics = ozon_IDC_metrics[['Артикул']\
    + [y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in list(ozon_IDC_metrics.columns)[1:]])]]
    L2 = len(ozon_IDC_metrics.columns)
else:
    L2 = L1 
if len(ozon_IDC_metrics) - len(list(set(ozon_IDC_metrics['Артикул']))) != 0:
    print('В ozon_IDC_metrics неуникальные артикулы')
    exit()
else:
    if L1 < L2:
        ozon_IDC_metrics.to_csv(path_ozon_IDC_metrics, index=False, sep=";", encoding='utf-8-sig')    
    elif L1 == L2:
        ozon_IDC_metrics = None
    else:
        exit()

### Удаление данных за день

In [29]:
def Apocalypse():
    ozon_orders_metrics = pd.read_csv(path_ozon_orders_metrics, sep=';')
    ozon_orders_metrics = ozon_orders_metrics.iloc[:, :-1]
    ozon_orders_metrics.to_csv(path_ozon_orders_metrics, index=False, sep=";", encoding='utf-8-sig')
    
    stocks_ozon_metrics = pd.read_csv(path_stocks_ozon_metrics, sep=';')
    stocks_ozon_metrics = stocks_ozon_metrics.iloc[:, :-1]
    stocks_ozon_metrics.to_csv(path_stocks_ozon_metrics, index=False, sep=";", encoding='utf-8-sig')
    
    
    ozon_price_metrics = pd.read_csv(path_ozon_price_metrics, sep=';')
    ozon_price_metrics = ozon_price_metrics.iloc[:, :-1]
    ozon_price_metrics.to_csv(path_ozon_price_metrics, index=False, sep=";", encoding='utf-8-sig')
    
    
    ozon_size_metrics = pd.read_csv(path_ozon_size_metrics, sep=';')
    ozon_size_metrics = ozon_size_metrics.iloc[:, :-1]
    ozon_size_metrics.to_csv(path_ozon_size_metrics, index=False, sep=";", encoding='utf-8-sig')

In [30]:
# Apocalypse()

### Добавление статусов

In [31]:
# def add_st_new(data):
#     st = pd.read_excel(path_st, 
#                        dtype={'Артикул': 'str', 'Статус': 'str'})
#     data = data.rename(columns={data.columns[0]: 'Артикул', data.columns[-1]: 'Новый статус'})
#     data['Артикул'] = data['Артикул'].astype(str)
#     data['Новый статус'] = data['Новый статус'].astype(str)
#     if len(data) - len(list(set(data['Артикул']))) == 0:
#         for i in range(len(data)):
#             row = data.loc[i]['Артикул']
#             if row in list(set(st['Артикул'])):
#                 st['Статус'].loc[st[st['Артикул']==row].index[0]] = data.loc[i]['Новый статус']
#             else:
#                 st.loc[len(st)] = list(data.loc[i].values)
#     else:
#         print('В файле с изменениями есть неуникальные артикулы')
#     st.to_excel(path_st, index=False)

In [32]:
# add_st_new(pd.DataFrame([['50370', 'товары в пути'], ['50371', 'товары в пути']], columns=['Артиул', 'Новый статус']))
# st = pd.read_excel(path_st)
# st['Артикул'] = st['Артикул'].astype(str)
# st['Статус'] = st['Статус'].astype(str)
# print(len(st) - len(list(set(st['Артикул']))))
# st.value_counts('Статус')

## Функции обработки OZON метрик

### Функция замены русских букв, если они есть в "Артикулах"

In [33]:
def letter_substitution(data):
    
    data = data.rename(columns={data.columns[0]: 'Артикул'})
    data['Артикул'] = data['Артикул'].astype(str)
    data = data.reset_index(drop=True)
    
    fa = list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(data['Артикул'])))

    if len(fa) > 0:

        Q = {'е': 'e', 
             'Е': 'E', 
             'Т': 'T', 
             'у': 'y', 
             'о': 'o', 
             'О': 'O', 
             'р': 'p', 
             'Р': 'P', 
             'а': 'a', 
             'А': 'A', 
             'Н': 'H',
             'К': 'K',
             'х': 'x',
             'Х': 'X',
             'с': 'c',
             'С': 'C',
             'В': 'B',
             'М': 'M'}

        for i in range(len(data)):

            for row in list(Q.keys()):

                if row in data.loc[i]['Артикул']:

                    data['Артикул'].loc[i] = data.loc[i]['Артикул'].replace(row, Q[row])


    fa2 = list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(data['Артикул'])))

    if len(fa2) == 0:

        return data





### Функция сортировки столбцов в порядке течения времени

In [34]:
# def data_columns_sorted(data):
#     col = [x for x in list(data.columns) if x not in dt_col(data)]\
#           + [z.strftime('%d-%B-%Y') for z in sorted([pd.to_datetime(y) for y in dt_col(data)])]
#     data = data[col]
#     return data

In [35]:
def data_columns_sorted(data):
    dt_col_data = dt_col(data)
    col_not_dt_data = [x for x in list(data.columns) if x not in dt_col_data]
    dt_col_data_s1 = sorted([pd.to_datetime(y) for y in dt_col_data])
    dt_col_data_s2 = [z.strftime('%d-%B-%Y') for z in dt_col_data_s1]
    col = col_not_dt_data + dt_col_data_s2
    dt_col_data = None
    col_not_dt_data = None
    dt_col_data_s1 = None
    dt_col_data_s2 = None
    data = data[col]
    return data

### Функция добавления новых товаров в метрики (если такие имеются в выгрузке по дню)

In [36]:
def add_new_art_in_ozon_orders_metrics(data, data2):     # data - озон метрики, data2 - файл с выгруженными остатками по дню

    data2 = data2[['Артикул', 'Наименование товара', 'FBO OZON SKU ID', 'Barcode', 'Бренд']]
    data2 = data2.rename(columns={'Наименование товара': 'Название',
                                          'FBO OZON SKU ID': 'Номенклaтурa',
                                          'Barcode': 'Баркод',
                                          'Бренд': 'ТМ'})

    data2['Артикул'] = data2['Артикул'].astype(str)
    data2['Название'] = data2['Название'].astype(str)

    Q = []
    for i in range(len(data2)):

        try:
            data2['Номенклaтурa'].loc[i] = int(data2.loc[i]['Номенклaтурa'])
        except:
            Q.append(i)


    Q = list(set(Q))

    if len(Q) !=0:

        for row in Q:

            data2['Номенклaтурa'].loc[row] = 0


    Q = []
    for i in range(len(data2)):

        try:
            data2['Баркод'].loc[i] = int(data2.loc[i]['Баркод'])
        except:
            Q.append(i)


    Q = list(set(Q))

    if len(Q) !=0:

        for row in Q:

            data2['Баркод'].loc[row] = 0


    data2['Номенклaтурa'] = data2['Номенклaтурa'].astype(int)
    data2['Баркод'] = data2['Баркод'].astype(int)  

    data2['ТМ'] = data2['ТМ'].astype(str)

    data2.drop_duplicates().reset_index(drop=True)
    
    
    if len(data) - len(list(set(data['Артикул']))) !=0:
        
        return 'В метриках есть неуникальные артикулы!'
    
    elif len(data2) - len(list(set(data2['Артикул']))) !=0:
        
        return 'В "Остатках" есть неуникальные артикулы!'
    
    else:
    

        for row in list(set(data2['Артикул'])):

            if row not in list(set(data['Артикул'])):

                data = pd.concat([data, data2[data2['Артикул']==row]], ignore_index=True)

            else:

                None


        data = data.fillna(0)

        data[list(data.columns[5:])] = data[list(data.columns[5:])].astype(int)
        
        
        return data

### Функция обработки выгруженных за день заказов

In [37]:
def redact_ozon_orders_new(data):
    
    data = data[data['Статус'] != 'Отменен']
    data = data[['Принят в обработку', 'Артикул', 'Количество']]
    data['Принят в обработку'] = pd.to_datetime(data['Принят в обработку'])
    data['Принят в обработку'] = data['Принят в обработку'].apply(lambda x: x.strftime('%d-%B-%Y')) 
    data['Принят в обработку'] = pd.to_datetime(data['Принят в обработку'])
    date_now = pd.to_datetime(datetime.datetime.now().strftime("%Y-%m-%d"))
    data['date_diff'] = date_now - data['Принят в обработку']
    data['date_diff'] = data['date_diff'].astype('str')
    R = []
    for row in data['date_diff']:

        R.append(int(" ".join(re.findall('(\d+)', row))))

    data['date_diff'] = R

    data['Принят в обработку'] = list(data[data['date_diff'] == data['date_diff'].min()]['Принят в обработку'])[0]

    data = data.drop('date_diff', axis=1)
    data['Принят в обработку'] = data['Принят в обработку'].apply(lambda x: x.strftime('%d-%B-%Y')) 
    data_pivot = data.pivot_table(index=['Артикул'], columns='Принят в обработку', values='Количество', aggfunc='sum')
    data_pivot = data_pivot.fillna(0)
    data_pivot['Артикул'] = list(data_pivot.index)
    data_pivot.reset_index(drop=True, inplace=True)
    data = pd.DataFrame(data=data_pivot.values, columns=[list(data_pivot.columns)[0], 'Артикул'])
    
    if len(data) - len(list(set(data['Артикул']))) != 0:
        
        return 'В "новых заказах" есть неуникальные артикулы!'
    
    else:
    
        return data



### Функция сведения новых заказов по дню с метриками

In [38]:
def add_ozon_orders_new_from_ozon_orders_metrics(data, data2): # data - ozon_orders_metrics, data2 - ozon_orders_new
    
    data['Артикул'] = data['Артикул'].astype(str)
    data2['Артикул'] = data2['Артикул'].astype(str)
    
    L = [len(data) - len(list(set(data['Артикул']))), len(data2) - len(list(set(data2['Артикул'])))]
    
    c = 0
    
    for row in list(set(data2['Артикул'])):
        
        if row not in list(set(data['Артикул'])):
            
            c += 1
    
    if sum(L) != 0:
        
        return 'Есть неуникальные артикулы!'
    
    elif len(data2) > len(data):
        
        return 'В выгрузке по ценам больше артикулов, чем в итоговых метриках!'
    
    elif c != 0:
        
        return 'В выгрузке по ценам есть артикулы, которых нет в метриках!'
    
    else:
        
        if data2.columns[0] not in list(data.columns):
        
            data = data.merge(data2, on='Артикул', how='left')

            data = data.fillna(0)

            data[list(data.columns[5:])] = data[list(data.columns[5:])].astype(int)
            
        else:
            
            None
        
        data = data_columns_sorted(data)
        
        return data
        
        
        

### Функция сведения остатков по дню с главными метриками

In [39]:
def redact_and_add_ozon_stocks_new_from_ozon_orders_metrics(data, data2): # data - ozon_orders_metrics, data2 - ozon_stocks_new
    
    if 'Остаток' not in list(data.columns):
        try:
            data2 = data2[['Артикул', 'Доступно на складе Ozon, шт']]
            data2 = data2.rename(columns={'Доступно на складе Ozon, шт': 'Остаток'})
        except:
            data2 = data2[['Артикул', 'Доступно к продаже по схеме FBO, шт.']].dropna().reset_index(drop=True)
            data2 = data2.rename(columns={'Доступно к продаже по схеме FBO, шт.': 'Остаток'})
        data2['Артикул'] = data2['Артикул'].astype(str)
        
        if len(data2) - len(list(set(data2['Артикул']))) != 0:
            
            return 'В "Остатках" есть неуникальные артикулы!'
        
        else:
            
            c = 0
            
            for row in list(set(data2['Артикул'])):
                
                if row not in list(set(data['Артикул'])):
                    
                    c += 1
                    
                else:
                    None
                    
                    
            if c != 0:
                
                return 'в "Остатках" есть новые артикулы!'
            
            else:
                
                data = data.merge(data2, on='Артикул', how='left')
                
                
                data = data.fillna(0)
                
                data[list(data.columns[5:])] = data[list(data.columns[5:])].astype(int)
                
                return data
            
            
    else:
            
        return data

    
    

### Функция добавления новых остатков к метрикам по остаткам

In [40]:
def add_ozon_stocks_new_from_stocks_ozon_metric(data, data2):  # data - ozon_orders_metrics, data2 - stocks_ozon_metric


    R = (pd.to_datetime(data.columns[-2], format='%d-%B-%Y') + datetime.timedelta(days=1)).strftime(format='%d-%B-%Y')
    
    if R+'-Ost' not in list(data2.columns):
    
    
        if len(data) - len(list(set(data['Артикул']))) == 0:

            if len(data2) - len(list(set(data2['Артикул']))) == 0:

                for row in list(set(data['Артикул'])):

                    if row not in list(set(data2['Артикул'])):

                        data2 = pd.concat([data2, data[data['Артикул']==row].iloc[:, :5]], 
                                                       ignore_index=True)

                        data2 = data2.fillna(0)

                        data2[list(data2.columns[5:])] = data2[list(data2.columns[5:])].astype(int)

                    else:

                        None

                data_Ost = data.copy()
                data_Ost = data_Ost[['Артикул', 'Остаток']]
                data_Ost = data_Ost.rename(columns={'Остаток': R+'-Ost'})

                data2 = data2.merge(data_Ost, on='Артикул', how='left')


                return data2


            else:

                return 'В метриках по остаткам есть неуникальные артикулы!'



        else:

            return 'В метриках есть неуникальные артикулы!'
        
        
    else:
        
        return data2
        

### Функция добавления "Себестоимости"

In [41]:
def add_cost_price(data):

    
    if 'Себестоимость' not in list(data.columns):
        
        cost_price = pd.read_excel(path_cost_price)
    
        cost_price['Артикул'] = cost_price['Артикул'].astype(str)
        data['Артикул'] = data['Артикул'].astype(str)

        cost_price = cost_price.drop_duplicates().reset_index(drop=True)
        
        
        if len(cost_price) - len(list(set(cost_price['Артикул']))) == 0:

            data = data.merge(cost_price, on='Артикул', how='left')

            data = data.fillna(0)

            data[list(data.columns[5:-1])] = data[list(data.columns[5:-1])].astype(int)
            
        else:
            
            data = 'В "Себестоимости" есть неуникальные артикулы!'
        
    else:
        
        None
        
    
    
    return data

### Функция добавления "Цены"

In [42]:
def add_price(data):
    
    price = pd.read_excel(path_price)
    
    price['Артикул'] = price['Артикул'].astype(str)
    data['Артикул'] = data['Артикул'].astype(str)
    
    data = data.merge(price[['Артикул', 'Цена']], on='Артикул', how='left')
    
    return data

### Функция добавления новых товаров в метрики по ценам и в метрики по остаткам (если такие появились в основных метриках)

In [43]:
def add_new_art_for_ozon_price_and_ozon_stocks_metrics(data, data1, data2):  # data - ozon_orders_metrics
                                                                             # data1 - ozon_price_metric
    if len(data) - len(list(set(data['Артикул']))) != 0:                     # data2 - stocks_ozon_metric
        
        return 'В метриках по озон заказам есть неуникальные артикулы!'
    
    elif len(data1) - len(list(set(data1['Артикул']))) != 0:
        
        return 'В метриках по озон ценам есть неуникальные артикулы!'
    
    elif len(data2) - len(list(set(data2['Артикул']))) != 0:
        
        return 'В метриках по озон остаткам есть неуникальные артикулы!'
    
    else:
        
    

        data['Артикул'] = data['Артикул'].astype(str)
        data1['Артикул'] = data1['Артикул'].astype(str)
        data2['Артикул'] = data2['Артикул'].astype(str)


        for row in list(data['Артикул']):

            if row not in list(data1['Артикул']):

                data1 = pd.concat([data1, data[data['Артикул']==row].iloc[:, :5]], ignore_index=True)

                data1 = data1.fillna(0)

                try:
                    data1[list(data1.columns[5:])] = data1[list(data1.columns[5:])].astype(int)
                except:
                    None

            else:

                None



        for row in list(data['Артикул']):

            if row not in list(data2['Артикул']):

                data2 = pd.concat([data2, data[data['Артикул']==row].iloc[:, :5]], ignore_index=True)

                data2 = data2.fillna(0)

                try:
                    data2[list(data2.columns[5:])] = data2[list(data2.columns[5:])].astype(int)
                except:
                    None

            else:

                None
            
            
            
        return data1, data2
            
            

### Функция добавления новой "цены" в метрики по ценам

In [44]:
def add_price_from_ozon_price_metric(data, data1):   # data - ozon_price_metric
                                                     # data1 - ozon_orders_metrics
        
    data['Артикул'] = data['Артикул'].astype(str)   
    data1['Артикул'] = data1['Артикул'].astype(str)
        
    if len(data) != len(data1):
        
        return 'Метрики по ценам и метрики по заказам отличаются по длине датасета!'
    
    else:
        
#         R = (pd.to_datetime(data1.columns[-6], format='%d-%B-%Y') + datetime.timedelta(days=1)).strftime(format='%d-%B-%Y')
        R = (pd.to_datetime(dt_col(data1)[-1], format='%d-%B-%Y') + datetime.timedelta(days=1))\
            .strftime(format='%d-%B-%Y')
        
        if R not in list(data.columns):
            
            Q = []
            
            for i in range(len(data)):
                
                row = data.loc[i]['Артикул']
                
                Q.append(data1[data1['Артикул']==row]['Цена'].values[0])

            data[R] = Q
            
        else:
            
            None
            
    
    return data

### Функция добавления признака "Последняя миля"

In [45]:
def last_mile(data):
    
    if 'Последняя миля' not in data:
    
        R = []

        for i in range(len(data)):
            if data['Цена'][i] * 0.055 <= 20:
                R.append(20)
            elif data['Цена'][i] * 0.055 >= 500:
                R.append(500)
            else:
                R.append(data['Цена'][i] * 0.055)

        data['Последняя миля'] = R
        
    else:
        
        None
        
    return data
    
    

### Добавление объема

In [46]:
# def add_volume(data):
    
#     volume = pd.read_csv(path_ozon_stocks_new, sep=';')[['Артикул', 'Объем товара, л']]
#     volume['Артикул'] = [x.replace("'", '') for x in list(volume['Артикул'])]
#     volume['Объем товара, л'] = [x.replace("'", '') for x in list(volume['Объем товара, л'])]
#     volume['Артикул'] = volume['Артикул'].astype(str)
#     volume['Объем товара, л'] = volume['Объем товара, л'].astype(float)
#     if len(volume) - len(list(set(volume['Артикул']))) == 0:
#         if 'Объем товара, л' not in list(data.columns):
#             data = data.merge(volume, on='Артикул', how='left')
#             data['Объем товара, л'] = data['Объем товара, л'].round(1)
#         else:
#             None
#     else:
#         data = 'Неуникальные артикулы в файле ozon_stocks_new'
        
#     return data

### Расчет скидки на логистику по индексу локализации

In [47]:
def oz_general_localization_index_discount(oz_general_localization_index):
    
    oz_general_localization_index = oz_general_localization_index - 5
    
    if oz_general_localization_index >= 0 and oz_general_localization_index <= 59:
        discount = 1.2
    elif oz_general_localization_index >= 60 and oz_general_localization_index <= 64:
        discount = 1.1
    elif oz_general_localization_index >= 65 and oz_general_localization_index <= 74:
        discount = 1.0
    elif oz_general_localization_index >= 75 and oz_general_localization_index <= 79:
        discount = 0.95
    elif oz_general_localization_index >= 80 and oz_general_localization_index <= 84:
        discount = 0.9
    elif oz_general_localization_index >= 85 and oz_general_localization_index <= 89:
        discount = 0.85
    elif oz_general_localization_index >= 90 and oz_general_localization_index <= 94:
        discount = 0.8
    elif oz_general_localization_index >= 95:
        discount = 0.5
    else:
        print('Ошибка в oz_general_localization_index')
        exit()
    
    return discount

### Редкатирование объема - добавление Объема товара в рублях

In [48]:
def redact_volume(data, flag='fbo', discount=1):
    if flag == 'fbo':
        if 'Объем товара, руб' not in data:
            R = []
            data['Объем товара, л'] = data['Объем товара, л'].round(2)
            for i in range(len(data)):
                if data['Объем товара, л'][i] >= 0 and data['Объем товара, л'][i] <= 1.9:
                    R.append(58)
                elif data['Объем товара, л'][i] > 1.9 and data['Объем товара, л'][i] <= 2.9:
                    R.append(61)
                elif data['Объем товара, л'][i] > 2.9 and data['Объем товара, л'][i] <= 4.9:
                    R.append(63)
                elif data['Объем товара, л'][i] > 4.9 and data['Объем товара, л'][i] <= 5.9:
                    R.append(67)
                elif data['Объем товара, л'][i] > 5.9 and data['Объем товара, л'][i] <= 6.9:
                    R.append(69)
                elif data['Объем товара, л'][i] > 6.9 and data['Объем товара, л'][i] <= 7.9:
                    R.append(71)
                elif data['Объем товара, л'][i] > 7.9 and data['Объем товара, л'][i] <= 8.4:
                    R.append(73)
                elif data['Объем товара, л'][i] > 8.4 and data['Объем товара, л'][i] <= 8.9:
                    R.append(75)
                elif data['Объем товара, л'][i] > 8.9 and data['Объем товара, л'][i] <= 9.4:
                    R.append(76)
                elif data['Объем товара, л'][i] > 9.4 and data['Объем товара, л'][i] <= 9.9:
                    R.append(77)
                elif data['Объем товара, л'][i] > 9.9 and data['Объем товара, л'][i] <= 14.9:
                    R.append(85)
                elif data['Объем товара, л'][i] > 14.9 and data['Объем товара, л'][i] <= 19.9:
                    R.append(111)
                elif data['Объем товара, л'][i] > 19.9 and data['Объем товара, л'][i] <= 24.9:
                    R.append(126)
                elif data['Объем товара, л'][i] > 24.9 and data['Объем товара, л'][i] <= 29.9:
                    R.append(141)
                elif data['Объем товара, л'][i] > 29.9 and data['Объем товара, л'][i] <= 34.9:
                    R.append(166)
                elif data['Объем товара, л'][i] > 34.9 and data['Объем товара, л'][i] <= 39.9:
                    R.append(191)
                elif data['Объем товара, л'][i] > 39.9 and data['Объем товара, л'][i] <= 44.9:
                    R.append(216)
                elif data['Объем товара, л'][i] > 44.9 and data['Объем товара, л'][i] <= 49.9:
                    R.append(231)
                elif data['Объем товара, л'][i] > 49.9 and data['Объем товара, л'][i] <= 54.9:
                    R.append(271)
                elif data['Объем товара, л'][i] > 54.9 and data['Объем товара, л'][i] <= 59.9:
                    R.append(296)
                elif data['Объем товара, л'][i] > 59.9 and data['Объем товара, л'][i] <= 64.9:
                    R.append(321)
                elif data['Объем товара, л'][i] > 64.9 and data['Объем товара, л'][i] <= 69.9:
                    R.append(356)
                elif data['Объем товара, л'][i] > 69.9 and data['Объем товара, л'][i] <= 74.9:
                    R.append(376)
                elif data['Объем товара, л'][i] > 74.9 and data['Объем товара, л'][i] <= 99.9:
                    R.append(406)
                elif data['Объем товара, л'][i] > 99.9 and data['Объем товара, л'][i] <= 124.9:
                    R.append(531)
                elif data['Объем товара, л'][i] > 124.9 and data['Объем товара, л'][i] <= 149.9:
                    R.append(706)
                elif data['Объем товара, л'][i] > 149.9 and data['Объем товара, л'][i] <= 174.9:
                    R.append(906)
                elif data['Объем товара, л'][i] > 174.9:
                    R.append(1106)
                else:
                    R.append(None)
            data['Объем товара, руб'] = R 
            data['Объем товара, руб'] = data['Объем товара, руб'] * discount
            
        else:
            None
    elif flag == 'fbs': 
        if 'Объем товара, руб' not in data:
            R = []
            data['Объем товара, л'] = data['Объем товара, л'].round(2)
            for i in range(len(data)):
                if data['Объем товара, л'][i] >= 0 and data['Объем товара, л'][i] <= 1.9:
                    R.append(70)
                elif data['Объем товара, л'][i] > 1.9 and data['Объем товара, л'][i] <= 2.9:
                    R.append(73)
                elif data['Объем товара, л'][i] > 2.9 and data['Объем товара, л'][i] <= 4.9:
                    R.append(76)
                elif data['Объем товара, л'][i] > 4.9 and data['Объем товара, л'][i] <= 5.9:
                    R.append(80)
                elif data['Объем товара, л'][i] > 5.9 and data['Объем товара, л'][i] <= 6.9:
                    R.append(83)
                elif data['Объем товара, л'][i] > 6.9 and data['Объем товара, л'][i] <= 7.9:
                    R.append(85)
                elif data['Объем товара, л'][i] > 7.9 and data['Объем товара, л'][i] <= 8.4:
                    R.append(88)
                elif data['Объем товара, л'][i] > 8.4 and data['Объем товара, л'][i] <= 8.9:
                    R.append(90)
                elif data['Объем товара, л'][i] > 8.9 and data['Объем товара, л'][i] <= 9.4:
                    R.append(91)
                elif data['Объем товара, л'][i] > 9.4 and data['Объем товара, л'][i] <= 9.9:
                    R.append(92)
                elif data['Объем товара, л'][i] > 9.9 and data['Объем товара, л'][i] <= 14.9:
                    R.append(102)
                elif data['Объем товара, л'][i] > 14.9 and data['Объем товара, л'][i] <= 19.9:
                    R.append(133)
                elif data['Объем товара, л'][i] > 19.9 and data['Объем товара, л'][i] <= 24.9:
                    R.append(151)
                elif data['Объем товара, л'][i] > 24.9 and data['Объем товара, л'][i] <= 29.9:
                    R.append(169)
                elif data['Объем товара, л'][i] > 29.9 and data['Объем товара, л'][i] <= 34.9:
                    R.append(199)
                elif data['Объем товара, л'][i] > 34.9 and data['Объем товара, л'][i] <= 39.9:
                    R.append(229)
                elif data['Объем товара, л'][i] > 39.9 and data['Объем товара, л'][i] <= 44.9:
                    R.append(259)
                elif data['Объем товара, л'][i] > 44.9 and data['Объем товара, л'][i] <= 49.9:
                    R.append(277)
                elif data['Объем товара, л'][i] > 49.9 and data['Объем товара, л'][i] <= 54.9:
                    R.append(325)
                elif data['Объем товара, л'][i] > 54.9 and data['Объем товара, л'][i] <= 59.9:
                    R.append(355)
                elif data['Объем товара, л'][i] > 59.9 and data['Объем товара, л'][i] <= 64.9:
                    R.append(385)
                elif data['Объем товара, л'][i] > 64.9 and data['Объем товара, л'][i] <= 69.9:
                    R.append(427)
                elif data['Объем товара, л'][i] > 69.9 and data['Объем товара, л'][i] <= 74.9:
                    R.append(451)
                elif data['Объем товара, л'][i] > 74.9 and data['Объем товара, л'][i] <= 99.9:
                    R.append(487)
                elif data['Объем товара, л'][i] > 99.9 and data['Объем товара, л'][i] <= 124.9:
                    R.append(637)
                elif data['Объем товара, л'][i] > 124.9:
                    R.append(847)
                else:
                    R.append(None)
            data['Объем товара, руб'] = R
        else:
            None  
    else:
        None
    return data

### Функция добавления "расчетных признаков" (новый алгоритм)

In [49]:


# 'Последяя миля':   
    
# for i in range(len(data)):
#     if data['Цена'][i] * 0.055 <= 20:
#         R.append(20)
#     elif data['Цена'][i] * 0.055 >= 500:
#         R.append(500)
#     else:
#         R.append(data['Цена'][i] * 0.055)


# 'Комиссия' = 'Цена' * 'Комиссия в процентах' / 100
# for i in range(len(data)):
#     if  i 'Комиссия' < 0.12:
#         'Комиссия' i  = 0.12
        
# 'Логистика' =  'Объем товара, руб' + 'Последняя миля'

# 'НДС' = (('Цена' * 0.2 / 1.2) - 
#                ('Себестоимость' * 0.2 / 1.2) - 
#                ('Комиссия' * 0.2 / 1.2) - 
#                ('Логистика' * 0.2 / 1.2))

# 'Налог на прибыль' = ((('Цена' - ('Цена' * 0.2 / 1.2)) - 
#                              ('Себестоимость' - ('Себестоимость' * 0.2 / 1.2)) - 
#                              ('Комиссия' - ('Комиссия' * 0.2 / 1.2)) - 
#                              ('Эквайринг') - 
#                              ('Логистика' - ('Логистика' * 0.2 / 1.2))) * 0.2)

# 'Налог' = 'НДС' + 'Налог на прибыль'

# 'Прибыль' = ('Цена' - 'Себестоимость' - 
#                'Комиссия' - 'Логистика' - 'Налог' - 'Эквайринг')

# 'Наценка' = 'Прибыль' / 'Себестоимость' * 100


In [50]:
def add_settlement_features_new(data, flag='fbo'):
    
    if flag == 'fbs':
        if 'Обработка' not in list(data.columns):
            data['Обработка'] = oz_shipment_processing
        else:
            None
    else:
        None
    
    if flag == 'fbs': 
        data['Комиссия в процентах'] = data['Комиссия в % fbs'] 
    elif flag == 'fbo':
        data['Комиссия в процентах'] = data['Комиссия в %']
    else:
        None
         
    if 'Комиссия' not in list(data.columns):
        data['Комиссия'] = data['Цена'] * data['Комиссия в процентах'] / 100
        for i in range(len(data)):
            if data.loc[i]['Комиссия'] < 0.12:
                data['Комиссия'].loc[i] = 0.12
    else:
        None
    
    if flag == 'fbo':   
        if 'Логистика' not in list(data.columns):
            data['Логистика'] = data['Объем товара, руб'] + data['Последняя миля']
        else:
            None
    elif flag == 'fbs':
        if 'Логистика' not in list(data.columns):
            data['Логистика'] = data['Объем товара, руб'] + data['Последняя миля'] + data['Обработка']
        else:
            None

    if 'НДС' not in list(data.columns):
        data['НДС'] = ((data['Цена'] * 0.2 / 1.2) - 
                       (data['Себестоимость'] * 0.2 / 1.2) - 
                       (data['Комиссия'] * 0.2 / 1.2) - 
                       (data['Логистика'] * 0.2 / 1.2))
    else:
        None
        
        
        
    if 'Налог на прибыль'  not in list(data.columns):
        data['Налог на прибыль'] = (((data['Цена'] - (data['Цена'] * 0.2 / 1.2)) - 
                                     (data['Себестоимость'] - (data['Себестоимость'] * 0.2 / 1.2)) - 
                                     (data['Комиссия'] - (data['Комиссия'] * 0.2 / 1.2)) - 
                                     (data['Эквайринг']) - 
                                     (data['Логистика'] - (data['Логистика'] * 0.2 / 1.2))) * 0.2)
    else:
        None
            
    if 'Налог' not in list(data.columns):
        data['Налог'] = data['НДС'] + data['Налог на прибыль']
    else:
        None
        
    if '% налога' not in list(data.columns):
        data['% налога'] = data['Налог'] * 100 / data['Цена']
    else:
        None
        
    if 'Прибыль' not in list(data.columns):        
        data['Прибыль'] = (data['Цена'] - data['Себестоимость'] - 
                           data['Комиссия'] - data['Логистика'] - data['Налог'] - data['Эквайринг'])
    else:
        None
        
          
    if 'Наценка' not in list(data.columns):
        data['Наценка'] = data['Прибыль'] / data['Себестоимость'] * 100
        data['Наценка'] = data['Наценка'].round(2)
    else:
        None
        
    return data

### Функция добавления всех "расчетных" признаков вместе (новый алгоритм)

In [51]:
def calc_features_fbo(data):
    data = data.merge(price.drop(['Цена', 'Баркод'], axis=1), on='Артикул', how='left')
    data = redact_volume(data, 
                         flag='fbo', 
                         discount=oz_general_localization_index_discount(oz_general_localization_index))
    data = last_mile(data)
    data = add_settlement_features_new(data, flag='fbo')
    return data

In [52]:
def calc_features_fbs(data):
    data = data.merge(price.drop(['Цена', 'Баркод'], axis=1), on='Артикул', how='left')
    data = redact_volume(data, 
                         flag='fbs', 
                         discount=1)
    data = last_mile(data)
    data = add_settlement_features_new(data, flag='fbs')
    return data

### Функция обработки основных метрик для презентации new

In [53]:
def table_for_presentation_new(data):
    
    if 'Объемный вес' in list(data.columns):
        data = data.drop('Объемный вес', axis=1)
    else:
        None
        
    if 'Объемный вес' in list(data.columns):
        data = data.drop('Объемный вес', axis=1)
    else:
        None
        
        
#     data = data.drop([
#      'Себестоимость',
#      'Эквайринг',
#      'Комиссия в %',
#      'Объемный вес в руб.',
#      'Последняя миля',
#      'Комиссия',
#      'Логистика',
#      'НДС',
#      'Налог на прибыль',
#      'Налог',
#      '% налога',
#      'Прибыль'
#                    ], axis=1)
    
    
    data = data.drop([
#      'Себестоимость',
#      'Эквайринг',
#      'Комиссия в %',
#      'Объем товара, руб',
#      'Объем товара, л',
#      'Последняя миля',
#      'Комиссия',
#      'Логистика',
#      'НДС',
#      'Налог на прибыль',
#      'Налог',
#      '% налога',
#      'Прибыль'
     'Себестоимость',
     'Эквайринг',
     'Комиссия в %',
     'Объем товара, руб',
     'Объем товара, л',
     'Последняя миля',
     'Комиссия',
     'Логистика',
     'НДС',
     'Налог на прибыль',
     'Налог',
     '% налога',
     'Прибыль',
     'Обработка отправления, максимум FBS',
     'Обработка отправления, минимум FBS',
     'Логистика Ozon, минимум, FBS',
     'Логистика Ozon, минимум, FBO',
     'Комиссия в % fbs',
     'Комиссия в процентах',
     'Логистика Ozon, максимум, FBO',
     'Логистика Ozon, максимум, FBS',
     'Цена до скидки'
        
                   ], axis=1)


    data = data.sort_values(by='Артикул')
    data['   '] = '   '

    J = 0

    for i in range(len(list(data.columns))):

        if len(re.findall('(\d+)', list(data.columns)[i])) == 2: 
            J = i

    data = data[list(data.columns)[:5] + list(data.columns)[J - 48: J + 1] + [list(data.columns)[-1]] + list(data.columns)[J +1:-1]]

    data2 = data.copy()
    data2 = data2.loc[1]
    data2['Артикул'] = 'Итого'
    data2[list(data.columns)[1]] = None
    data2[list(data.columns)[2]] = None
    data2['Баркод'] = None
    data2['ТМ'] = None
    data2['Цена'] = 0
    data2['Наценка'] = 0


    for row in list(data.columns)[5:-4]:
        data2[row] = sum(data[row])

    data2['   '] = '   '
    data2['Остаток'] = sum(data['Остаток'])
    data = data.append(data2, ignore_index=True)
    
    
    return data
    
    

### Функция добавления "промежуточных расчетов"

In [54]:
def add_intermediate_settlements(data):
    
    data1 = data.copy()
    z = list(data.iloc[:, 5:-4].columns)

    for i in range(len(z)):
        if pd.to_datetime(z[i]).dayofweek == 6 and i > 6:
            data1.insert((data1.columns.get_loc(z[i]))+1, f"{pd.to_datetime(z[i-6]):%d-%B}-{pd.to_datetime(z[i]):%d-%B}", 0)



    data = data1

    R = []
    for i in range(len(list(data.columns))):
        try:
            if len((re.findall('(\d+)',list(data.columns)[i]))[1]) != 4:
                R.append(list(data.columns)[i])

        except:
            None

    for row in R:
        T = []
        for i in range(len(data)):
            T.append((data.iloc[i, data.columns.get_loc(row)-7:data.columns.get_loc(row)]).sum())
  
        data[row] = T
    
    return data

### Функция добавления "РЦ"

In [55]:
def add_rc(data):
    
    rc = pd.read_excel(path_rc)
    
    if len(rc) - len(rc['Артикул'].unique()) != 0:
        
        return 'В РЦ есть неуникальные артикулы!'
    
    else:
        
        data = data.merge(rc, on='Артикул', how='left')
        
        data['Остаток на РЦ'] = data['Остаток на РЦ'].fillna(0)
        
        data['Остаток на РЦ'].loc[len(data)-1] = data['Остаток на РЦ'].sum()

        data = data.reset_index(drop=True)
        
        
        return data

### Функция добавления "Статусов"

In [56]:
# def add_st(data):
    
#     ozomfs = ozon_orders_metrics_for_status[['Артикул']+dt_col(ozon_orders_metrics_for_status)]
#     ozomfs = data[['Артикул']].merge(ozomfs, on='Артикул', how='left')
    
#     def add_st0(data):
    
#         if 'Статус' in list(data.columns):
#             data = data.drop('Статус', axis=1)
#         else:
#             None

#         st = pd.read_excel(path_st)

#         Q = []

#         for i in range(len(data)):
#             row = data.loc[i]['Артикул']
#             if data.loc[i]['Артикул'] in list(set(st['Артикул'])):

#                 Q.append(st[st['Артикул']==row]['Статус'].values[0])


#             else:
#                 Q.append('товары в пути')

#         Q[-1] = None

#         data['Статус'] = Q

#         data = data[list(data.columns[:5]) + ['Статус'] + list(data.columns[5:-1])]
        
#         return data
    
    
#     data = add_st0(data)
    
    
    
    
#     rc_irc = pd.read_excel(path_downloads + 'РЦ Иркутск.xlsx', 
#                            dtype={'Артикул': 'str'})
    
    
        
#     W = []
#     S = []
#     for i in range(len(data)):
#         if data.loc[i]['Статус'] == 'товары в пути':
#             if data.loc[i]['Остаток на РЦ'] > 0:
#                 W.append(data.loc[i]['Артикул'])
#             elif data.loc[i]['Артикул'] in list(set(rc_irc['Артикул'])):
#                 if rc_irc[rc_irc['Артикул']==data.loc[i]['Артикул']]['Остаток РЦ Иркутск'].values[0] > 0:
#                     W.append(data.loc[i]['Артикул'])
#             else:
#                 for row in dt_col(data)[::-1]:
#                     if data.loc[i][row] != 0:
#                         W.append(data.loc[i]['Артикул'])
                        
                        
#         elif data.loc[i]['Статус'] == 'новинка Даша':
#             if len([x for x in list(ozomfs.loc[i][dt_col(ozomfs)]) if x > 0]) >= 30:
#                 S.append(data.loc[i]['Артикул'])   
#         else:
#             None
            
            
                        
#     W = list(set(W))
#     S = list(set(S))
    
#     if len(W) > 0 or len(S) > 0:

#         st = pd.read_excel(path_st)
#         st['Артикул'] = st['Артикул'].astype(str)
#         st['Статус'] = st['Статус'].astype(str)
#         st = st.query('Артикул not in @W').reset_index(drop=True)
#         for row in W:
#             st.loc[len(st)] = [row, 'новинка Даша']

#         for row in S:
#             if row in list(set(st['Артикул'])):
#                 st.loc[st[st['Артикул']==row].index[0], 'Статус'] = 'текущий Дарья'
#             else:
#                 st.loc[len(st)] = [row, 'текущий Дарья']
            
#         st.to_excel(path_st, index=False) 
        
        
#         data = add_st0(data)
        
#     else:
#         None
    
    
    

    
#     return data

### Функция редактирования метрик с добавлением или изменением наименований

In [57]:
def redact_with_dop_name(data):
    
    dop_name = pd.read_excel(path_dop_name)
    
    dop_name['Артикул'] = dop_name['Артикул'].astype(str)
    
    dop_name['наименование'].loc[dop_name[dop_name['Артикул']=='18702'].index[0]] = 'Беспроводное зарядное устройство, белое'
    
    dop_name['наименование'].loc[dop_name[dop_name['Артикул']=='18703'].index[0]] = 'Беспроводное зарядное устройство, черное'
    
    for i in range(len(data)):
        row = data.loc[i]['Артикул']
    try:
        data['Номенклaтурa'].loc[i] = dop_name[dop_name['Артикул']==row]['FBO OZON SKU ID'].values[0]
        data['Баркод'].loc[i] = dop_name[dop_name['Артикул']==row]['Barcode'].values[0]
        data['ТМ'].loc[i] = dop_name[dop_name['Артикул']==row]['Бренд'].values[0]
        data['Название'].loc[i] = dop_name[dop_name['Артикул']==row]['наименование'].values[0]
    except:
        None
        
        
    return data




### Функция редактирования метрик по "объемам", добавление новых дней

In [58]:
def redact_ozon_size_metrics():
    
    ozon_size_metrics = pd.read_csv(path_ozon_size_metrics, sep=';')
    

    
    if 'Diff' in list(ozon_size_metrics.columns):
        
        ozon_size_metrics = ozon_size_metrics.drop('Diff', axis=1)
        
    else:
        
        None
    
    ozon_size = pd.read_csv(path_ozon_stocks_new, sep=';')
    
    
    ozon_size['Артикул'] = ozon_size['Артикул'].astype(str)
    ozon_size['Артикул'] = list(map(lambda x: x.replace("'", ""), list(ozon_size['Артикул'].values)))
    

    ozon_size = ozon_size[['Артикул', 'FBO OZON SKU ID', 'Barcode', 'Наименование товара', 'Бренд', 'Объем товара, л']]

    ozon_size = ozon_size.rename(columns={'FBO OZON SKU ID': 'Номенклaтурa', 
                                          'Barcode': 'Баркод', 
                                          'Наименование товара': 'Название', 
                                          'Бренд': 'ТМ'})
    
    ozon_size['Артикул'] = ozon_size['Артикул'].astype(str)

    ozon_size_2 = ozon_size.drop('Объем товара, л', axis=1).copy()

    ozon_size = ozon_size[['Артикул', 'Объем товара, л']]

    ozon_size['Объем товара, л'] = list(map(lambda x: x.replace("'", ""), ozon_size['Объем товара, л']))

    ozon_size['Объем товара, л'] = ozon_size['Объем товара, л'].astype(float)

    ozon_size = ozon_size.rename(columns={ozon_size.columns[-1]: pd.to_datetime(datetime.datetime.now()).strftime('%d-%B-%Y')})


    if ozon_size.columns[-1] in list(ozon_size_metrics.columns):

        ozon_size_metrics = ozon_size_metrics.drop(ozon_size.columns[-1], axis=1)

    else:

        None

    ozon_size_metrics = ozon_size_metrics.merge(ozon_size, on='Артикул', how='outer')


    for i in range(len(ozon_size_metrics)):

        row = ozon_size_metrics.loc[i]['Артикул']

        for col in ['Название', 'Номенклaтурa', 'Баркод', 'ТМ']:

            if ozon_size_metrics.loc[i:i][col].isna().values[0] == True:

                ozon_size_metrics[col].loc[i] = ozon_size_2[ozon_size_2['Артикул']==row][col].values[0]

            else:

                None
                
                
    D = ozon_size_metrics[ozon_size_metrics.columns[-1]] - ozon_size_metrics[ozon_size_metrics.columns[-2]]
    
    ozon_size_metrics['Diff'] = D
                
                
    return ozon_size_metrics

         

In [59]:
ozon_size = pd.read_csv(path_ozon_stocks_new, sep=';')
    
    
ozon_size['Артикул'] = ozon_size['Артикул'].astype(str)
ozon_size['Артикул'] = list(map(lambda x: x.replace("'", ""), list(ozon_size['Артикул'].values)))


ozon_size = ozon_size[['Артикул', 'FBO OZON SKU ID', 'Barcode', 'Наименование товара', 'Бренд', 'Объем товара, л']]

ozon_size = ozon_size.rename(columns={'FBO OZON SKU ID': 'Номенклaтурa', 
                                      'Barcode': 'Баркод', 
                                      'Наименование товара': 'Название', 
                                      'Бренд': 'ТМ'})

ozon_size['Артикул'] = ozon_size['Артикул'].astype(str)

ozon_size_2 = ozon_size.drop('Объем товара, л', axis=1).copy()

ozon_size = ozon_size[['Артикул', 'Объем товара, л']]

ozon_size['Объем товара, л'] = list(map(lambda x: x.replace("'", ""), ozon_size['Объем товара, л']))

ozon_size['Объем товара, л'] = ozon_size['Объем товара, л'].astype(float)

ozon_size = ozon_size.rename(columns={ozon_size.columns[-1]: pd.to_datetime(datetime.datetime.now()).strftime('%d-%B-%Y')})


### Функция работы с "ТМ"

In [60]:
def add_and_redact_brend(data):


    
    def yyy_brend(y):
        try:
            if '0' in y:
                y = 'no_name'
            elif '\xa0' in y:
                y = y.replace('\xa0', '')
            elif '/' in y:
                y = y.replace('/', '')
            elif 'Limpid Side' in y:
                y = 'Limpide Side'
            elif 'REVOLUY' in y:
                y = 'REVOLUT'
            elif 'DENT & DONT' in y:
                y = 'DENT & DON’T'
            elif 'Revolut' in y:
                y = 'REVOLUT'
            elif 'Нет бренда' in y:
                y = 'no_name'
            elif 'nan' in y:
                y = 'no_name'
            else:
                None
        except:
            None
            
        return y
    
    
    dfb = pd.read_csv(path_ozon_stocks_new, sep=';')
    dfb = dfb[['Артикул', 'Бренд']]
    dfb['Артикул'] = dfb['Артикул'].astype(str)
    dfb['Бренд'] = dfb['Бренд'].astype(str)
    dfb = dfb.rename(columns={'Бренд': 'ТМ'})
    dfb = dfb.dropna().reset_index(drop=True)
    dfb['ТМ'] = list(map(lambda x: yyy_brend(x), list(dfb['ТМ'])))
    dfb['ТМ'] = list(map(lambda x: yyy_brend(x), list(dfb['ТМ'])))

    
    flag = 0
    if 'Бренд' in list(data.columns):
        flag = 1
        data = data.rename(columns={'Бренд': 'ТМ'})
    else:
        None
    

    data['ТМ'] = data['ТМ'].fillna('no_name') 
    data['Артикул'] = data['Артикул'].astype(str)
    data['ТМ'] = data['ТМ'].astype(str)
    data['ТМ'] = list(map(lambda x: yyy_brend(x), list(data['ТМ'])))
    data['ТМ'] = list(map(lambda x: yyy_brend(x), list(data['ТМ'])))

    for i in range(len(data)):
        if 'Итого' not in data.loc[i]['Артикул']:
            if data.loc[i]['ТМ'] == 'no_name':
                if data.loc[i]['Артикул'] in list(set(dfb['Артикул'])):
                    data['ТМ'].loc[i] = dfb[dfb['Артикул']==data.loc[i]['Артикул']]['ТМ'].values[0]
                else:
                    None
            else:
                None    
        else:
            data['ТМ'].loc[i] = None
    
    
    if flag == 1:
        data = data.rename(columns={'ТМ': 'Бренд'})
    else:
        None
        
    return data

### Функция наценки для акций

In [61]:
# def add_settlement_features_ac(data, col_price, proc):
    
    
#     data['Комиссия'] = data['Комиссия'] - proc


        
#     NDS = ((data[col_price] * 0.2 / 1.2) - 
#                    (data['Себестоимость'] * 0.2 / 1.2) - 
#                    (data['Комиссия'] * 0.2 / 1.2) - 
#                    (data['Логистика'] * 0.2 / 1.2))



#     NNP = (((data[col_price] - (data[col_price] * 0.2 / 1.2)) - 
#                                  (data['Себестоимость'] - (data['Себестоимость'] * 0.2 / 1.2)) - 
#                                  (data['Комиссия'] - (data['Комиссия'] * 0.2 / 1.2)) - 
#                                  (data['Эквайринг']) - 
#                                  (data['Логистика'] - (data['Логистика'] * 0.2 / 1.2))) * 0.2)

  

        
        
        
        

        
#     N = NDS + NNP
        

        
        

#     P = (data[col_price] - data['Себестоимость'] - 
#                        data['Комиссия'] - data['Логистика'] - N - data['Эквайринг'])
 
 
        
        
        
        

        
#     NA = P / data['Себестоимость'] * 100

#     NA = NA.round(2)
 
        
        
#     return NA
        
    

## Работа с OZON метриками

### Загрузка "исторических" метрик

In [62]:
ozon_orders_metrics = pd.read_csv(path_ozon_orders_metrics, sep=';')
# ozon_orders_metrics = pd.read_csv(path_ozon_orders_metrics, sep='\t')

In [63]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,29-March-2024,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,0,1,0,1,0,0,0,5,4,4
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,0,0,0,0,1,0,2,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0




### Загрузка остатков по дню

In [64]:
ozon_stocks_new = pd.read_csv(path_ozon_stocks_new, sep=';')
ozon_stocks_new

,Артикул,Ozon Product ID,FBO OZON SKU ID,FBS OZON SKU ID,Barcode,Наименование товара,Контент-рейтинг,Бренд,Статус товара,Видимость FBO,...,"Зарезервировано, шт","Доступно к продаже по схеме FBS, шт.","Доступно к продаже по схеме realFBS, шт.","Зарезервировано на моих складах, шт","Текущая цена с учетом скидки, ₽","Цена до скидки (перечеркнутая цена), ₽","Цена Premium, ₽","Рыночная цена, ₽",Актуальная ссылка на рыночную цену,"Размер НДС, %"
0,'16803,1211889,149314896,282996254,7432141949932,"SSY, Защитный чехол книжка для iPad 10.2 (2019...",'70,SSY,Готов к продаже,Показывается,...,0,0,0,0,946.0,2996.0,NaN,NaN,NaN,20%
1,'18309,1212145,149314939,149314939,7432141050065,"SSY, Чехол для наушников Apple AirPods 1, 2, К...",'70,SSY,Готов к продаже,Показывается,...,0,0,0,0,160.0,1010.0,NaN,NaN,NaN,20%
2,'18308,1214799,149314941,149314941,7432141050058,"SSY, Чехол для наушников Apple AirPods 1, 2, К...",'70,SSY,Готов к продаже,Показывается,...,0,0,0,0,180.0,1010.0,NaN,NaN,NaN,20%
3,'18307,1214800,149295199,149295199,7432141649689,"SSY, Чехол для наушников Apple AirPods 1, 2, К...",'70,SSY,Продается,Показывается,...,0,0,0,0,150.0,1010.0,NaN,NaN,NaN,20%
4,'18306,1214801,149459576,149459576,7432141649672,"SSY, Чехол для наушников Apple AirPods 1, 2, К...",'70,SSY,Продается,Показывается,...,1,1,0,0,160.0,1010.0,NaN,NaN,NaN,20%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222,'50976,836170214,1388744176,1388744176,4673660046877,"Birdhouse, Наклейки на стену для декора / Зерк...",'45,Birdhouse,Продается,Показывается,...,6,191,0,0,460.0,1010.0,NaN,NaN,NaN,20%
1223,'A5403,842208794,1393960786,1393960786,4673766212725,"Shiny Kitchen, Хлебница деревянная с прозрачно...",'30,Shiny Kitchen,Готов к продаже,Показывается,...,0,0,0,0,400.0,1010.0,NaN,NaN,NaN,20%
1224,'50458,929948512,1469117395,1469117395,4674076931641,"REVOLUT, Расческа для волос массажная складная...",'70,REVOLUT,Готов к продаже,Показывается,...,0,0,0,0,590.0,1010.0,NaN,NaN,NaN,20%
1225,'50459,929965852,1469118732,1469118732,4674382511209,Расческа для волос с зеркалом,'70,REVOLUT,Готов к продаже,Показывается,...,0,0,0,0,590.0,1010.0,NaN,NaN,NaN,20%


In [65]:
ozon_stocks_new['Артикул'] = ozon_stocks_new['Артикул'].astype(str)
ozon_stocks_new['Артикул'] = list(map(lambda x: x.replace("'", ""), list(ozon_stocks_new['Артикул'].values)))

### Добавление новых товаров из остатков по дню в основные метрики (если такие товары появились)

In [66]:
ozon_orders_metrics667 = ozon_orders_metrics.copy()

In [67]:
ozon_orders_metrics = add_new_art_in_ozon_orders_metrics(ozon_orders_metrics, ozon_stocks_new)

### Проверка 

In [68]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,29-March-2024,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,0,1,0,1,0,0,0,5,4,4
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,0,0,0,0,1,0,2,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
ozon_orders_metrics667 = None

### Загрузка метрик по остаткам и ценам

In [70]:
stocks_ozon_metrics = pd.read_csv(path_stocks_ozon_metrics, sep=';')
# stocks_ozon_metrics = pd.read_csv(path_stocks_ozon_metrics, sep='\t')

In [71]:
ozon_price_metrics = pd.read_csv(path_ozon_price_metrics, sep=';')
# ozon_price_metrics = pd.read_csv(path_ozon_price_metrics, sep='\t')

In [72]:
ozon_price_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,26-August-2022,29-August-2022,30-August-2022,31-August-2022,01-September-2022,...,26-March-2024,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,648,648,648,648,648,...,836,836,836,836,836,836,836,585,585,585
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,166,166,166,166,166,...,160,160,160,160,160,160,160,160,160,160
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,198,198,198,198,198,...,180,180,180,180,180,180,180,180,126,126
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,198,198,198,198,198,...,142,142,142,150,150,150,150,150,105,105
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,175,175,175,175,175,...,148,148,148,148,148,148,148,148,104,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,400,400,400,400,400,400,400,400,400,400
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,575,575,575,552,552,552,552,552,386,386
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,590,590,590,590,590,590,590,590,590,590
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,1824,1824,1824,1824,1824,1824,1824,1824,1824,1824


### Дополнение метрик по ценам и оcтаткам новыми товарами, если такие повились в "загрузках"

In [73]:
ozon_price_metrics, stocks_ozon_metrics = add_new_art_for_ozon_price_and_ozon_stocks_metrics(ozon_orders_metrics, 
                                                                                             ozon_price_metrics, 
                                                                                             stocks_ozon_metrics)



### Загрузка заказов по дню

In [74]:
ozon_orders_metrics667 = ozon_orders_metrics.copy()

In [75]:
paths

['/Users/dmitriymaslov/Мой диск/Загрузки Маслов/orders (28).csv']

In [76]:
if post == False:    
    for e in range(len(paths)):

        ozon_orders_new = pd.read_csv(paths[e], sep=';')

    #     ozon_orders_new_fbs = pd.read_csv(paths_fbs[e], sep=';')

        ozon_orders_new = redact_ozon_orders_new(ozon_orders_new)

    #     ozon_orders_new_fbs = redact_ozon_orders_new(ozon_orders_new_fbs)


        ozon_orders_new['Артикул'] = ozon_orders_new['Артикул'].astype('str')
    #     ozon_orders_new_fbs['Артикул'] = ozon_orders_new_fbs['Артикул'].astype('str')



    #     if ozon_orders_new_fbs.columns[0] in list(ozon_orders_new.columns):

    #         ozon_orders_new = pd.concat([ozon_orders_new, ozon_orders_new_fbs], ignore_index=True).groupby('Артикул')[ozon_orders_new.columns[0]].sum().reset_index(drop=False)

    #         ozon_orders_new = ozon_orders_new[[ozon_orders_new.columns[-1]]+['Артикул']]



    #         ## Подало в ошибку, добавил еще одну группировку
    #         ozon_orders_new = ozon_orders_new.groupby('Артикул')[ozon_orders_new.columns[0]].sum().reset_index(drop=False)

    #         ozon_orders_new = ozon_orders_new[[ozon_orders_new.columns[-1]]+['Артикул']]

    #     else:

    #         print('Ошибка в файле с заказами fbs')

    #         ozon_orders_new = 'error'


        ozon_orders_metrics = add_ozon_orders_new_from_ozon_orders_metrics(ozon_orders_metrics, ozon_orders_new)

elif post == True:
    
    
    for e in range(len(paths)):

        ozon_orders_new = pd.read_csv(paths[e], sep=';')

        ozon_orders_new_fbs = pd.read_csv(paths_fbs[e], sep=';')

        ozon_orders_new = redact_ozon_orders_new(ozon_orders_new)

        ozon_orders_new_fbs = redact_ozon_orders_new(ozon_orders_new_fbs)


        ozon_orders_new['Артикул'] = ozon_orders_new['Артикул'].astype('str')
        ozon_orders_new_fbs['Артикул'] = ozon_orders_new_fbs['Артикул'].astype('str')



        if ozon_orders_new_fbs.columns[0] in list(ozon_orders_new.columns):

            ozon_orders_new = pd.concat([ozon_orders_new, ozon_orders_new_fbs], ignore_index=True).groupby('Артикул')[ozon_orders_new.columns[0]].sum().reset_index(drop=False)

            ozon_orders_new = ozon_orders_new[[ozon_orders_new.columns[-1]]+['Артикул']]



            ## Подало в ошибку, добавил еще одну группировку
            ozon_orders_new = ozon_orders_new.groupby('Артикул')[ozon_orders_new.columns[0]].sum().reset_index(drop=False)

            ozon_orders_new = ozon_orders_new[[ozon_orders_new.columns[-1]]+['Артикул']]

        else:

            print('Ошибка в файле с заказами fbs')

            ozon_orders_new = 'error'


        ozon_orders_metrics = add_ozon_orders_new_from_ozon_orders_metrics(ozon_orders_metrics, ozon_orders_new)
    
    

In [77]:
# ozon_orders_new = ozon_orders_new.groupby('Артикул')[ozon_orders_new.columns[0]].sum().reset_index(drop=False)

# ozon_orders_new = ozon_orders_new[[ozon_orders_new.columns[-1]]+['Артикул']]
ozon_orders_new

,08-April-2024,Артикул
0,1,16920
1,6,16931
2,3,16950
3,2,16951
4,2,16952
...,...,...
640,1,Z0247
641,2,Z0400
642,1,Z0401
643,2,Z0500


In [78]:
list(ozon_orders_new.columns)

['08-April-2024', 'Артикул']

### Проверка

In [79]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,1,0,1,0,0,0,5,4,4,7
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,0,0,0,1,0,2,4,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Сортировка по дате

In [80]:
ozon_orders_metrics = data_columns_sorted(ozon_orders_metrics)

### Проверка

In [81]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,1,0,1,0,0,0,5,4,4,7
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,0,0,0,1,0,2,4,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
ozon_orders_metrics_for_status = ozon_orders_metrics.copy()

In [83]:
ozon_orders_metrics667 = None

### Добавление или изменение имен у метрик

In [84]:
ozon_orders_metrics667 = ozon_orders_metrics.copy()

In [85]:
ozon_orders_metrics = redact_with_dop_name(ozon_orders_metrics)

In [86]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,1,0,1,0,0,0,5,4,4,7
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,0,0,0,1,0,2,4,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
ozon_orders_metrics667 = None

In [88]:
ozon_orders_metrics_non_orders = ozon_orders_metrics.copy()

### Добавление "Остатков" к метрикам основным

In [89]:
ozon_orders_metrics667 = ozon_orders_metrics.copy()

In [90]:
ozon_stocks_new[['Артикул', 'Доступно к продаже по схеме FBO, шт.']].dropna().reset_index(drop=True)

,Артикул,"Доступно к продаже по схеме FBO, шт."
0,16803,0
1,18309,0
2,18308,0
3,18307,99
4,18306,222
...,...,...
1222,50976,385
1223,A5403,0
1224,50458,0
1225,50459,0


In [91]:
ozon_orders_metrics = redact_and_add_ozon_stocks_new_from_ozon_orders_metrics(ozon_orders_metrics, ozon_stocks_new)

### Проверка

In [92]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,0,1,0,0,0,5,4,4,7,99
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,0,0,1,0,2,4,1,0,4,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,100
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
ozon_orders_metrics667 = None

### Сведение метрик остатков с новыми остатками

In [94]:
stocks_ozon_metrics = add_ozon_stocks_new_from_stocks_ozon_metric(ozon_orders_metrics, stocks_ozon_metrics)

### Проверка

In [95]:
stocks_ozon_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,26-August-2022-Ost,29-August-2022-Ost,30-August-2022-Ost,31-August-2022-Ost,01-September-2022-Ost,...,27-March-2024-Ost,28-March-2024-Ost,29-March-2024-Ost,01-April-2024-Ost,02-April-2024-Ost,03-April-2024-Ost,04-April-2024-Ost,05-April-2024-Ost,08-April-2024-Ost,09-April-2024-Ost
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,20,20,50,50,48,...,0,0,0,0,0,0,0,0,0,0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,146,118,311,297,285,...,0,0,0,0,0,0,0,0,0,0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,275,272,272,272,271,...,0,0,0,0,0,0,0,0,0,0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,232,214,208,204,199,...,125,123,122,121,120,120,120,119,106,99
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,328,325,325,325,322,...,236,236,236,236,236,235,235,233,228,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,100,100,100,100,100,100,100,100,100,100
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Загрузка "Себестоимости" и сведение с метриками основными

In [96]:
ozon_orders_metrics = add_cost_price(ozon_orders_metrics)

### Загрузка "Цены" и сведение с метриками основными

In [97]:
ozon_orders_metrics = add_price(ozon_orders_metrics)

In [98]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Себестоимость,Цена
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,243.71,585.0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,50.42,160.0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,45.37,126.0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,0,0,0,5,4,4,7,99,34.27,105.0
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,1,0,2,4,1,0,4,222,47.96,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,400.0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,100,317.29,386.0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,590.0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,1824.0


### Загрузка новых цен в метрики по ценам 

In [99]:
ozon_price_metrics = add_price_from_ozon_price_metric(ozon_price_metrics, ozon_orders_metrics)

ozon_price_metrics = data_columns_sorted(ozon_price_metrics)

### Проверка 

In [100]:
ozon_price_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,26-August-2022,29-August-2022,30-August-2022,31-August-2022,01-September-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,648,648,648,648,648,...,836,836,836,836,836,836,585,585,585,585.0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,166,166,166,166,166,...,160,160,160,160,160,160,160,160,160,160.0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,198,198,198,198,198,...,180,180,180,180,180,180,180,126,126,126.0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,198,198,198,198,198,...,142,142,150,150,150,150,150,105,105,105.0
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,175,175,175,175,175,...,148,148,148,148,148,148,148,104,104,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,400,400,400,400,400,400,400,400,400,400.0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,575,575,552,552,552,552,552,386,386,386.0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,590,590,590,590,590,590,590,590,590,590.0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,1824,1824,1824,1824,1824,1824,1824,1824,1824,1824.0


In [101]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Себестоимость,Цена
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,243.71,585.0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,50.42,160.0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,45.37,126.0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,0,0,0,5,4,4,7,99,34.27,105.0
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,1,0,2,4,1,0,4,222,47.96,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,400.0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,100,317.29,386.0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,590.0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,1824.0


In [102]:
# # ozon_orders_metrics_fbo_new = ozon_orders_metrics.copy()
# ozon_orders_metrics_fbs_new = ozon_orders_metrics.copy()

In [103]:
# try:
#     oz_calc_f_fbs(ozon_orders_metrics_fbs_new, flag='fbs')\
# .to_excel(path_downloads + 'Расчетные признаки по озон на этапе вычисления наценки.xlsx', index=False)
# except:
#     oz_calc_f_fbs(ozon_orders_metrics_fbs_new, flag='fbs')\
# .to_excel(path_downloads + 'Расчетные признаки по озон на этапе вычисления наценки.xlsx', index=False)

### Расчет наценки по новому алгоритму и алгоритму для FBS 

In [104]:
# # ozon_orders_metrics_fbo_new = calc_features_new(ozon_orders_metrics_fbo_new)
# ozon_orders_metrics_fbs_new = calc_features_new(ozon_orders_metrics_fbs_new, 'fbs')

In [105]:
# # ozon_orders_metrics_fbo_new = ozon_orders_metrics_fbo_new[['Артикул', 'Наценка']]
# # ozon_orders_metrics_fbo_new.columns = ['Артикул', 'Наценка_новая']
# ozon_orders_metrics_fbs_new = ozon_orders_metrics_fbs_new[['Артикул', 'Наценка']]
# ozon_orders_metrics_fbs_new.columns = ['Артикул', 'Наценка_fbs']

In [106]:
ozon_orders_metrics_fbs_new = calc_features_fbs(ozon_orders_metrics.copy())[['Артикул', 'Наценка']].rename(columns={'Наценка': 'Наценка_fbs'})


### Добавление расчетных признаков

In [107]:
# ozon_orders_metrics_for_dop_margin = ozon_orders_metrics[ozon_orders_metrics['Артикул']=='20800'].copy().reset_index(drop=True)
# ozon_orders_metrics_for_dop_margin.loc[0, ['Цена']] = 329
# ozon_orders_metrics_for_dop_margin = calc_features_new(ozon_orders_metrics_for_dop_margin)
# ozon_orders_metrics_for_dop_margin.loc[0]['Наценка']

In [108]:
ozon_orders_metrics_fbs_new

,Артикул,Наценка_fbs
0,16803,24.27
1,18309,-38.11
2,18308,-72.91
3,18307,-106.26
4,18306,-95.80
...,...,...
1226,A5403,inf
1227,28900,-24.27
1228,50459,inf
1229,A8304,inf


In [109]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Себестоимость,Цена
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,243.71,585.0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,50.42,160.0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,45.37,126.0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,0,0,0,5,4,4,7,99,34.27,105.0
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,1,0,2,4,1,0,4,222,47.96,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,400.0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,100,317.29,386.0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,590.0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,1824.0


In [110]:
# data = ozon_orders_metrics.copy()

In [111]:
# data = data.merge(price.drop(['Цена', 'Баркод'], axis=1), on='Артикул', how='left')

In [112]:
# data['Комиссия в %'] = data['Комиссия в %'] + 5

In [113]:
# data = redact_volume(data, 
#                      flag='fbo', 
#                      discount=oz_general_localization_index_discount(oz_general_localization_index))
# data = last_mile(data)
# data = add_settlement_features_new(data, flag='fbo')

In [114]:
# data[['Артикул', 'Наценка']].rename(columns={'Наценка': 'Нацена в случае потери скидки'})\
# .merge(ozon_orders_metrics[['Артикул', 'Наценка']], on='Артикул', how='outer')\
# .merge(pd.read_excel(path_downloads + 'Сравнение комиссий на озон (1).xlsx',
#               dtype={'Артикул': 'str'})[['Артикул']], on='Артикул', how='inner')[['Артикул', 'Наценка', 'Нацена в случае потери скидки']]\
# .to_excel(path_downloads + 'Наценка озон в случае потери скидки.xlsx', index=False)

In [115]:
# ozon_orders_metrics[['Артикул', 'Наценка']]

In [116]:
# pd.read_excel(path_downloads + 'Сравнение комиссий на озон (1).xlsx',
#               dtype={'Артикул': 'str'})[['Артикул']]

In [117]:
# calc_features_fbo(ozon_orders_metrics.drop('Цена', axis=1).merge(pd.read_excel(path_downloads + 'Озон_для ИУ.xlsx',
#                                                              dtype={'Артикул': 'str'}),
#                                               on='Артикул',
#                                               how='inner').rename(columns={'Цена 16.11': 'Цена'}))\
# [['Артикул', 'Цена', 'Наценка']].rename(columns={'Цена': 'Цена 16.11'})\
# .to_excel(path_downloads + 'Наценка озон 16.11.xlsx', index=False)

In [118]:
# def add_settlement_features_new(data, flag='fbo'):
    
#     if flag == 'fbs':
#         if 'Обработка' not in list(data.columns):
#             data['Обработка'] = oz_shipment_processing
#         else:
#             None
#     else:
#         None
    
#     if flag == 'fbs': 
#         data['Комиссия в процентах'] = data['Комиссия в % fbs'] 
#     elif flag == 'fbo':
#         data['Комиссия в процентах'] = data['Комиссия в %']
#     else:
#         None
         
#     if 'Комиссия' not in list(data.columns):
#         data['Комиссия'] = data['Цена'] * data['Комиссия в процентах'] / 100
#         for i in range(len(data)):
#             if data.loc[i]['Комиссия'] < 0.12:
#                 data['Комиссия'].loc[i] = 0.12
#     else:
#         None
    
#     if flag == 'fbo':   
#         if 'Логистика' not in list(data.columns):
#             data['Логистика'] = data['Объем товара, руб'] + data['Последняя миля']
#         else:
#             None
#     elif flag == 'fbs':
#         if 'Логистика' not in list(data.columns):
#             data['Логистика'] = data['Объем товара, руб'] + data['Последняя миля'] + data['Обработка']
#         else:
#             None

#     if 'НДС' not in list(data.columns):
#         data['НДС'] = ((data['Цена'] * 0.2 / 1.2) - 
#                        (data['Комиссия'] * 0.2 / 1.2) - 
#                        (data['Логистика'] * 0.2 / 1.2))
#     else:
#         None
        
        
        
#     if 'Налог на прибыль'  not in list(data.columns):
#         data['Налог на прибыль'] = (((data['Цена'] - (data['Цена'] * 0.2 / 1.2)) - 
#                                      (data['Комиссия'] - (data['Комиссия'] * 0.2 / 1.2)) - 
#                                      (data['Эквайринг']) - 
#                                      (data['Логистика'] - (data['Логистика'] * 0.2 / 1.2))) * 0.2)
#     else:
#         None
            
#     if 'Налог' not in list(data.columns):
#         data['Налог'] = data['НДС'] + data['Налог на прибыль']
#     else:
#         None
        
#     if '% налога' not in list(data.columns):
#         data['% налога'] = data['Налог'] * 100 / data['Цена']
#     else:
#         None
        
#     if 'Прибыль' not in list(data.columns):        
#         data['Прибыль'] = (data['Цена'] - 
#                            data['Комиссия'] - data['Логистика'] - data['Налог'] - data['Эквайринг'])
#     else:
#         None
        
          
# #     if 'Наценка' not in list(data.columns):
# #         data['Наценка'] = data['Прибыль'] / data['Себестоимость'] * 100
# #         data['Наценка'] = data['Наценка'].round(2)
# #     else:
# #         None
        
#     return data

In [119]:
if flag_stop_nac == True:
    exit()

In [120]:
def nac_pr_oz(art, price):
    data = ozon_orders_metrics[ozon_orders_metrics['Артикул']==art].reset_index(drop=True).copy()
    data.loc[0, 'Цена'] = price
    data = calc_features_fbo(data)[['Артикул', 'Цена', 'Наценка']]
    
    return data
    
    

In [121]:
# nac_pr_oz('50200', 20000)

In [122]:
# oz = mwm.select_table('dds_oz_orders', schema=mwm.dds_schema)
# oz = wwm.expand_df(oz[(oz['date'] >= datetime.date(2024, 1, 1)) \
#                       & (oz['date'] <= datetime.date(2024, 1, 31)) \
#                       & (oz['account']=='ir')].drop('account', axis=1).reset_index(drop=True),
#                    key_column='item_code',
#                    result_column='orders')
# date_cols = dt_col(oz)
# oz = oz.query(f"index in {[x for x in list(oz.index) if oz.iloc[x, 1:].sum() > 0]}")\
# .reset_index(drop=True)
# oz['sum_orders'] = [oz.iloc[x, 1:].sum() for x in list(oz.index)]
# oz = oz.rename(columns={'item_code': 'Артикул'})

# ozp = pd.read_excel(wwm.path_full_OZON_df,
#                     sheet_name='ozon_price_metrics',
#                     dtype={'Баркод': 'int',
#                            'Артикул': 'str'})

# ozp = ozp[['Артикул']+[x for x in list(ozp.columns) if x in date_cols]]

# ozp['01-January-2024'] = ozp['02-January-2024']

# ozp['03-January-2024'] = ozp['02-January-2024']
# ozp['04-January-2024'] = ozp['02-January-2024']
# ozp['05-January-2024'] = ozp['02-January-2024']
# ozp['06-January-2024'] = ozp['02-January-2024']

# ozp['06-January-2024'] = ozp['07-January-2024']
# ozp['08-January-2024'] = ozp['07-January-2024']

# ozp['13-January-2024'] = ozp['12-January-2024']
# ozp['14-January-2024'] = ozp['12-January-2024']

# ozp['20-January-2024'] = ozp['19-January-2024']
# ozp['21-January-2024'] = ozp['19-January-2024']

# ozp['27-January-2024'] = ozp['26-January-2024']
# ozp['28-January-2024'] = ozp['26-January-2024']

# ozp = ozp[['Артикул']+[y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in dt_col(ozp)])]]

# oz = oz.sort_values('Артикул').reset_index(drop=True)

# ozp = ozp[['Артикул']+[y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in dt_col(ozp)])]]
# ozp.iloc[:, 1:] = ozp.iloc[:, 1:].astype(int)
# ozp['Артикул'] = ozp['Артикул'].astype(str)
# ozp = ozp[ozp['Артикул'].isin(list(set(oz['Артикул'])))].reset_index(drop=True)

# for row in date_cols:
#     if row not in ozp.columns:
#         print(row)
# for row in date_cols:
#     if row not in ozp.columns:
#         print(row)
# ozp = ozp[['Артикул']+date_cols]
# mwm.unique_items(ozp)
# ozp['mean_price'] = [round(np.mean([y for y in list(ozp.iloc[x, 1:]) if y > 0])) for x in list(ozp.index)]
# ozp = ozp.sort_values('Артикул').reset_index(drop=True)
# mwm.unique_items(ozp)

# ozv = oz[['Артикул']].copy()
# for col in date_cols:
#     ozv[col] = None
# for i in range(len(ozv)):
#     for col in date_cols:
#         ozv.loc[i, col] = oz.loc[i][col] * ozp.loc[i][col]
# for row in list(set(ozp['Артикул'])):
#     if row not in list(set(oz['Артикул'])):
#         print(row)
# ozv['sum_earnings'] = [ozv.iloc[x, 1:].sum() for x in list(ozv.index)]
# mwm.unique_items(ozv)
# ozv = ozv[ozv['Артикул'].isin(list(set(oz['Артикул'])))].reset_index(drop=True)
# ozv = ozv.sort_values('Артикул').reset_index(drop=True)

# cp = mwm.select_table_with_last_date_sort_list('cost_price', 'date')
# cp['Артикул'] = cp['Артикул'].astype('str')
# cp = cp[['Артикул', 'Себестоимость']]
# cp = cp[cp['Себестоимость']>0].reset_index(drop=True)
# cp = mwm.clear_article(cp, 'Артикул').reset_index(drop=True)
# Q = []
# for i in range(len(cp)):
#     art = cp.loc[i]['Артикул']
#     if len(cp[cp['Артикул']==art])> 1:
#         sub_df = cp[cp['Артикул']==art]
#         sub_df = sub_df.sort_values('Себестоимость')
#         for ind in list(sub_df.index)[:-1]:
#             Q.append(ind)
# Q = list(set(Q))
# cp = cp.query('index not in @Q').reset_index(drop=True)
# mwm.unique_items(cp)
# cp = cp[cp['Артикул'].isin(list(set(oz['Артикул'])))].sort_values('Артикул').reset_index(drop=True)
# for col in date_cols:
#     cp[col] = None
#     for i in range(len(cp)):
#         cp.loc[i, col] = round((cp.loc[i]['Себестоимость'] * oz.loc[i][col]), 2)
# cp['sum_cost_price'] = [cp.iloc[x, 2:].sum() for x in list(cp.index)]

# oz_dop = oz[['Артикул']].copy()
# for col in date_cols:
#     sub_df = calc_features_fbo(cp[['Артикул', 'Себестоимость']].merge(ozp[['Артикул', col]]\
#                                                                       .rename(columns={col: 'Цена'}).copy(),
#                                                                       on='Артикул', 
#                                                                       how='left'))[['Артикул', 
#                                                                                     'Логистика', 
#                                                                                     'Комиссия', 
#                                                                                     'Эквайринг', 
#                                                                                     'НДС', 
#                                                                                     'Налог на прибыль']]\
#     .rename(columns={'Логистика': col + ' Логистика',
#                      'Комиссия': col + ' Комиссия',
#                      'Эквайринг': col + ' Эквайринг',
#                      'НДС': col + ' НДС',
#                      'Налог на прибыль': col + ' Налог на прибыль'})
#     oz_dop = oz_dop.merge(sub_df, on='Артикул', how='left')
    
# oz_logistic = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Логистика' in x]].copy()
# oz_logistic.columns = ['Артикул'] + date_cols
# oz_komm = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Комиссия' in x]].copy()
# oz_komm.columns = ['Артикул'] + date_cols
# oz_ekv = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Эквайринг' in x]].copy()
# oz_ekv.columns = ['Артикул'] + date_cols
# oz_nds = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'НДС' in x]].copy()
# oz_nds.columns = ['Артикул'] + date_cols
# oz_nnp = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Налог на прибыль' in x]].copy()
# oz_nnp.columns = ['Артикул'] + date_cols
# for i in range(len(oz_logistic)):
#     for col in date_cols:
#         oz_logistic.loc[i, col] = round((oz_logistic.loc[i][col] * oz.loc[i][col]), 2)
# for i in range(len(oz_komm)):
#     for col in date_cols:
#         oz_komm.loc[i, col] = round((oz_komm.loc[i][col] * oz.loc[i][col]), 2)    
# for i in range(len(oz_ekv)):
#     for col in date_cols:
#         oz_ekv.loc[i, col] = round((oz_ekv.loc[i][col] * oz.loc[i][col]), 2)     
# for i in range(len(oz_nds)):
#     for col in date_cols:
#         oz_nds.loc[i, col] = round((oz_nds.loc[i][col] * oz.loc[i][col]), 2)
# for i in range(len(oz_nnp)):
#     for col in date_cols:
#         oz_nnp.loc[i, col] = round((oz_nnp.loc[i][col] * oz.loc[i][col]), 2)
# oz_logistic['sum_logistic'] = [oz_logistic.iloc[x, 1:].sum() for x in list(oz_logistic.index)]
# oz_komm['sum_komm'] = [oz_komm.iloc[x, 1:].sum() for x in list(oz_komm.index)]
# oz_ekv['sum_ekv'] = [oz_ekv.iloc[x, 1:].sum() for x in list(oz_ekv.index)]
# oz_nds['sum_nds'] = [oz_nds.iloc[x, 1:].sum() for x in list(oz_nds.index)]
# oz_nnp['sum_nnp'] = [oz_nnp.iloc[x, 1:].sum() for x in list(oz_nnp.index)]

# final_df = wwm.Reestr()\
#                        .merge(pd.read_excel(wwm.path_st,
#                                             dtype={'Артикул': 'str'}), on='Артикул', how='left')\
#                        .merge(oz[['Артикул', 'sum_orders']], on='Артикул', how='right')\
#                        .merge(ozp[['Артикул', 'mean_price']], on='Артикул', how='right')\
#                        .merge(ozv[['Артикул', 'sum_earnings']], on='Артикул', how='right')\
#                        .merge(cp[['Артикул', 'sum_cost_price']], on='Артикул', how='right')\
#                        .merge(oz_logistic[['Артикул', 'sum_logistic']], on='Артикул', how='right')\
#                        .merge(oz_komm[['Артикул', 'sum_komm']], on='Артикул', how='right')\
#                        .merge(oz_ekv[['Артикул', 'sum_ekv']], on='Артикул', how='right')\
#                        .merge(oz_nds[['Артикул', 'sum_nds']], on='Артикул', how='right')\
#                        .merge(oz_nnp[['Артикул', 'sum_nnp']], on='Артикул', how='right')

# with pd.ExcelWriter(path_downloads + 'Озон, заказы и расходы, 1-31 января.xlsx') as writer:
#     oz.to_excel(writer, sheet_name="Заказы", index=False)
#     for column in oz:
#         column_width = max(oz[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz.columns.get_loc(column)
#         writer.sheets['Заказы'].set_column(col_idx, col_idx, column_width)
#     ozp.to_excel(writer, sheet_name="Цены", index=False)
#     for column in ozp:
#         column_width = max(ozp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = ozp.columns.get_loc(column)
#         writer.sheets['Цены'].set_column(col_idx, col_idx, column_width)
#     ozv.to_excel(writer, sheet_name="Выручка", index=False)
#     for column in ozv:
#         column_width = max(ozv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = ozv.columns.get_loc(column)
#         writer.sheets['Выручка'].set_column(col_idx, col_idx, column_width)
#     cp.to_excel(writer, sheet_name="Себестоимость", index=False)
#     for column in cp:
#         column_width = max(cp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = cp.columns.get_loc(column)
#         writer.sheets['Себестоимость'].set_column(col_idx, col_idx, column_width)
#     oz_logistic.to_excel(writer, sheet_name="Логистика", index=False)
#     for column in oz_logistic:
#         column_width = max(oz_logistic[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_logistic.columns.get_loc(column)
#         writer.sheets['Логистика'].set_column(col_idx, col_idx, column_width)
#     oz_komm.to_excel(writer, sheet_name="Комиссия", index=False)
#     for column in oz_komm:
#         column_width = max(oz_komm[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_komm.columns.get_loc(column)
#         writer.sheets['Комиссия'].set_column(col_idx, col_idx, column_width)
#     oz_ekv.to_excel(writer, sheet_name="Эквайринг", index=False)
#     for column in oz_ekv:
#         column_width = max(oz_ekv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_ekv.columns.get_loc(column)
#         writer.sheets['Эквайринг'].set_column(col_idx, col_idx, column_width)
#     oz_nds.to_excel(writer, sheet_name="НДС", index=False)
#     for column in oz_nds:
#         column_width = max(oz_nds[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_nds.columns.get_loc(column)
#         writer.sheets['НДС'].set_column(col_idx, col_idx, column_width)
#     oz_nnp.to_excel(writer, sheet_name="Налог на прибыль", index=False)
#     for column in oz_nnp:
#         column_width = max(oz_nnp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_nnp.columns.get_loc(column)
#         writer.sheets['Налог на прибыль'].set_column(col_idx, col_idx, column_width)
#     final_df.to_excel(writer, sheet_name="Итоговая", index=False)
#     for column in final_df:
#         column_width = max(final_df[column].astype(str).map(len).max()+2, len(column))
#         col_idx = final_df.columns.get_loc(column)
#         writer.sheets['Итоговая'].set_column(col_idx, col_idx, column_width)
# writer.save()

In [123]:
# oz = mwm.select_table('dds_oz_orders', schema=mwm.dds_schema)
# oz = wwm.expand_df(oz[(oz['date'] >= datetime.date(2024, 1, 1)) \
#                       & (oz['date'] <= datetime.date(2024, 1, 31)) \
#                       & (oz['account']=='ir')].drop('account', axis=1).reset_index(drop=True),
#                    key_column='item_code',
#                    result_column='orders')
# date_cols = dt_col(oz)
# oz = oz.query(f"index in {[x for x in list(oz.index) if oz.iloc[x, 1:].sum() > 0]}")\
# .reset_index(drop=True)
# oz['sum_orders'] = [oz.iloc[x, 1:].sum() for x in list(oz.index)]

# ozp = mwm.select_table('full_oz', schema=mwm.ir_schema)
# ozp = ozp[ozp['marketplace']=='oz'][['item_code', 'date', 'prices']]
# ozp = wwm.expand_df(ozp[(ozp['date'] >= datetime.date(2024, 1, 1)) \
#                       & (ozp['date'] <= datetime.date(2024, 1, 31))].reset_index(drop=True),
#                    key_column='item_code',
#                    result_column='prices',
#                    func='mean')
# oz = oz.rename(columns={'item_code': 'Артикул'})
# ozp = ozp.rename(columns={'item_code': 'Артикул'})

# oz = oz.sort_values('Артикул').reset_index(drop=True)

# ozp = ozp[['Артикул']+[y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in dt_col(ozp)])]]
# ozp.iloc[:, 1:] = ozp.iloc[:, 1:].astype(int)
# ozp['Артикул'] = ozp['Артикул'].astype(str)
# ozp = ozp[ozp['Артикул'].isin(list(set(oz['Артикул'])))].reset_index(drop=True)

# for row in date_cols:
#     if row not in ozp.columns:
#         print(row)
# for row in date_cols:
#     if row not in ozp.columns:
#         print(row)
# ozp = ozp[['Артикул']+date_cols]
# mwm.unique_items(ozp)
# ozp['mean_price'] = [round(np.mean([y for y in list(ozp.iloc[x, 1:]) if y > 0])) for x in list(ozp.index)]
# ozp = ozp.sort_values('Артикул').reset_index(drop=True)
# mwm.unique_items(ozp)

# ozv = oz[['Артикул']].copy()
# for col in date_cols:
#     ozv[col] = None
# for i in range(len(ozv)):
#     for col in date_cols:
#         ozv.loc[i, col] = oz.loc[i][col] * ozp.loc[i][col]
# for row in list(set(ozp['Артикул'])):
#     if row not in list(set(oz['Артикул'])):
#         print(row)
# ozv['sum_earnings'] = [ozv.iloc[x, 1:].sum() for x in list(ozv.index)]
# mwm.unique_items(ozv)
# ozv = ozv[ozv['Артикул'].isin(list(set(oz['Артикул'])))].reset_index(drop=True)
# ozv = ozv.sort_values('Артикул').reset_index(drop=True)

# cp = mwm.select_table_with_last_date_sort_list('cost_price', 'date')
# cp['Артикул'] = cp['Артикул'].astype('str')
# cp = cp[['Артикул', 'Себестоимость']]
# cp = cp[cp['Себестоимость']>0].reset_index(drop=True)
# cp = mwm.clear_article(cp, 'Артикул').reset_index(drop=True)
# Q = []
# for i in range(len(cp)):
#     art = cp.loc[i]['Артикул']
#     if len(cp[cp['Артикул']==art])> 1:
#         sub_df = cp[cp['Артикул']==art]
#         sub_df = sub_df.sort_values('Себестоимость')
#         for ind in list(sub_df.index)[:-1]:
#             Q.append(ind)
# Q = list(set(Q))
# cp = cp.query('index not in @Q').reset_index(drop=True)
# mwm.unique_items(cp)
# cp = cp[cp['Артикул'].isin(list(set(oz['Артикул'])))].sort_values('Артикул').reset_index(drop=True)
# for col in date_cols:
#     cp[col] = None
#     for i in range(len(cp)):
#         cp.loc[i, col] = round((cp.loc[i]['Себестоимость'] * oz.loc[i][col]), 2)
# cp['sum_cost_price'] = [cp.iloc[x, 2:].sum() for x in list(cp.index)]

# oz_dop = oz[['Артикул']].copy()
# for col in date_cols:
#     sub_df = calc_features_fbo(cp[['Артикул', 'Себестоимость']].merge(ozp[['Артикул', col]]\
#                                                                       .rename(columns={col: 'Цена'}).copy(),
#                                                                       on='Артикул', 
#                                                                       how='left'))[['Артикул', 
#                                                                                     'Логистика', 
#                                                                                     'Комиссия', 
#                                                                                     'Эквайринг', 
#                                                                                     'НДС', 
#                                                                                     'Налог на прибыль']]\
#     .rename(columns={'Логистика': col + ' Логистика',
#                      'Комиссия': col + ' Комиссия',
#                      'Эквайринг': col + ' Эквайринг',
#                      'НДС': col + ' НДС',
#                      'Налог на прибыль': col + ' Налог на прибыль'})
#     oz_dop = oz_dop.merge(sub_df, on='Артикул', how='left')
    
# oz_logistic = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Логистика' in x]].copy()
# oz_logistic.columns = ['Артикул'] + date_cols
# oz_komm = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Комиссия' in x]].copy()
# oz_komm.columns = ['Артикул'] + date_cols
# oz_ekv = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Эквайринг' in x]].copy()
# oz_ekv.columns = ['Артикул'] + date_cols
# oz_nds = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'НДС' in x]].copy()
# oz_nds.columns = ['Артикул'] + date_cols
# oz_nnp = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Налог на прибыль' in x]].copy()
# oz_nnp.columns = ['Артикул'] + date_cols
# for i in range(len(oz_logistic)):
#     for col in date_cols:
#         oz_logistic.loc[i, col] = round((oz_logistic.loc[i][col] * oz.loc[i][col]), 2)
# for i in range(len(oz_komm)):
#     for col in date_cols:
#         oz_komm.loc[i, col] = round((oz_komm.loc[i][col] * oz.loc[i][col]), 2)    
# for i in range(len(oz_ekv)):
#     for col in date_cols:
#         oz_ekv.loc[i, col] = round((oz_ekv.loc[i][col] * oz.loc[i][col]), 2)     
# for i in range(len(oz_nds)):
#     for col in date_cols:
#         oz_nds.loc[i, col] = round((oz_nds.loc[i][col] * oz.loc[i][col]), 2)
# for i in range(len(oz_nnp)):
#     for col in date_cols:
#         oz_nnp.loc[i, col] = round((oz_nnp.loc[i][col] * oz.loc[i][col]), 2)
# oz_logistic['sum_logistic'] = [oz_logistic.iloc[x, 1:].sum() for x in list(oz_logistic.index)]
# oz_komm['sum_komm'] = [oz_komm.iloc[x, 1:].sum() for x in list(oz_komm.index)]
# oz_ekv['sum_ekv'] = [oz_ekv.iloc[x, 1:].sum() for x in list(oz_ekv.index)]
# oz_nds['sum_nds'] = [oz_nds.iloc[x, 1:].sum() for x in list(oz_nds.index)]
# oz_nnp['sum_nnp'] = [oz_nnp.iloc[x, 1:].sum() for x in list(oz_nnp.index)]

# final_df = wwm.Reestr()\
#                        .merge(pd.read_excel(wwm.path_st,
#                                             dtype={'Артикул': 'str'}), on='Артикул', how='left')\
#                        .merge(oz[['Артикул', 'sum_orders']], on='Артикул', how='right')\
#                        .merge(ozp[['Артикул', 'mean_price']], on='Артикул', how='right')\
#                        .merge(ozv[['Артикул', 'sum_earnings']], on='Артикул', how='right')\
#                        .merge(cp[['Артикул', 'sum_cost_price']], on='Артикул', how='right')\
#                        .merge(oz_logistic[['Артикул', 'sum_logistic']], on='Артикул', how='right')\
#                        .merge(oz_komm[['Артикул', 'sum_komm']], on='Артикул', how='right')\
#                        .merge(oz_ekv[['Артикул', 'sum_ekv']], on='Артикул', how='right')\
#                        .merge(oz_nds[['Артикул', 'sum_nds']], on='Артикул', how='right')\
#                        .merge(oz_nnp[['Артикул', 'sum_nnp']], on='Артикул', how='right')

# with pd.ExcelWriter(path_downloads + 'Озон, заказы и расходы, 1-31 января.xlsx') as writer:
#     oz.to_excel(writer, sheet_name="Заказы", index=False)
#     for column in oz:
#         column_width = max(oz[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz.columns.get_loc(column)
#         writer.sheets['Заказы'].set_column(col_idx, col_idx, column_width)
#     ozp.to_excel(writer, sheet_name="Цены", index=False)
#     for column in ozp:
#         column_width = max(ozp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = ozp.columns.get_loc(column)
#         writer.sheets['Цены'].set_column(col_idx, col_idx, column_width)
#     ozv.to_excel(writer, sheet_name="Выручка", index=False)
#     for column in ozv:
#         column_width = max(ozv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = ozv.columns.get_loc(column)
#         writer.sheets['Выручка'].set_column(col_idx, col_idx, column_width)
#     cp.to_excel(writer, sheet_name="Себестоимость", index=False)
#     for column in cp:
#         column_width = max(cp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = cp.columns.get_loc(column)
#         writer.sheets['Себестоимость'].set_column(col_idx, col_idx, column_width)
#     oz_logistic.to_excel(writer, sheet_name="Логистика", index=False)
#     for column in oz_logistic:
#         column_width = max(oz_logistic[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_logistic.columns.get_loc(column)
#         writer.sheets['Логистика'].set_column(col_idx, col_idx, column_width)
#     oz_komm.to_excel(writer, sheet_name="Комиссия", index=False)
#     for column in oz_komm:
#         column_width = max(oz_komm[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_komm.columns.get_loc(column)
#         writer.sheets['Комиссия'].set_column(col_idx, col_idx, column_width)
#     oz_ekv.to_excel(writer, sheet_name="Эквайринг", index=False)
#     for column in oz_ekv:
#         column_width = max(oz_ekv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_ekv.columns.get_loc(column)
#         writer.sheets['Эквайринг'].set_column(col_idx, col_idx, column_width)
#     oz_nds.to_excel(writer, sheet_name="НДС", index=False)
#     for column in oz_nds:
#         column_width = max(oz_nds[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_nds.columns.get_loc(column)
#         writer.sheets['НДС'].set_column(col_idx, col_idx, column_width)
#     oz_nnp.to_excel(writer, sheet_name="Налог на прибыль", index=False)
#     for column in oz_nnp:
#         column_width = max(oz_nnp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_nnp.columns.get_loc(column)
#         writer.sheets['Налог на прибыль'].set_column(col_idx, col_idx, column_width)
#     final_df.to_excel(writer, sheet_name="Итоговая", index=False)
#     for column in final_df:
#         column_width = max(final_df[column].astype(str).map(len).max()+2, len(column))
#         col_idx = final_df.columns.get_loc(column)
#         writer.sheets['Итоговая'].set_column(col_idx, col_idx, column_width)
# writer.save()

In [124]:
# oz = pd.read_excel(wwm.path_full_OZON_df,
#                    dtype={'Баркод': 'int',
#                           'Артикул': 'str'})
# date_cols = [x for x in dt_col(oz) if 'November-2023' in x and int(x[:2]) in list(range(1, 31))]
# oz_art = oz[['Артикул', 'Баркод']].copy()
# oz = oz[['Артикул']+date_cols]
# oz = oz.query(f"index in {[x for x in list(oz.index) if oz.iloc[x, 1:].sum() > 0]}")\
# .reset_index(drop=True)
# oz['sum_orders'] = [oz.iloc[x, 1:].sum() for x in list(oz.index)]
# oz = oz.sort_values('Артикул').reset_index(drop=True)
# mwm.unique_items(oz)
# ozp = pd.read_excel(wwm.path_full_OZON_df,
#                     sheet_name='ozon_price_metrics',
#                     dtype={'Баркод': 'int',
#                            'Артикул': 'str'})
# ozp = ozp[['Артикул']+[x for x in dt_col(ozp) if 'November-2023' in x and int(x[:2]) in list(range(1, 31))]]
# ozp['04-November-2023'] = ozp['03-November-2023'] 
# ozp['05-November-2023'] = ozp['03-November-2023'] 
# ozp['06-November-2023'] = ozp['03-November-2023'] 
# ozp['11-November-2023'] = ozp['10-November-2023'] 
# ozp['12-November-2023'] = ozp['10-November-2023'] 
# ozp['18-November-2023'] = ozp['17-November-2023'] 
# ozp['19-November-2023'] = ozp['17-November-2023'] 
# ozp['25-November-2023'] = ozp['24-November-2023'] 
# ozp['26-November-2023'] = ozp['24-November-2023'] 
# ozp = ozp[['Артикул']+[y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in dt_col(ozp)])]]
# ozp.iloc[:, 1:] = ozp.iloc[:, 1:].astype(int)
# ozp['Артикул'] = ozp['Артикул'].astype(str)
# ozp = ozp[ozp['Артикул'].isin(list(set(oz['Артикул'])))].reset_index(drop=True)
# for row in date_cols:
#     if row not in ozp.columns:
#         print(row)
# for row in date_cols:
#     if row not in ozp.columns:
#         print(row)
# ozp = ozp[['Артикул']+date_cols]
# mwm.unique_items(ozp)
# ozp['mean_price'] = [round(np.mean([y for y in list(ozp.iloc[x, 1:]) if y > 0])) for x in list(ozp.index)]
# ozp = ozp.sort_values('Артикул').reset_index(drop=True)
# mwm.unique_items(ozp)
# ozv = oz[['Артикул']].copy()
# for col in date_cols:
#     ozv[col] = None
# for i in range(len(ozv)):
#     for col in date_cols:
#         ozv.loc[i, col] = oz.loc[i][col] * ozp.loc[i][col]
# for row in list(set(ozp['Артикул'])):
#     if row not in list(set(oz['Артикул'])):
#         print(row)
# ozv['sum_earnings'] = [ozv.iloc[x, 1:].sum() for x in list(ozv.index)]
# mwm.unique_items(ozv)
# ozv = ozv[ozv['Артикул'].isin(list(set(oz['Артикул'])))].reset_index(drop=True)
# ozv = ozv.sort_values('Артикул').reset_index(drop=True)
# cp = mwm.select_table_with_last_date_sort_list('cost_price', 'date')
# cp['Артикул'] = cp['Артикул'].astype('str')
# cp = cp[['Артикул', 'Себестоимость']]
# cp = cp[cp['Себестоимость']>0].reset_index(drop=True)
# cp = mwm.clear_article(cp, 'Артикул').reset_index(drop=True)
# Q = []
# for i in range(len(cp)):
#     art = cp.loc[i]['Артикул']
#     if len(cp[cp['Артикул']==art])> 1:
#         sub_df = cp[cp['Артикул']==art]
#         sub_df = sub_df.sort_values('Себестоимость')
#         for ind in list(sub_df.index)[:-1]:
#             Q.append(ind)
# Q = list(set(Q))
# cp = cp.query('index not in @Q').reset_index(drop=True)
# mwm.unique_items(cp)
# cp = cp[cp['Артикул'].isin(list(set(oz['Артикул'])))].sort_values('Артикул').reset_index(drop=True)
# for col in date_cols:
#     cp[col] = None
#     for i in range(len(cp)):
#         cp.loc[i, col] = round((cp.loc[i]['Себестоимость'] * oz.loc[i][col]), 2)
# cp['sum_cost_price'] = [cp.iloc[x, 2:].sum() for x in list(cp.index)]
# oz_dop = oz[['Артикул']].copy()
# for col in date_cols:
#     sub_df = calc_features_fbo(cp[['Артикул', 'Себестоимость']].merge(ozp[['Артикул', col]]\
#                                                                       .rename(columns={col: 'Цена'}).copy(),
#                                                                       on='Артикул', 
#                                                                       how='left'))[['Артикул', 
#                                                                                     'Логистика', 
#                                                                                     'Комиссия', 
#                                                                                     'Эквайринг', 
#                                                                                     'НДС', 
#                                                                                     'Налог на прибыль']]\
#     .rename(columns={'Логистика': col + ' Логистика',
#                      'Комиссия': col + ' Комиссия',
#                      'Эквайринг': col + ' Эквайринг',
#                      'НДС': col + ' НДС',
#                      'Налог на прибыль': col + ' Налог на прибыль'})
#     oz_dop = oz_dop.merge(sub_df, on='Артикул', how='left')
# oz_logistic = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Логистика' in x]].copy()
# oz_logistic.columns = ['Артикул'] + date_cols
# oz_komm = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Комиссия' in x]].copy()
# oz_komm.columns = ['Артикул'] + date_cols
# oz_ekv = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Эквайринг' in x]].copy()
# oz_ekv.columns = ['Артикул'] + date_cols
# oz_nds = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'НДС' in x]].copy()
# oz_nds.columns = ['Артикул'] + date_cols
# oz_nnp = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Налог на прибыль' in x]].copy()
# oz_nnp.columns = ['Артикул'] + date_cols
# for i in range(len(oz_logistic)):
#     for col in date_cols:
#         oz_logistic.loc[i, col] = round((oz_logistic.loc[i][col] * oz.loc[i][col]), 2)
# for i in range(len(oz_komm)):
#     for col in date_cols:
#         oz_komm.loc[i, col] = round((oz_komm.loc[i][col] * oz.loc[i][col]), 2)    
# for i in range(len(oz_ekv)):
#     for col in date_cols:
#         oz_ekv.loc[i, col] = round((oz_ekv.loc[i][col] * oz.loc[i][col]), 2)     
# for i in range(len(oz_nds)):
#     for col in date_cols:
#         oz_nds.loc[i, col] = round((oz_nds.loc[i][col] * oz.loc[i][col]), 2)
# for i in range(len(oz_nnp)):
#     for col in date_cols:
#         oz_nnp.loc[i, col] = round((oz_nnp.loc[i][col] * oz.loc[i][col]), 2)
# oz_logistic['sum_logistic'] = [oz_logistic.iloc[x, 1:].sum() for x in list(oz_logistic.index)]
# oz_komm['sum_komm'] = [oz_komm.iloc[x, 1:].sum() for x in list(oz_komm.index)]
# oz_ekv['sum_ekv'] = [oz_ekv.iloc[x, 1:].sum() for x in list(oz_ekv.index)]
# oz_nds['sum_nds'] = [oz_nds.iloc[x, 1:].sum() for x in list(oz_nds.index)]
# oz_nnp['sum_nnp'] = [oz_nnp.iloc[x, 1:].sum() for x in list(oz_nnp.index)]
# final_df = wwm.Reestr().merge(oz_art, on='Артикул', how='left')\
#                        .merge(pd.read_excel(wwm.path_st,
#                                             dtype={'Артикул': 'str'}), on='Артикул', how='left')\
#                        .merge(oz[['Артикул', 'sum_orders']], on='Артикул', how='right')\
#                        .merge(ozp[['Артикул', 'mean_price']], on='Артикул', how='right')\
#                        .merge(ozv[['Артикул', 'sum_earnings']], on='Артикул', how='right')\
#                        .merge(cp[['Артикул', 'sum_cost_price']], on='Артикул', how='right')\
#                        .merge(oz_logistic[['Артикул', 'sum_logistic']], on='Артикул', how='right')\
#                        .merge(oz_komm[['Артикул', 'sum_komm']], on='Артикул', how='right')\
#                        .merge(oz_ekv[['Артикул', 'sum_ekv']], on='Артикул', how='right')\
#                        .merge(oz_nds[['Артикул', 'sum_nds']], on='Артикул', how='right')\
#                        .merge(oz_nnp[['Артикул', 'sum_nnp']], on='Артикул', how='right')
# with pd.ExcelWriter(path_downloads + 'Озон, заказы и расходы, 1-30 ноября.xlsx') as writer:
#     oz.to_excel(writer, sheet_name="Заказы", index=False)
#     for column in oz:
#         column_width = max(oz[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz.columns.get_loc(column)
#         writer.sheets['Заказы'].set_column(col_idx, col_idx, column_width)
#     ozp.to_excel(writer, sheet_name="Цены", index=False)
#     for column in ozp:
#         column_width = max(ozp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = ozp.columns.get_loc(column)
#         writer.sheets['Цены'].set_column(col_idx, col_idx, column_width)
#     ozv.to_excel(writer, sheet_name="Выручка", index=False)
#     for column in ozv:
#         column_width = max(ozv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = ozv.columns.get_loc(column)
#         writer.sheets['Выручка'].set_column(col_idx, col_idx, column_width)
#     cp.to_excel(writer, sheet_name="Себестоимость", index=False)
#     for column in cp:
#         column_width = max(cp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = cp.columns.get_loc(column)
#         writer.sheets['Себестоимость'].set_column(col_idx, col_idx, column_width)
#     oz_logistic.to_excel(writer, sheet_name="Логистика", index=False)
#     for column in oz_logistic:
#         column_width = max(oz_logistic[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_logistic.columns.get_loc(column)
#         writer.sheets['Логистика'].set_column(col_idx, col_idx, column_width)
#     oz_komm.to_excel(writer, sheet_name="Комиссия", index=False)
#     for column in oz_komm:
#         column_width = max(oz_komm[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_komm.columns.get_loc(column)
#         writer.sheets['Комиссия'].set_column(col_idx, col_idx, column_width)
#     oz_ekv.to_excel(writer, sheet_name="Эквайринг", index=False)
#     for column in oz_ekv:
#         column_width = max(oz_ekv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_ekv.columns.get_loc(column)
#         writer.sheets['Эквайринг'].set_column(col_idx, col_idx, column_width)
#     oz_nds.to_excel(writer, sheet_name="НДС", index=False)
#     for column in oz_nds:
#         column_width = max(oz_nds[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_nds.columns.get_loc(column)
#         writer.sheets['НДС'].set_column(col_idx, col_idx, column_width)
#     oz_nnp.to_excel(writer, sheet_name="Налог на прибыль", index=False)
#     for column in oz_nnp:
#         column_width = max(oz_nnp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_nnp.columns.get_loc(column)
#         writer.sheets['Налог на прибыль'].set_column(col_idx, col_idx, column_width)
#     final_df.to_excel(writer, sheet_name="Итоговая", index=False)
#     for column in final_df:
#         column_width = max(final_df[column].astype(str).map(len).max()+2, len(column))
#         col_idx = final_df.columns.get_loc(column)
#         writer.sheets['Итоговая'].set_column(col_idx, col_idx, column_width)
# writer.save()

In [125]:
# oz = pd.read_excel(wwm.path_full_OZON_df,
#                    dtype={'Баркод': 'int',
#                           'Артикул': 'str'})
# date_cols = [x for x in dt_col(oz) if 'November-2023' in x and int(x[:2]) in list(range(1, 31))]
# oz_art = oz[['Артикул', 'Баркод']].copy()
# oz = oz[['Артикул']+date_cols]
# oz = oz.query(f"index in {[x for x in list(oz.index) if oz.iloc[x, 1:].sum() > 0]}")\
# .reset_index(drop=True)
# oz['sum_orders'] = [oz.iloc[x, 1:].sum() for x in list(oz.index)]
# oz = oz.sort_values('Артикул').reset_index(drop=True)
# mwm.unique_items(oz)
# df = mwm.get_tabl_from_bd('oz_prices', flag=1)
# df = df[['offer_id', 'price', 'date']]
# df = df.rename(columns={'offer_id': 'Артикул'})
# df['Артикул'] = df['Артикул'].astype(str)
# df = wwm.clear_article(df, 'Артикул')
# df['date'] = [pd.to_datetime(x).strftime('%d-%B-%Y') for x in list(df['date'])]
# df = df[df['date'].isin(date_cols)].reset_index(drop=True)
# df_0 = pd.DataFrame(list(set(df['Артикул'])), columns=['Артикул'])
# for row in [y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in list(df['date'].unique())])]:
#     df_0 = df_0.merge(df[df['date']==row].groupby('Артикул')['price'].mean()\
#     .reset_index(drop=False).rename(columns={'price': row}), on='Артикул', how='left')
# df_0 = df_0.fillna(0)
# df, df_0 = df_0, None
# ozp, df = df, None

# ozp.iloc[:, 1:] = ozp.iloc[:, 1:].astype(int)
# ozp['Артикул'] = ozp['Артикул'].astype(str)
# ozp = ozp[ozp['Артикул'].isin(list(set(oz['Артикул'])))].reset_index(drop=True)
# for row in date_cols:
#     if row not in ozp.columns:
#         print(row)
# ozp['18-November-2023'] = ozp['17-November-2023']
# ozp['19-November-2023'] = ozp['17-November-2023']
# for row in date_cols:
#     if row not in ozp.columns:
#         print(row)
# ozp = ozp[['Артикул']+date_cols]
# mwm.unique_items(ozp)
# ozp['mean_price'] = [round(np.mean([y for y in list(ozp.iloc[x, 1:]) if y > 0])) for x in list(ozp.index)]
# ozp = ozp.sort_values('Артикул').reset_index(drop=True)

# mwm.unique_items(ozp)
# ozv = oz[['Артикул']].copy()
# for col in date_cols:
#     ozv[col] = None
# for i in range(len(ozv)):
#     for col in date_cols:
#         ozv.loc[i, col] = oz.loc[i][col] * ozp.loc[i][col]
# for row in list(set(ozp['Артикул'])):
#     if row not in list(set(oz['Артикул'])):
#         print(row)
# ozv['sum_earnings'] = [ozv.iloc[x, 1:].sum() for x in list(ozv.index)]
# mwm.unique_items(ozv)
# ozv = ozv[ozv['Артикул'].isin(list(set(oz['Артикул'])))].reset_index(drop=True)
# ozv = ozv.sort_values('Артикул').reset_index(drop=True)
# cp = mwm.select_table_with_last_date_sort_list('cost_price', 'date')
# cp['Артикул'] = cp['Артикул'].astype('str')
# cp = cp[['Артикул', 'Себестоимость']]
# cp = cp[cp['Себестоимость']>0].reset_index(drop=True)
# cp = mwm.clear_article(cp, 'Артикул').reset_index(drop=True)
# Q = []
# for i in range(len(cp)):
#     art = cp.loc[i]['Артикул']
#     if len(cp[cp['Артикул']==art])> 1:
#         sub_df = cp[cp['Артикул']==art]
#         sub_df = sub_df.sort_values('Себестоимость')
#         for ind in list(sub_df.index)[:-1]:
#             Q.append(ind)
# Q = list(set(Q))
# cp = cp.query('index not in @Q').reset_index(drop=True)
# mwm.unique_items(cp)
# cp = cp[cp['Артикул'].isin(list(set(oz['Артикул'])))].sort_values('Артикул').reset_index(drop=True)
# for col in date_cols:
#     cp[col] = None
#     for i in range(len(cp)):
#         cp.loc[i, col] = round((cp.loc[i]['Себестоимость'] * oz.loc[i][col]), 2)
# cp['sum_cost_price'] = [cp.iloc[x, 2:].sum() for x in list(cp.index)]
# oz_dop = oz[['Артикул']].copy()
# for col in date_cols:
#     sub_df = calc_features_fbo(cp[['Артикул', 'Себестоимость']].merge(ozp[['Артикул', col]]\
#                                                                       .rename(columns={col: 'Цена'}).copy(),
#                                                                       on='Артикул', 
#                                                                       how='left'))[['Артикул', 
#                                                                                     'Логистика', 
#                                                                                     'Комиссия', 
#                                                                                     'Эквайринг', 
#                                                                                     'НДС', 
#                                                                                     'Налог на прибыль']]\
#     .rename(columns={'Логистика': col + ' Логистика',
#                      'Комиссия': col + ' Комиссия',
#                      'Эквайринг': col + ' Эквайринг',
#                      'НДС': col + ' НДС',
#                      'Налог на прибыль': col + ' Налог на прибыль'})
#     oz_dop = oz_dop.merge(sub_df, on='Артикул', how='left')
# oz_logistic = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Логистика' in x]].copy()
# oz_logistic.columns = ['Артикул'] + date_cols
# oz_komm = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Комиссия' in x]].copy()
# oz_komm.columns = ['Артикул'] + date_cols
# oz_ekv = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Эквайринг' in x]].copy()
# oz_ekv.columns = ['Артикул'] + date_cols
# oz_nds = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'НДС' in x]].copy()
# oz_nds.columns = ['Артикул'] + date_cols
# oz_nnp = oz_dop[['Артикул']+[x for x in list(oz_dop.columns) if 'Налог на прибыль' in x]].copy()
# oz_nnp.columns = ['Артикул'] + date_cols
# for i in range(len(oz_logistic)):
#     for col in date_cols:
#         oz_logistic.loc[i, col] = round((oz_logistic.loc[i][col] * oz.loc[i][col]), 2)
# for i in range(len(oz_komm)):
#     for col in date_cols:
#         oz_komm.loc[i, col] = round((oz_komm.loc[i][col] * oz.loc[i][col]), 2)    
# for i in range(len(oz_ekv)):
#     for col in date_cols:
#         oz_ekv.loc[i, col] = round((oz_ekv.loc[i][col] * oz.loc[i][col]), 2)     
# for i in range(len(oz_nds)):
#     for col in date_cols:
#         oz_nds.loc[i, col] = round((oz_nds.loc[i][col] * oz.loc[i][col]), 2)
# for i in range(len(oz_nnp)):
#     for col in date_cols:
#         oz_nnp.loc[i, col] = round((oz_nnp.loc[i][col] * oz.loc[i][col]), 2)
# oz_logistic['sum_logistic'] = [oz_logistic.iloc[x, 1:].sum() for x in list(oz_logistic.index)]
# oz_komm['sum_komm'] = [oz_komm.iloc[x, 1:].sum() for x in list(oz_komm.index)]
# oz_ekv['sum_ekv'] = [oz_ekv.iloc[x, 1:].sum() for x in list(oz_ekv.index)]
# oz_nds['sum_nds'] = [oz_nds.iloc[x, 1:].sum() for x in list(oz_nds.index)]
# oz_nnp['sum_nnp'] = [oz_nnp.iloc[x, 1:].sum() for x in list(oz_nnp.index)]
# final_df = wwm.Reestr().merge(oz_art, on='Артикул', how='left')\
#                        .merge(pd.read_excel(wwm.path_st,
#                                             dtype={'Артикул': 'str'}), on='Артикул', how='left')\
#                        .merge(oz[['Артикул', 'sum_orders']], on='Артикул', how='right')\
#                        .merge(ozp[['Артикул', 'mean_price']], on='Артикул', how='right')\
#                        .merge(ozv[['Артикул', 'sum_earnings']], on='Артикул', how='right')\
#                        .merge(cp[['Артикул', 'sum_cost_price']], on='Артикул', how='right')\
#                        .merge(oz_logistic[['Артикул', 'sum_logistic']], on='Артикул', how='right')\
#                        .merge(oz_komm[['Артикул', 'sum_komm']], on='Артикул', how='right')\
#                        .merge(oz_ekv[['Артикул', 'sum_ekv']], on='Артикул', how='right')\
#                        .merge(oz_nds[['Артикул', 'sum_nds']], on='Артикул', how='right')\
#                        .merge(oz_nnp[['Артикул', 'sum_nnp']], on='Артикул', how='right')
# with pd.ExcelWriter(path_downloads + 'Озон, заказы и расходы, 1-30 ноября.xlsx') as writer:
#     oz.to_excel(writer, sheet_name="Заказы", index=False)
#     for column in oz:
#         column_width = max(oz[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz.columns.get_loc(column)
#         writer.sheets['Заказы'].set_column(col_idx, col_idx, column_width)
#     ozp.to_excel(writer, sheet_name="Цены", index=False)
#     for column in ozp:
#         column_width = max(ozp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = ozp.columns.get_loc(column)
#         writer.sheets['Цены'].set_column(col_idx, col_idx, column_width)
#     ozv.to_excel(writer, sheet_name="Выручка", index=False)
#     for column in ozv:
#         column_width = max(ozv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = ozv.columns.get_loc(column)
#         writer.sheets['Выручка'].set_column(col_idx, col_idx, column_width)
#     cp.to_excel(writer, sheet_name="Себестоимость", index=False)
#     for column in cp:
#         column_width = max(cp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = cp.columns.get_loc(column)
#         writer.sheets['Себестоимость'].set_column(col_idx, col_idx, column_width)
#     oz_logistic.to_excel(writer, sheet_name="Логистика", index=False)
#     for column in oz_logistic:
#         column_width = max(oz_logistic[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_logistic.columns.get_loc(column)
#         writer.sheets['Логистика'].set_column(col_idx, col_idx, column_width)
#     oz_komm.to_excel(writer, sheet_name="Комиссия", index=False)
#     for column in oz_komm:
#         column_width = max(oz_komm[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_komm.columns.get_loc(column)
#         writer.sheets['Комиссия'].set_column(col_idx, col_idx, column_width)
#     oz_ekv.to_excel(writer, sheet_name="Эквайринг", index=False)
#     for column in oz_ekv:
#         column_width = max(oz_ekv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_ekv.columns.get_loc(column)
#         writer.sheets['Эквайринг'].set_column(col_idx, col_idx, column_width)
#     oz_nds.to_excel(writer, sheet_name="НДС", index=False)
#     for column in oz_nds:
#         column_width = max(oz_nds[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_nds.columns.get_loc(column)
#         writer.sheets['НДС'].set_column(col_idx, col_idx, column_width)
#     oz_nnp.to_excel(writer, sheet_name="Налог на прибыль", index=False)
#     for column in oz_nnp:
#         column_width = max(oz_nnp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = oz_nnp.columns.get_loc(column)
#         writer.sheets['Налог на прибыль'].set_column(col_idx, col_idx, column_width)
#     final_df.to_excel(writer, sheet_name="Итоговая", index=False)
#     for column in final_df:
#         column_width = max(final_df[column].astype(str).map(len).max()+2, len(column))
#         col_idx = final_df.columns.get_loc(column)
#         writer.sheets['Итоговая'].set_column(col_idx, col_idx, column_width)
# writer.save()

In [126]:
df_for_param_nc = price.drop('Баркод', axis=1).copy()
mwm.unique_items(df_for_param_nc)
df_for_param_nc['date'] = wwm.dt_now.strftime('%d-%B-%Y')
df_for_param_nc = df_for_param_nc[['Артикул', 
       'Объемный вес', 'Объем товара, л',
       'Эквайринг', 'Комиссия в %', 'Логистика Ozon, минимум, FBO',
       'Логистика Ozon, максимум, FBO', 'Комиссия в % fbs',
       'Обработка отправления, минимум FBS',
       'Обработка отправления, максимум FBS', 'Логистика Ozon, минимум, FBS',
       'Логистика Ozon, максимум, FBS']]
df_for_param_nc = mwm.redact_columns_with_bd_table(df_for_param_nc)
df_for_param_nc['oz_general_localization_index'] = oz_general_localization_index
df_for_param_nc['oz_shipment_processing'] = oz_shipment_processing
df_for_param_nc['date'] = wwm.dt_now.strftime('%d-%B-%Y')
df_for_param_nc = df_for_param_nc.rename(columns={'Комиссия_в_': 'Комиссия_в_процентах'})
mwm.update_or_create_table_date(df_for_param_nc,
                                'oz_params_for_markups',
                                ['Артикул', 'date'],
                                'date')

In [127]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Себестоимость,Цена
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,243.71,585.0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,50.42,160.0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,45.37,126.0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,0,0,0,5,4,4,7,99,34.27,105.0
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,1,0,2,4,1,0,4,222,47.96,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,400.0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,100,317.29,386.0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,590.0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00,1824.0


In [128]:
# ozon_orders_metrics = calc_features_2(ozon_orders_metrics)
ozon_orders_metrics = calc_features_fbo(ozon_orders_metrics)

### Проверка

In [129]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,Последняя миля,Комиссия в процентах,Комиссия,Логистика,НДС,Налог на прибыль,Налог,% налога,Прибыль,Наценка
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,32.175,21.0,122.85,90.175,21.377500,19.915500,41.293000,7.058632,79.662000,32.69
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,20.000,21.0,33.60,78.000,-0.336667,-0.736667,-1.073333,-0.670833,-2.946667,-5.84
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,20.000,21.0,26.46,78.000,-3.971667,-4.287667,-8.259333,-6.555026,-17.150667,-37.80
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,20.000,21.0,22.05,78.000,-4.886667,-5.174667,-10.061333,-9.582222,-20.698667,-60.40
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,20.000,21.0,21.84,78.000,-7.300000,-7.560000,-14.860000,-14.288462,-30.240000,-63.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,22.000,19.0,76.00,91.000,38.833333,37.833333,76.666667,19.166667,151.333333,inf
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,21.230,19.0,73.34,79.230,-13.976667,-14.942667,-28.919333,-7.492055,-59.770667,-18.84
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,32.450,17.0,100.30,90.450,66.541667,65.065667,131.607333,22.306328,260.262667,inf
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,100.320,19.0,346.56,158.320,219.853333,215.293333,435.146667,23.856725,861.173333,inf


### Проверка метрик на русские буквы в "Артикулах"

In [130]:
len(ozon_orders_metrics['Артикул'].unique())

1231

In [131]:
ozon_orders_metrics = letter_substitution(ozon_orders_metrics)

In [132]:
len(ozon_orders_metrics['Артикул'].unique())

1231

In [133]:
len(ozon_orders_metrics)

1231

In [134]:
OZON_full_features_metric = ozon_orders_metrics.copy()

In [135]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,Последняя миля,Комиссия в процентах,Комиссия,Логистика,НДС,Налог на прибыль,Налог,% налога,Прибыль,Наценка
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,32.175,21.0,122.85,90.175,21.377500,19.915500,41.293000,7.058632,79.662000,32.69
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,20.000,21.0,33.60,78.000,-0.336667,-0.736667,-1.073333,-0.670833,-2.946667,-5.84
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,20.000,21.0,26.46,78.000,-3.971667,-4.287667,-8.259333,-6.555026,-17.150667,-37.80
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,20.000,21.0,22.05,78.000,-4.886667,-5.174667,-10.061333,-9.582222,-20.698667,-60.40
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,20.000,21.0,21.84,78.000,-7.300000,-7.560000,-14.860000,-14.288462,-30.240000,-63.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,22.000,19.0,76.00,91.000,38.833333,37.833333,76.666667,19.166667,151.333333,inf
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,21.230,19.0,73.34,79.230,-13.976667,-14.942667,-28.919333,-7.492055,-59.770667,-18.84
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,32.450,17.0,100.30,90.450,66.541667,65.065667,131.607333,22.306328,260.262667,inf
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,100.320,19.0,346.56,158.320,219.853333,215.293333,435.146667,23.856725,861.173333,inf


In [136]:
ozon_orders_metrics.loc[ozon_orders_metrics['Артикул']=='29604',:'Остаток']

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток
212,29604,"Пакеты для вакууматора, 15х500 см, 3 рулона",271753819,4643784894333,Shiny Kitchen,0,0,0,0,0,...,31,43,5,1,14,25,14,13,8,747


In [137]:
# buyouts = mwm.select_table('buyouts')

# buyouts = buyouts[buyouts['mplace']=='OZON'][['date', 'article', 'buyouts']]

# buyouts['article'] = buyouts['article'].astype(str)
# buyouts['buyouts'] = buyouts['buyouts'].astype(int)

# buyouts = wwm.expand_df(df=buyouts,
#               key_column = 'article',
#               result_column = 'buyouts')

# buyouts = buyouts.iloc[:, :-5]
# buyouts = buyouts.rename(columns={'article': 'Артикул'})

# mwm.unique_items(buyouts)

# buyouts_sub = pd.read_excel(path_downloads + 'IR Выкупы 2024 Q1.xlsx',
#               sheet_name=2,
#               usecols=[2, 95, 96, 97,98,99,100],
#               names=['Артикул', '27-February-2024', '28-February-2024', '29-February-2024', '01-March-2024','02-March-2024','03-March-2024']).dropna()
# buyouts_sub['Артикул'] = buyouts_sub['Артикул'].astype(str)
# buyouts_sub['27-February-2024'] = buyouts_sub['27-February-2024'].astype(int)
# buyouts_sub['28-February-2024'] = buyouts_sub['28-February-2024'].astype(int)
# buyouts_sub['29-February-2024'] = buyouts_sub['29-February-2024'].astype(int)
# buyouts_sub['01-March-2024'] = buyouts_sub['01-March-2024'].astype(int)
# buyouts_sub['02-March-2024'] = buyouts_sub['02-March-2024'].astype(int)
# buyouts_sub['03-March-2024'] = buyouts_sub['03-March-2024'].astype(int)
# buyouts_sub = buyouts_sub.groupby('Артикул').sum().reset_index(drop=False)

# mwm.unique_items(buyouts_sub)

# buyouts = buyouts.merge(buyouts_sub, on='Артикул', how='outer').fillna(0)
# buyouts['27-February-2024'] = buyouts['27-February-2024'].astype(int)
# buyouts['28-February-2024'] = buyouts['28-February-2024'].astype(int)
# buyouts['29-February-2024'] = buyouts['29-February-2024'].astype(int)
# buyouts_sub['01-March-2024'] = buyouts_sub['01-March-2024'].astype(int)
# buyouts_sub['02-March-2024'] = buyouts_sub['02-March-2024'].astype(int)
# buyouts_sub['03-March-2024'] = buyouts_sub['03-March-2024'].astype(int)

# ozon_orders_metrics222 = ozon_orders_metrics.copy()

# buyouts = buyouts[list(set(buyouts.columns) & set(ozon_orders_metrics.columns))]
# buyouts = buyouts[['Артикул']+[y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in dt_col(buyouts)])]]

# for i in range(len(ozon_orders_metrics)):
#     art = ozon_orders_metrics.loc[i]['Артикул']
#     if art in list(set(buyouts['Артикул'])):
#         for col in buyouts.columns[1:]:
#             r = ozon_orders_metrics.loc[i][col] - buyouts[buyouts['Артикул']==art][col].values[0]
#             if r > 0:
#                 ozon_orders_metrics.loc[i, col] = r
#             else:
#                 ozon_orders_metrics.loc[i, col] = r

In [138]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,Последняя миля,Комиссия в процентах,Комиссия,Логистика,НДС,Налог на прибыль,Налог,% налога,Прибыль,Наценка
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,32.175,21.0,122.85,90.175,21.377500,19.915500,41.293000,7.058632,79.662000,32.69
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,20.000,21.0,33.60,78.000,-0.336667,-0.736667,-1.073333,-0.670833,-2.946667,-5.84
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,20.000,21.0,26.46,78.000,-3.971667,-4.287667,-8.259333,-6.555026,-17.150667,-37.80
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,20.000,21.0,22.05,78.000,-4.886667,-5.174667,-10.061333,-9.582222,-20.698667,-60.40
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,20.000,21.0,21.84,78.000,-7.300000,-7.560000,-14.860000,-14.288462,-30.240000,-63.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,22.000,19.0,76.00,91.000,38.833333,37.833333,76.666667,19.166667,151.333333,inf
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,21.230,19.0,73.34,79.230,-13.976667,-14.942667,-28.919333,-7.492055,-59.770667,-18.84
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,32.450,17.0,100.30,90.450,66.541667,65.065667,131.607333,22.306328,260.262667,inf
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,100.320,19.0,346.56,158.320,219.853333,215.293333,435.146667,23.856725,861.173333,inf


### Создание таблицы для презентации

In [139]:
ozon_orders_metrics667 = ozon_orders_metrics.copy()

In [140]:
# list(set(ozon_orders_metrics.columns) - set(dt_col(ozon_orders_metrics)))

In [141]:
ozon_orders_metrics = table_for_presentation_new(ozon_orders_metrics)

/var/folders/kd/z6thdxpn5wgf06_21113yy0c0000gq/T/ipykernel_39750/2761686437.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data2, ignore_index=True)


### Проверка 

In [142]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,,Остаток,Цена,Наценка
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,0,1,2,0,1,...,0,0,0,0,0,0,,0,623.0,38.80
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,1,4,2,1,1,...,0,0,0,0,0,0,,0,608.0,37.22
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,1,0,0,1,0,...,0,0,0,0,0,0,,0,585.0,32.69
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,,0,722.0,59.67
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,19,4,7,5,3,...,1,0,1,0,1,1,,514,232.0,114.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,1,4,6,2,2,...,2,1,0,1,1,1,,721,1569.0,64.46
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,0,0,0,0,0,...,0,0,0,0,0,0,,0,3900.0,inf
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,24,19,10,5,1,...,2,2,1,0,1,2,,275,3354.0,100.48
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,6,6,3,5,2,...,1,2,4,1,3,1,,265,1770.0,98.69


In [143]:
ozon_orders_metrics667 = None

### Добавление расчетных промежуточных признаков

In [144]:
ozon_orders_metrics = add_intermediate_settlements(ozon_orders_metrics)

### Добавление РЦ

In [145]:
ozon_orders_metrics667 = ozon_orders_metrics.copy()

In [146]:
ozon_orders_metrics = add_rc(ozon_orders_metrics)

### Проверка

In [147]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,05-April-2024,06-April-2024,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,0,1,2,0,1,...,0,0,0,0,0,,0,623.0,38.80,3.0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,1,4,2,1,1,...,0,0,0,0,0,,0,608.0,37.22,4.0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,1,0,0,1,0,...,0,0,0,0,0,,0,585.0,32.69,4.0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,,0,722.0,59.67,0.0
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,19,4,7,5,3,...,1,0,1,4,1,,514,232.0,114.47,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,1,4,6,2,2,...,0,1,1,11,1,,721,1569.0,64.46,0.0
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,0,0,0,0,0,...,0,0,0,0,0,,0,3900.0,inf,0.0
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,24,19,10,5,1,...,1,0,1,14,2,,275,3354.0,100.48,4.0
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,6,6,3,5,2,...,4,1,3,11,1,,265,1770.0,98.69,7.0


In [148]:
ozon_orders_metrics667 = None

In [149]:
ozon_orders_metrics667 = ozon_orders_metrics.copy()

## Добавление и изменение статуса

In [150]:
Q = list(pd.read_excel(wwm.path_st,
         dtype={'Артикул': 'str'}).merge(ozon_orders_metrics[['Артикул']], 
                                         on='Артикул',
                                         how='right')['Статус'])
ozon_orders_metrics.insert(5, 'Статус', Q)

In [151]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,05-April-2024,06-April-2024,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,отказ,0,1,2,0,...,0,0,0,0,0,,0,623.0,38.80,3.0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,отказ,1,4,2,1,...,0,0,0,0,0,,0,608.0,37.22,4.0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,отказ,1,0,0,1,...,0,0,0,0,0,,0,585.0,32.69,4.0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,архив,0,0,0,0,...,0,0,0,0,0,,0,722.0,59.67,0.0
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,текущий,19,4,7,5,...,1,0,1,4,1,,514,232.0,114.47,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,текущий Дарья,1,4,6,2,...,0,1,1,11,1,,721,1569.0,64.46,0.0
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,архив,0,0,0,0,...,0,0,0,0,0,,0,3900.0,inf,0.0
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,текущий Дарья,24,19,10,5,...,1,0,1,14,2,,275,3354.0,100.48,4.0
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,текущий Дарья,6,6,3,5,...,4,1,3,11,1,,265,1770.0,98.69,7.0


### Проверка 

In [152]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,05-April-2024,06-April-2024,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,отказ,0,1,2,0,...,0,0,0,0,0,,0,623.0,38.80,3.0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,отказ,1,4,2,1,...,0,0,0,0,0,,0,608.0,37.22,4.0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,отказ,1,0,0,1,...,0,0,0,0,0,,0,585.0,32.69,4.0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,архив,0,0,0,0,...,0,0,0,0,0,,0,722.0,59.67,0.0
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,текущий,19,4,7,5,...,1,0,1,4,1,,514,232.0,114.47,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,текущий Дарья,1,4,6,2,...,0,1,1,11,1,,721,1569.0,64.46,0.0
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,архив,0,0,0,0,...,0,0,0,0,0,,0,3900.0,inf,0.0
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,текущий Дарья,24,19,10,5,...,1,0,1,14,2,,275,3354.0,100.48,4.0
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,текущий Дарья,6,6,3,5,...,4,1,3,11,1,,265,1770.0,98.69,7.0


In [153]:
ozon_orders_metrics['Статус'].unique()

array(['отказ', 'архив', 'текущий', 'текущий Дарья', 'сезонка',
       'новинка Даша', 'товары в пути', nan], dtype=object)

In [154]:
ozon_orders_metrics667 = None

### Дополнение и изменение имен

In [155]:
ozon_orders_metrics667 = ozon_orders_metrics.copy()

In [156]:
ozon_orders_metrics = redact_with_dop_name(ozon_orders_metrics)

### Проверка

In [157]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,05-April-2024,06-April-2024,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,отказ,0,1,2,0,...,0,0,0,0,0,,0,623.0,38.80,3.0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,отказ,1,4,2,1,...,0,0,0,0,0,,0,608.0,37.22,4.0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,отказ,1,0,0,1,...,0,0,0,0,0,,0,585.0,32.69,4.0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,архив,0,0,0,0,...,0,0,0,0,0,,0,722.0,59.67,0.0
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,текущий,19,4,7,5,...,1,0,1,4,1,,514,232.0,114.47,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,текущий Дарья,1,4,6,2,...,0,1,1,11,1,,721,1569.0,64.46,0.0
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,архив,0,0,0,0,...,0,0,0,0,0,,0,3900.0,inf,0.0
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,текущий Дарья,24,19,10,5,...,1,0,1,14,2,,275,3354.0,100.48,4.0
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,текущий Дарья,6,6,3,5,...,4,1,3,11,1,,265,1770.0,98.69,7.0




### Расчет и добавление "выручки", "доли", "итоговой наценки"

In [158]:
df = None

df = OZON_full_features_metric[['Артикул', 'Прибыль', 'Себестоимость', 'Наценка', dt_col(OZON_full_features_metric)[-1]]]\
     .rename(columns={dt_col(OZON_full_features_metric)[-1]: 'Продажи'})

df = df[(df['Себестоимость'].astype(str).isin(['0', 'nan', '0.0'])==False)\
        &(df['Продажи']>0)]\
     .reset_index(drop=True)

ITOG_NAC = round((((df['Прибыль'] * df['Продажи']).sum() / (df['Себестоимость'] *  df['Продажи']).sum()) * 100), 2)

df = None

In [159]:
ITOG_NAC

46.88

In [160]:
Q = []
for row in ozon_orders_metrics.columns[-20:]:
    
    try:
        row = pd.to_datetime(row)
        
        Q.append(row)
        
    except:
        
        None
        
Q.sort()

Q = list(map(lambda x: x.strftime(format='%d-%B-%Y'), Q))

ozon_orders_metrics['Выручка'] = ozon_orders_metrics['Цена'] * ozon_orders_metrics[Q[-1]] 
ozon_orders_metrics['Выручка'].loc[len(ozon_orders_metrics)-1] = 0
ozon_orders_metrics['Выручка'] = ozon_orders_metrics['Выручка'].astype(float)
s = ozon_orders_metrics['Выручка'].sum()
ozon_orders_metrics['Доля выручки'] = (ozon_orders_metrics['Выручка'] * 100) / s
ozon_orders_metrics['Выручка'].loc[len(ozon_orders_metrics)-1] = s
# ozon_orders_metrics['Итого наценка'] = ozon_orders_metrics['Доля выручки'] / 100 * ozon_orders_metrics['Наценка']
# e = ozon_orders_metrics.loc[ozon_orders_metrics['Итого наценка']!=np.inf, 'Итого наценка'].sum()
ozon_orders_metrics['Итого наценка'] = None
ozon_orders_metrics['Итого наценка'].loc[len(ozon_orders_metrics)-1] = ITOG_NAC




### Проверка

In [161]:
ozon_orders_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ,Выручка,Доля выручки,Итого наценка
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,отказ,0,1,2,0,...,0,0,,0,623.0,38.80,3.0,0.0,0.000000,None
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,отказ,1,4,2,1,...,0,0,,0,608.0,37.22,4.0,0.0,0.000000,None
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,отказ,1,0,0,1,...,0,0,,0,585.0,32.69,4.0,0.0,0.000000,None
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,архив,0,0,0,0,...,0,0,,0,722.0,59.67,0.0,0.0,0.000000,None
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,текущий,19,4,7,5,...,4,1,,514,232.0,114.47,0.0,232.0,0.009015,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,текущий Дарья,1,4,6,2,...,11,1,,721,1569.0,64.46,0.0,1569.0,0.060969,None
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,архив,0,0,0,0,...,0,0,,0,3900.0,inf,0.0,0.0,0.000000,None
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,текущий Дарья,24,19,10,5,...,14,2,,275,3354.0,100.48,4.0,6708.0,0.260665,None
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,текущий Дарья,6,6,3,5,...,11,1,,265,1770.0,98.69,7.0,1770.0,0.068780,None


In [162]:
ozon_orders_metrics667 = None

In [163]:
OZON_full_df = ozon_orders_metrics.copy()

ozon_orders_metrics = None

## Работа с "ТМ"

In [164]:
ozon_orders_metrics_non_orders = add_and_redact_brend(ozon_orders_metrics_non_orders)
ozon_price_metrics = add_and_redact_brend(ozon_price_metrics)
stocks_ozon_metrics = add_and_redact_brend(stocks_ozon_metrics)
OZON_full_features_metric = add_and_redact_brend(OZON_full_features_metric)
OZON_full_df = add_and_redact_brend(OZON_full_df)

In [165]:
OZON_full_df

,Артикул,Название,Номенклaтурa,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ,Выручка,Доля выручки,Итого наценка
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,отказ,0,1,2,0,...,0,0,,0,623.0,38.80,3.0,0.0,0.000000,None
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,отказ,1,4,2,1,...,0,0,,0,608.0,37.22,4.0,0.0,0.000000,None
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,отказ,1,0,0,1,...,0,0,,0,585.0,32.69,4.0,0.0,0.000000,None
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,архив,0,0,0,0,...,0,0,,0,722.0,59.67,0.0,0.0,0.000000,None
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,текущий,19,4,7,5,...,4,1,,514,232.0,114.47,0.0,232.0,0.009015,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,текущий Дарья,1,4,6,2,...,11,1,,721,1569.0,64.46,0.0,1569.0,0.060969,None
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,архив,0,0,0,0,...,0,0,,0,3900.0,inf,0.0,0.0,0.000000,None
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,текущий Дарья,24,19,10,5,...,14,2,,275,3354.0,100.48,4.0,6708.0,0.260665,None
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,текущий Дарья,6,6,3,5,...,11,1,,265,1770.0,98.69,7.0,1770.0,0.068780,None


## Выгрузка

### Основные метрики, без остатков

In [166]:
ozon_orders_metrics_non_orders

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,1,0,1,0,0,0,5,4,4,7
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,0,0,0,1,0,2,4,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [167]:
ozon_orders_metrics_non_orders = wwm.rename_data_from_reestr(ozon_orders_metrics_non_orders, 
                                                             'Название', 
                                                             reestr=reestr)

In [168]:
ozon_orders_metrics_non_orders.to_csv(path_ozon_orders_metrics, index=False, sep=";", encoding='utf-8-sig')

### Метрики по ценам

In [169]:
ozon_price_metrics[ozon_price_metrics[ozon_price_metrics.columns[-1]].isna()]

,Артикул,Название,Номенклaтурa,Баркод,ТМ,26-August-2022,29-August-2022,30-August-2022,31-August-2022,01-September-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
1010,J0601,Контейнеры для обуви. 10 шт,962815830,4636526732628,Birdhouse,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1198,A6707,Бутылка для воды розовая,1119350800,4663404619290,Limpide Side,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1201,G0643,"Бра белые, 2шт",0,4664989025858,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1202,G0642,"Бра черные, 2шт",0,4664969672386,SSY,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [170]:
ozon_price_metrics[ozon_price_metrics.columns[6:]]=ozon_price_metrics[ozon_price_metrics.columns[6:]].fillna(0).astype('int64')
ozon_price_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,26-August-2022,29-August-2022,30-August-2022,31-August-2022,01-September-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,648,648,648,648,648,...,836,836,836,836,836,836,585,585,585,585
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,166,166,166,166,166,...,160,160,160,160,160,160,160,160,160,160
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,198,198,198,198,198,...,180,180,180,180,180,180,180,126,126,126
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,198,198,198,198,198,...,142,142,150,150,150,150,150,105,105,105
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,175,175,175,175,175,...,148,148,148,148,148,148,148,104,104,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,400,400,400,400,400,400,400,400,400,400
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,575,575,552,552,552,552,552,386,386,386
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,590,590,590,590,590,590,590,590,590,590
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,1824,1824,1824,1824,1824,1824,1824,1824,1824,1824


In [171]:
ozon_price_metrics = wwm.rename_data_from_reestr(ozon_price_metrics, 
                                                 'Название', 
                                                 reestr=reestr)

In [172]:
ozon_price_metrics.to_csv(path_ozon_price_metrics, index=False, sep=";", encoding='utf-8-sig')

### Метрики по остаткам

In [173]:
stocks_ozon_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,26-August-2022-Ost,29-August-2022-Ost,30-August-2022-Ost,31-August-2022-Ost,01-September-2022-Ost,...,27-March-2024-Ost,28-March-2024-Ost,29-March-2024-Ost,01-April-2024-Ost,02-April-2024-Ost,03-April-2024-Ost,04-April-2024-Ost,05-April-2024-Ost,08-April-2024-Ost,09-April-2024-Ost
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,20,20,50,50,48,...,0,0,0,0,0,0,0,0,0,0
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,146,118,311,297,285,...,0,0,0,0,0,0,0,0,0,0
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,275,272,272,272,271,...,0,0,0,0,0,0,0,0,0,0
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,232,214,208,204,199,...,125,123,122,121,120,120,120,119,106,99
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,328,325,325,325,322,...,236,236,236,236,236,235,235,233,228,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,100,100,100,100,100,100,100,100,100,100
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [174]:
stocks_ozon_metrics = wwm.rename_data_from_reestr(stocks_ozon_metrics, 
                                                  'Название', 
                                                  reestr=reestr)

In [175]:
stocks_ozon_metrics.to_csv(path_stocks_ozon_metrics, index=False, sep=";", encoding='utf-8-sig')

### Метрики с вычисленными признаками и остатками

In [176]:
OZON_full_features_metric

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,Последняя миля,Комиссия в процентах,Комиссия,Логистика,НДС,Налог на прибыль,Налог,% налога,Прибыль,Наценка
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,32.175,21.0,122.85,90.175,21.377500,19.915500,41.293000,7.058632,79.662000,32.69
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,20.000,21.0,33.60,78.000,-0.336667,-0.736667,-1.073333,-0.670833,-2.946667,-5.84
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,20.000,21.0,26.46,78.000,-3.971667,-4.287667,-8.259333,-6.555026,-17.150667,-37.80
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,20.000,21.0,22.05,78.000,-4.886667,-5.174667,-10.061333,-9.582222,-20.698667,-60.40
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,20.000,21.0,21.84,78.000,-7.300000,-7.560000,-14.860000,-14.288462,-30.240000,-63.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,22.000,19.0,76.00,91.000,38.833333,37.833333,76.666667,19.166667,151.333333,inf
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,21.230,19.0,73.34,79.230,-13.976667,-14.942667,-28.919333,-7.492055,-59.770667,-18.84
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,32.450,17.0,100.30,90.450,66.541667,65.065667,131.607333,22.306328,260.262667,inf
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,100.320,19.0,346.56,158.320,219.853333,215.293333,435.146667,23.856725,861.173333,inf


In [177]:
# OZON_full_features_metric.to_csv(path_OZON_full_features_metric, index=False, sep=";", encoding='utf-8-sig')

### Добавление "Цены до скидки"

In [178]:
OZON_full_df = OZON_full_df.merge(price[['Артикул', 'Цена до скидки']], on='Артикул', how='left')

if len(OZON_full_df) - len(list(set(OZON_full_df['Артикул']))) !=0:
    print('error')
    int('error')
else:
    None

In [179]:
# OZON_full_features_metric = OZON_full_features_metric.merge(price[['Артикул', 'Цена до скидки']], on='Артикул', how='left')

# if len(OZON_full_features_metric) - len(list(set(OZON_full_features_metric['Артикул']))) !=0:
#     print('error')
#     int('error')
# else:
#     None
    
# price_dop_2 = None

In [180]:
OZON_full_features_metric

,Артикул,Название,Номенклaтурa,Баркод,ТМ,05-April-2021,06-April-2021,07-April-2021,08-April-2021,09-April-2021,...,Последняя миля,Комиссия в процентах,Комиссия,Логистика,НДС,Налог на прибыль,Налог,% налога,Прибыль,Наценка
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,0,0,0,0,0,...,32.175,21.0,122.85,90.175,21.377500,19.915500,41.293000,7.058632,79.662000,32.69
1,18309,"Чехол для airpods 2 с карабином, черный",149314939,7432141050065,SS&Y Group,0,0,0,0,0,...,20.000,21.0,33.60,78.000,-0.336667,-0.736667,-1.073333,-0.670833,-2.946667,-5.84
2,18308,"Чехол для airpods 2 с карабином, голубой",149314941,7432141050058,SS&Y Group,0,0,0,0,0,...,20.000,21.0,26.46,78.000,-3.971667,-4.287667,-8.259333,-6.555026,-17.150667,-37.80
3,18307,"Чехол для airpods 2 с карабином, сиреневый",149295199,7432141649689,SS&Y Group,0,0,0,0,0,...,20.000,21.0,22.05,78.000,-4.886667,-5.174667,-10.061333,-9.582222,-20.698667,-60.40
4,18306,"Чехол для airpods 2 с карабином, коралловый",149459576,7432141649672,SS&Y Group,0,0,0,0,0,...,20.000,21.0,21.84,78.000,-7.300000,-7.560000,-14.860000,-14.288462,-30.240000,-63.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1393960786,4673766212725,Shiny Kitchen,0,0,0,0,0,...,22.000,19.0,76.00,91.000,38.833333,37.833333,76.666667,19.166667,151.333333,inf
1227,28900,"Массажер гуаша для лица, розовый кварц",167276817,7432141540883,SSY,0,0,0,0,0,...,21.230,19.0,73.34,79.230,-13.976667,-14.942667,-28.919333,-7.492055,-59.770667,-18.84
1228,50459,"Расческа с зеркальцем матовая, черная",1469118732,4674382511209,REVOLUT,0,0,0,0,0,...,32.450,17.0,100.30,90.450,66.541667,65.065667,131.607333,22.306328,260.262667,inf
1229,A8304,Набор для специй 6 баночек,1469193842,4674444646573,Shiny Kitchen,0,0,0,0,0,...,100.320,19.0,346.56,158.320,219.853333,215.293333,435.146667,23.856725,861.173333,inf


### Дополнительный признак "сервис"

In [181]:
spr_wb = pd.read_excel(path_spr_wb, 
                       dtype={'Артикул': 'str'})

spr_oz = pd.read_excel(path_spr_oz, 
                       dtype={'Артикул': 'str'})

OZON_full_df = OZON_full_df.merge(spr_oz, on='Артикул', how='left')\
             .merge(spr_wb, on='Артикул', how='left')

In [182]:
OZON_full_df

,Артикул,Название,Номенклaтурa,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,Остаток,Цена,Наценка,Остаток на РЦ,Выручка,Доля выручки,Итого наценка,Цена до скидки,Спринт озон,Спринт вб
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,отказ,0,1,2,0,...,0,623.0,38.80,3.0,0.0,0.000000,None,2996.0,NaN,NaN
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,отказ,1,4,2,1,...,0,608.0,37.22,4.0,0.0,0.000000,None,2996.0,NaN,NaN
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,отказ,1,0,0,1,...,0,585.0,32.69,4.0,0.0,0.000000,None,2996.0,NaN,NaN
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,архив,0,0,0,0,...,0,722.0,59.67,0.0,0.0,0.000000,None,2996.0,NaN,NaN
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,текущий,19,4,7,5,...,514,232.0,114.47,0.0,232.0,0.009015,None,1010.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,текущий Дарья,1,4,6,2,...,721,1569.0,64.46,0.0,1569.0,0.060969,None,2399.0,NaN,NaN
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,архив,0,0,0,0,...,0,3900.0,inf,0.0,0.0,0.000000,None,6630.0,NaN,NaN
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,текущий Дарья,24,19,10,5,...,275,3354.0,100.48,4.0,6708.0,0.260665,None,6590.0,ТЗ 30 (16.10 - 25.10),NaN
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,текущий Дарья,6,6,3,5,...,265,1770.0,98.69,7.0,1770.0,0.068780,None,2550.0,ТЗ 34 (21.11-01.12),NaN


### Добавление оценки IDC и доп. признаков

In [183]:
if len(OZON_full_df) - len(list(set(OZON_full_df['Артикул']))) != 0:
    print('error')
    int('error')
else:
    if len(in_tur_oz) - len(list(set(in_tur_oz['Артикул']))) != 0:
        print('error')
        int('error')
    else:
        OZON_full_df = OZON_full_df.merge(in_tur_oz, on='Артикул', how='left')

### Доп.

In [184]:
Q = []
for i in range(len(OZON_full_df)):
    if OZON_full_df.loc[i]['Артикул'] in list(set(one['Артикул'])):
        Q.append(1)
    else:
        Q.append(None)
        
OZON_full_df['1'] = Q

### Доп. Шаблон цена

In [185]:
price

,Артикул,Баркод,Объемный вес,"Объем товара, л",Цена,Цена до скидки,Эквайринг,Комиссия в %,"Логистика Ozon, минимум, FBO","Логистика Ozon, максимум, FBO",Комиссия в % fbs,"Обработка отправления, минимум FBS","Обработка отправления, максимум FBS","Логистика Ozon, минимум, FBS","Логистика Ozon, максимум, FBS"
0,16803,7432141949932,0.4,0.6750,585,2996.0,7.31,21,31.5,75.6,22.5,5.0,30.0,76.0,76.0
1,18309,7432141050065,0.1,0.1280,160,1010.0,2.00,21,31.5,75.6,22.5,5.0,30.0,76.0,76.0
2,18308,7432141050058,0.1,0.0360,126,1010.0,1.58,21,31.5,75.6,22.5,5.0,30.0,76.0,76.0
3,18307,7432141649689,0.1,0.2090,115,1010.0,1.44,21,31.5,75.6,22.5,5.0,30.0,76.0,76.0
4,18306,7432141649672,0.1,0.1050,104,1010.0,1.30,21,31.5,75.6,22.5,5.0,30.0,76.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222,50976,4673660046877,0.1,0.2160,390,1010.0,4.88,17,31.5,75.6,18.0,5.0,30.0,76.0,76.0
1223,A5403,4673766212725,1.3,6.4125,400,1010.0,5.00,19,38.5,92.4,20.0,5.0,30.0,94.0,94.0
1224,50458,4674076931641,0.1,0.3200,590,1010.0,7.38,17,31.5,75.6,18.0,5.0,30.0,76.0,76.0
1225,50459,4674382511209,0.1,0.3200,590,1010.0,7.38,17,31.5,75.6,18.0,5.0,30.0,76.0,76.0


In [186]:
price_dop = price_dop[['Артикул', 'Рыночная цена конкурентов, руб.', 'Моя цена на других площадках, руб.', 'Индекс цен', 'Ценовой индекс товара на Ozon', 'Ценовой индекс товара на рынке', 'Ценовой индекс товара на рынке на мои товары', 'Ссылка на рыночную цену конкурентов']]
price_dop['Артикул'] = price_dop['Артикул'].astype(str)

In [187]:
dpr = price_dop


In [188]:
OZON_full_df['                              '] = None
OZON_full_df = OZON_full_df.merge(dpr, on='Артикул', how='left')

In [189]:
ozon_fbs_metrics = pd.read_csv(path_ozon_fbs_metrics, sep=';')
ozon_fbs_metrics['Артикул'] = ozon_fbs_metrics['Артикул'].astype(str)
ozon_fbs = pd.read_csv(path_ozon_fbs, sep=';')
ozon_fbs = ozon_fbs[['Артикул', 'Доступно на моих складах, шт']]
ozon_fbs['Доступно на моих складах, шт'] = ozon_fbs['Доступно на моих складах, шт'].astype(int)
ozon_fbs = ozon_fbs[ozon_fbs['Доступно на моих складах, шт']!=0].reset_index(drop=True)
ozon_fbs['Артикул'] = ozon_fbs['Артикул'].astype(str)
ozon_fbs = ozon_fbs.groupby('Артикул')['Доступно на моих складах, шт'].sum().reset_index(drop=False)
ozon_fbs['Доступно на моих складах, шт'] = ozon_fbs['Доступно на моих складах, шт'].astype(int)
ozon_fbs.columns = ['Артикул', 'Остаток fbs']
dtn = datetime.datetime.now().strftime('%d-%B-%Y')
if dtn not in ozon_fbs_metrics.columns:
    ozon_fbs_2 = ozon_fbs.copy()
    ozon_fbs_2.columns = ['Артикул', dtn]
    ozon_fbs_metrics = ozon_fbs_metrics.merge(ozon_fbs_2, on='Артикул', how='outer')
    ozon_fbs_2 = None
else:
    None

In [190]:
ozon_fbs_metrics[dtn].sum()

31200.0

In [191]:
OZON_full_df = OZON_full_df.merge(ozon_fbs, on='Артикул', how='left')

In [192]:
LIST = list(OZON_full_df.columns[:OZON_full_df.columns.get_loc('Цена')]) + ['Остаток fbs'] + list(OZON_full_df.columns[OZON_full_df.columns.get_loc('Цена'):-1])

In [193]:
OZON_full_df = OZON_full_df[LIST]

In [194]:
OZON_full_df = OZON_full_df.merge(oz_rank_feature_metrics, on='Артикул', how='left')

#### Добавление наценок и цен по акциям

In [195]:
# def margin_and_price_dop_ac(data, data_features): # data - DF_ac or DF_ac_2, data_features - OZON_full_features_metric
    
#     data_features['Артикул'] = data_features['Артикул'].astype(str)
    
#     for col in data.columns:
#         if 'Артикул' not in col:
#             data[col] = data[col].astype(float)

#     data_features_2 = data_features.copy()
#     data_features_2 = data_features_2[['Артикул', 'Эквайринг', 'Комиссия', 'Логистика', 'Себестоимость']]
#     data = data.merge(data_features_2, on='Артикул')


#     data_features_2 = None


#     for col in list(data.columns[1:-4]):
#         perc = 0
#         for y in [f'-{x}%' for x in range(0, 100)]:
#             if y in col:
#                 perc = int(y[1])

#         data['Наценка_'+col[:-5]] = add_settlement_features_ac(data, col, perc)


#     data = data[['Артикул'] + list(filter(lambda x: 'Цена' in x, list(data.columns)))\
#                 + list(filter(lambda x: 'Наценка' in x, list(data.columns)))]



#     for i in range(len(data)):

#         for row in [x[:-5] for x in list(filter(lambda x: 'Цена' in x, list(data.columns)))]:
#             if data.loc[i][row + ' Цена'] == 0:
#                 data[row + ' Цена'].loc[i] = None
#                 data['Наценка_' + row].loc[i] = None
#             else:
#                 None  
                
#     return data

In [196]:
Col_for_del_oz = []
for row in DF_ac.columns[1:]:
    OZON_full_df = OZON_full_df\
    .merge(add_settlement_features_new(OZON_full_features_metric[['Артикул', 
                               'Комиссия в %', 
                               'Объем товара, руб', 
                               'Последняя миля', 
                               'Себестоимость', 
                               'Эквайринг']]\
    .merge(DF_ac[['Артикул', row]], on='Артикул', how='inner')\
    .rename(columns={row: 'Цена'}))\
    .rename(columns={'Цена': row, 'Наценка': row\
    .split('Цена')[0] + 'Наценка'})[['Артикул', row, row.split('Цена')[0] + 'Наценка']], on='Артикул', how='left')
    Col_for_del_oz.append(row)
    Col_for_del_oz.append(row.split('Цена')[0] + 'Наценка')

In [197]:
Col_for_del_oz

['Хиты. Без ограничения срока действия Цена',
 'Хиты. Без ограничения срока действия Наценка',
 'СуперХиты. Без ограничения срока действия Цена',
 'СуперХиты. Без ограничения срока действия Наценка',
 'СуперХиты со скидкой на комиссию - 5%.Без ограничения срока действия Цена',
 'СуперХиты со скидкой на комиссию - 5%.Без ограничения срока действия Наценка',
 ' МегаХиты. Без ограничения срока действия Цена',
 ' МегаХиты. Без ограничения срока действия Наценка',
 'МегаХиты со скидкой на комиссию - 5%. Без ограничения срока действия Цена',
 'МегаХиты со скидкой на комиссию - 5%. Без ограничения срока действия Наценка']

In [198]:
pd.DataFrame(Col_for_del_oz, 
             columns=['Col_for_not_del']).to_excel(path_downloads + 'Col_for_not_del_oz.xlsx', index=False)

In [199]:
pd.DataFrame(Col_for_del_oz, columns=['Col_for_del_oz'])\
.to_excel(path_downloads + 'Col_for_del_oz.xlsx', index=False)
Col_for_del_oz=None

### Категории

In [200]:
OZON_full_df = OZON_full_df.merge(cat, on='Артикул', how='left')

In [201]:
obor = (OZON_full_df[['Артикул']+dt_col(OZON_full_df)[-14:]+['Остаток']].iloc[:-1, :]).copy()
obor['mean_orders_14'] = [obor[dt_col(obor)].loc[x].mean() for x in list(obor.index)]
obor = obor[obor['Остаток'] > 0].reset_index(drop=True)
obor = obor[obor['mean_orders_14'] > 0].reset_index(drop=True)
obor['turnover'] = (obor['Остаток'] / obor['mean_orders_14']).round(2)
obor = obor[['Артикул', 'turnover']].reset_index(drop=True)
obor = obor[obor['turnover'] >= 30]

### Оборачиваемость

In [202]:
OZON_full_df = OZON_full_df.merge(obor, on='Артикул', how='left')

### В резерве

In [203]:
vr = pd.read_excel(path_vr)
vr['Артикул'] = vr['Артикул'].astype(str)

OZON_full_df = OZON_full_df.merge(vr, on='Артикул', how='left')

OZON_full_df = OZON_full_df[list(OZON_full_df.columns[:OZON_full_df.columns.get_loc('Остаток на РЦ')+1]) + ['В резерве'] + list(OZON_full_df.columns[OZON_full_df.columns.get_loc('Остаток на РЦ')+1:-1])]



### Добавление наценки по FBS

In [204]:
ozon_orders_metrics_fbs_new

,Артикул,Наценка_fbs
0,16803,24.27
1,18309,-38.11
2,18308,-72.91
3,18307,-106.26
4,18306,-95.80
...,...,...
1226,A5403,inf
1227,28900,-24.27
1228,50459,inf
1229,A8304,inf


In [205]:
OZON_full_df = OZON_full_df.merge(ozon_orders_metrics_fbs_new, on='Артикул', how='left')

In [206]:
OZON_full_df

,Артикул,Название,Номенклaтурa,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,СуперХиты. Без ограничения срока действия Наценка,СуперХиты со скидкой на комиссию - 5%.Без ограничения срока действия Цена,СуперХиты со скидкой на комиссию - 5%.Без ограничения срока действия Наценка,МегаХиты. Без ограничения срока действия Цена,МегаХиты. Без ограничения срока действия Наценка,МегаХиты со скидкой на комиссию - 5%. Без ограничения срока действия Цена,МегаХиты со скидкой на комиссию - 5%. Без ограничения срока действия Наценка,Категория,turnover,Наценка_fbs
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,отказ,0,1,2,0,...,75.32,747.0,65.60,747.0,65.60,703.0,56.09,NaN,NaN,29.95
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,отказ,1,4,2,1,...,72.87,0.0,-94.21,729.0,63.36,0.0,-94.21,Miscellaneous accessories,NaN,28.70
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,отказ,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miscellaneous accessories,NaN,24.27
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miscellaneous accessories,NaN,50.69
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,текущий,19,4,7,5,...,103.83,0.0,-194.40,218.0,95.84,0.0,-194.40,School&Stationery&Equipment,1028.00,76.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,текущий Дарья,1,4,6,2,...,51.46,1396.0,47.67,1396.0,47.67,1317.0,40.01,NaN,348.07,59.82
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,текущий Дарья,24,19,10,5,...,94.71,3244.0,94.19,3244.0,94.19,3132.0,87.80,NaN,104.05,95.86
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,текущий Дарья,6,6,3,5,...,81.42,1566.0,76.81,1566.0,76.81,1479.0,67.47,NaN,115.94,93.43


In [207]:
list(OZON_full_df.columns)

['Артикул',
 'Название',
 'Номенклaтурa',
 'Баркод',
 'ТМ',
 'Статус',
 '20-February-2024',
 '21-February-2024',
 '22-February-2024',
 '23-February-2024',
 '24-February-2024',
 '25-February-2024',
 '26-February-2024',
 '27-February-2024',
 '28-February-2024',
 '29-February-2024',
 '01-March-2024',
 '02-March-2024',
 '03-March-2024',
 '26-February-03-March',
 '04-March-2024',
 '05-March-2024',
 '06-March-2024',
 '07-March-2024',
 '08-March-2024',
 '09-March-2024',
 '10-March-2024',
 '04-March-10-March',
 '11-March-2024',
 '12-March-2024',
 '13-March-2024',
 '14-March-2024',
 '15-March-2024',
 '16-March-2024',
 '17-March-2024',
 '11-March-17-March',
 '18-March-2024',
 '19-March-2024',
 '20-March-2024',
 '21-March-2024',
 '22-March-2024',
 '23-March-2024',
 '24-March-2024',
 '18-March-24-March',
 '25-March-2024',
 '26-March-2024',
 '27-March-2024',
 '28-March-2024',
 '29-March-2024',
 '30-March-2024',
 '31-March-2024',
 '25-March-31-March',
 '01-April-2024',
 '02-April-2024',
 '03-April-2

In [208]:
len(ozon_orders_metrics_non_orders) - len(list(set(ozon_orders_metrics_non_orders['Артикул'])))

0

In [209]:
len(ozon_price_metrics) - len(list(set(ozon_price_metrics['Артикул'])))

0

In [210]:
len(stocks_ozon_metrics) - len(list(set(stocks_ozon_metrics['Артикул'])))

0

In [211]:
len(OZON_full_features_metric) - len(list(set(OZON_full_features_metric['Артикул'])))

0

In [212]:
len(OZON_full_df) - len(list(set(OZON_full_df['Артикул'])))

0

### Работа с метриками по "объемам"

In [213]:
ozon_size_metrics = redact_ozon_size_metrics()

### Работа с общей итоговой наценкой

In [214]:
# OZON_full_df['Итого наценка'] = OZON_full_df['Итого наценка'].fillna(0.0)

In [215]:
# for i in range(len(OZON_full_df)):
#     if 'inf' in str(OZON_full_df.loc[i]['Итого наценка']):
#         OZON_full_df['Итого наценка'].loc[i] = 0

In [216]:
# OZON_full_df['Итого наценка'].loc[len(OZON_full_df)-1] = (OZON_full_df.iloc[:-1, :]['Итого наценка'].sum())

In [217]:
# OZON_full_df[['Итого наценка']]

### Добавление комментария по ассортименту

In [218]:
# OZON_full_df = OZON_full_df.merge(dop_comment_a, on='Номенклaтурa', how='left')

In [219]:
len(ozon_size_metrics) - len(list(set(ozon_size_metrics['Артикул'])))

0

In [220]:
len(ozon_fbs_metrics) - len(list(set(ozon_fbs_metrics['Артикул'])))

0

### Добавление аналитики по ценам за последние несколько дней

In [221]:
ozap = ozon_price_metrics[['Артикул']+dt_col(ozon_price_metrics)].copy()

def list_none(List):
    for i in range(len(List)):
        if List[i] == 0:
            List[i] = None
    return List
    
# Разница в цене предыдущего дня
Q = []
Q = list_none(ozap[dt_col(ozon_price_metrics)[-1]] - ozap[dt_col(ozon_price_metrics)[-2]])
    
# Стандартное отклонение цен за последние 7 дней
W = []
W = list_none([np.std(list(ozap.iloc[x, -7:])).round(2) for x in list(ozap.index)])

# Стандартное отклонение цен за последние 3 дня
E = []
E = list_none([np.std(list(ozap.iloc[x, -3:])).round(2) for x in list(ozap.index)])

# Отличие максимальной цены от минимальной за 3 дня 

R = []
R = list_none([(ozap.iloc[x, -3:].max() - ozap.iloc[x, -3:].min()).round(2) for x in list(ozap.index)])



ozap = ozap[['Артикул']]
ozap['Разница в цене предыдущего дня'] = Q
ozap['Стандартное отклонение цен за последние 7 дней'] = W
ozap['Стандартное отклонение цен за последние 3 дня'] = E
ozap['Отличие максимальной цены от минимальной за 3 дня'] = R


OZON_full_df = OZON_full_df.merge(ozap, on='Артикул', how='left')

In [222]:
OZON_full_df

,Артикул,Название,Номенклaтурa,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,МегаХиты. Без ограничения срока действия Наценка,МегаХиты со скидкой на комиссию - 5%. Без ограничения срока действия Цена,МегаХиты со скидкой на комиссию - 5%. Без ограничения срока действия Наценка,Категория,turnover,Наценка_fbs,Разница в цене предыдущего дня,Стандартное отклонение цен за последние 7 дней,Стандартное отклонение цен за последние 3 дня,Отличие максимальной цены от минимальной за 3 дня
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,отказ,0,1,2,0,...,65.60,703.0,56.09,NaN,NaN,29.95,NaN,132.13,None,None
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,отказ,1,4,2,1,...,63.36,0.0,-94.21,Miscellaneous accessories,NaN,28.70,NaN,129.16,None,None
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,отказ,1,0,0,1,...,NaN,NaN,NaN,Miscellaneous accessories,NaN,24.27,NaN,124.21,None,None
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,архив,0,0,0,0,...,NaN,NaN,NaN,Miscellaneous accessories,NaN,50.69,NaN,23.26,None,None
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,текущий,19,4,7,5,...,95.84,0.0,-194.40,School&Stationery&Equipment,1028.00,76.40,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,текущий Дарья,1,4,6,2,...,47.67,1317.0,40.01,NaN,348.07,59.82,NaN,NaN,None,None
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,inf,NaN,NaN,None,None
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,текущий Дарья,24,19,10,5,...,94.19,3132.0,87.80,NaN,104.05,95.86,NaN,NaN,None,None
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,текущий Дарья,6,6,3,5,...,76.81,1479.0,67.47,NaN,115.94,93.43,NaN,NaN,None,None


In [223]:
rc_irc = list(OZON_full_df[['Артикул']]\
         .merge(pd.read_excel(path_downloads + 'РЦ Иркутск.xlsx', 
                dtype={'Артикул': 'str'}).groupby('Артикул')['Остаток РЦ Иркутск'].sum()\
                .reset_index(drop=False), 
                on='Артикул', 
                how='left')['Остаток РЦ Иркутск'])

OZON_full_df.insert(OZON_full_df.columns.get_loc('Остаток на РЦ')+1, 'Остаток РЦ Иркутск', rc_irc)

## цена до скидки

In [224]:
Q = list(OZON_full_df['Цена до скидки'])
OZON_full_df = OZON_full_df.drop('Цена до скидки', axis=1)
OZON_full_df.insert(list(OZON_full_df.columns).index('Цена')+1, 'Цена до скидки', Q)

## прибыль с единицы товара

In [225]:
OZON_full_df = OZON_full_df.merge(OZON_full_features_metric[['Артикул', 
                                                       'Прибыль']]\
                              .rename(columns={'Прибыль': 'прибыль с единицы товара'}),
                              on='Артикул', 
                              how='left') 
OZON_full_df['прибыль с единицы товара'] = OZON_full_df['прибыль с единицы товара'].round(2)

## Создание таблицы с изменением цен

In [226]:
if len(ozon_price_metrics) - len(list(set(ozon_price_metrics['Артикул']))) !=0:
    print('error')
    int('error')
else:
    ozon_price_difference = ozon_price_metrics[['Артикул']+dt_col(ozon_price_metrics)[-13:]].copy()
    cols_for_analysis = list(ozon_price_difference.columns)[1:][::-1]
    for i in range(len(cols_for_analysis)-1):
        ozon_price_difference['Изм. цены ' + pd.to_datetime(cols_for_analysis[i])\
                            .strftime('%d.%m')] = ozon_price_difference[cols_for_analysis[i]]\
                                                    - ozon_price_difference[cols_for_analysis[i+1]]
#         ozon_price_difference['Цена '+ pd.to_datetime(cols_for_analysis[i])\
#                             .strftime('%d.%m')] = ozon_price_difference[cols_for_analysis[i]]
    ozon_price_difference = ozon_price_difference.drop(cols_for_analysis, axis=1)
    ozon_price_difference = ozon_price_difference[['Артикул']\
                                             + list(ozon_price_difference.columns[1:])[::-1]]
    C_price = pd.read_excel(path_downloads + 'Целевая цена.xlsx',
                            sheet_name='ozon', 
                            dtype={'Артикул': 'str'})
    ozon_price_difference = ozon_price_difference.merge(C_price, on='Артикул', how='left')
    ozon_price_difference.to_excel(path_downloads + 'Изменения цены озон.xlsx', index=False)

## Работа с остатками в движении

In [227]:
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True
wb = xlrd.open_workbook(path_os)
sh = wb.sheet_by_name('Остатки и товары в пути')
header = sh.row_values(3, start_colx=0, end_colx=None)
os = pd.DataFrame()
for i in range(sh.ncols):
    os[header[i]] = sh.col_values(i, start_rowx=4, end_rowx=None)
    
for row in ['Артикул', 'Название склада', 'Название товара']:
    os[row] = os[row].astype(str)
    
os['Доступный к продаже товар'] = os['Доступный к продаже товар'].astype(int)
    
os = os.query(f"Артикул not in {list(set(list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(os['Артикул'])))))}").reset_index(drop=True)
    
def replace_fresh_name(y):
    if 'FRESH' in y:
        y = 'FRESH'
    else:
        None
    return y

os['Название склада'] = list(map(lambda x: replace_fresh_name(x), list(os['Название склада'])))
osn = os[['Артикул', 'SKU', 'Название товара']].copy()
osn = osn.drop_duplicates().reset_index(drop=True)
for row in list(os['Название склада'].unique()):
    df = os[os['Название склада']==row].groupby('Артикул')['Доступный к продаже товар'].sum().reset_index(drop=False).copy()
    df = df.rename(columns={df.columns[-1]:row})
    osn = osn.merge(df, on='Артикул', how='left')
os = osn
osn = None

if len(os) - len(list(set(os['SKU']))) != 0:
    print('error')
    int('error')
else:
    None
    
    
Q = []
for i in range(len(os)):
    art = os.loc[i]['Артикул']
    if len(os[os['Артикул']==art]) >1:
        for sku in sorted(os[os['Артикул']==art]['SKU'].values)[:-1]:
            Q.append(sku)
Q = list(set(Q))
os = os.query('SKU not in @Q').reset_index(drop=True)

os['Артикул'] = os['Артикул'].astype(str)

if len(os) - len(list(set(os['Артикул']))) != 0:
    print('error')
    int('error')
else:
    None
    
    
os = os.merge(OZON_full_features_metric[['Артикул', 'Остаток']], on='Артикул', how='left')


# def unloading():
    
#     with pd.ExcelWriter(path_algoritm + 'Все файлы для ежедневной презентации/Остатки товаров в движении ОЗОН.xlsx') as writer:

#         # use to_excel function and specify the sheet_name and index
#         # to store the dataframe in specified sheet
#         os.to_excel(writer, sheet_name="Остатки товаров в движении ОЗОН", index=False)
#         for column in os:
#             column_width = max(os[column].astype(str).map(len).max()+15, len(column))
#             col_idx = os.columns.get_loc(column)
#             writer.sheets['Остатки товаров в движении ОЗОН'].set_column(col_idx, col_idx, column_width)


#     writer.save()



# try:
#     unloading()
# except:
#     unloading()

In [228]:
Moscow_warehouses = []
for row in [['гривно', 'рфц'], 
            ['гривно', 'негабарит'],
            ['давыдовское', 'негабарит'],
            ['павло-слободское', 'негабарит'],
            ['петровское', 'рфц'], 
            ['хоругвино', 'негабарит'],
            ['хоругвино', 'рфц']]:
    sklad = [x for x in list(os.columns) if row[0] in x.lower() and row[1] in x.lower()]
    if len(sklad) > 0:
        for sk in sklad:
            Moscow_warehouses.append(sk)
Moscow_warehouses = list(set(Moscow_warehouses))
        
os_M = os[['Артикул']+ Moscow_warehouses].copy()
os_M['Moscow_warehouses'] = [os_M.iloc[x, 1:].sum() for x in list(os_M.index)]
os_M = os_M[['Артикул', 'Moscow_warehouses']]


OZON_full_df = OZON_full_df.merge(os_M, on='Артикул', how='left')

### Доп признак от Славы

In [229]:
dop_columns_for_Slava = pd.read_excel(path_downloads + 'Поставки ВБ и Озон.xlsx', 
                                      sheet_name='Озон',
                                      dtype={'Артикул': 'str'})
OZON_full_df = OZON_full_df.merge(dop_columns_for_Slava, on='Артикул', how='left')

## Остатки fbs из базы

In [230]:
oz_fbs_stocks = mwm.select_table_with_last_date_sort_list('oz_fbs_stocks', 
                                                          'date')
oz_products = mwm.select_table('oz_products')
oz_fbs_stocks['product_id'] = oz_fbs_stocks['product_id'].astype(int)
if len(oz_products) - len(list(set(oz_products['ozon_product_id']))) !=0:
    exit()
oz_products = oz_products[['Артикул', 'ozon_product_id']]\
              .rename(columns={'ozon_product_id': 'product_id'})
oz_fbs_stocks = oz_products.merge(oz_fbs_stocks, 
                                  on='product_id', 
                                  how='right')
result_column = 'present'
df_0 = pd.DataFrame(list(set(oz_fbs_stocks['Артикул'])), columns=['Артикул'])
for row in list(set(oz_fbs_stocks['warehouse_name'])):
    df = oz_fbs_stocks[oz_fbs_stocks['warehouse_name']==row][['Артикул', result_column]]\
         .rename(columns={result_column: row})\
         .groupby('Артикул').sum()\
         .reset_index(drop=False)
    df.iloc[:, 1] = df.iloc[:, 1].astype(float)
    if df.iloc[:, 1].sum() > 0:
        df_0 = df_0.merge(df, on='Артикул', how='outer')
oz_fbs_stocks, df_0 = df_0, None
oz_fbs_stocks['Артикул'] = oz_fbs_stocks['Артикул'].astype(str)

In [231]:
if 'ПСР' not in oz_fbs_stocks.columns:
    oz_fbs_stocks['ПСР'] = 0

In [232]:
oz_fbs_stocks['Остаток fbs'] = [oz_fbs_stocks.iloc[x, 1:].fillna(0).sum() for x in list(oz_fbs_stocks.index)]

In [233]:
oz_fbs_stocks

,Артикул,Склад Химки (FBS обычные товары),Склад Иркутск,ПСР,Остаток fbs
0,Z0060,0.0,0.0,0,0.0
1,G0286,189.0,12.0,0,201.0
2,30001,0.0,0.0,0,0.0
3,K0060,0.0,0.0,0,0.0
4,30404,0.0,0.0,0,0.0
...,...,...,...,...,...
1178,G0195,12.0,6.0,0,18.0
1179,A3503,1.0,0.0,0,1.0
1180,29315,0.0,0.0,0,0.0
1181,18307,0.0,0.0,0,0.0


In [234]:
oz_fbs_stocks['Остаток fbs'].sum()

31227.0

In [235]:
# oz__fbs_stocks = wwm.get_tabl_from_bd('oz__fbs_stocks', flag=1)
# oz__fbs_stocks['product_id'] = oz__fbs_stocks['product_id'].astype(int)
# oz__products = wwm.get_tabl_from_bd('oz__products', flag=1)
# if len(oz__products) - len(list(set(oz__products['Ozon_Product_ID']))) !=0:
#     exit()
# oz__products = oz__products[['Артикул', 'Ozon_Product_ID']]\
#                .rename(columns={'Ozon_Product_ID': 'product_id'})
# oz__fbs_stocks = oz__products.merge(oz__fbs_stocks, 
#                                     on='product_id', 
#                                     how='right')
# result_column = 'present'
# df_0 = pd.DataFrame(list(set(oz__fbs_stocks['Артикул'])), columns=['Артикул'])
# for row in list(set(oz__fbs_stocks['warehouse_name'])):
#     df = oz__fbs_stocks[oz__fbs_stocks['warehouse_name']==row][['Артикул', result_column]]\
#          .rename(columns={result_column: row})\
#          .groupby('Артикул').sum()\
#          .reset_index(drop=False)
#     df.iloc[:, 1] = df.iloc[:, 1].astype(float)
#     if df.iloc[:, 1].sum() > 0:
#         df_0 = df_0.merge(df, on='Артикул', how='outer')
# oz__fbs_stocks, df_0 = df_0, None
# oz__fbs_stocks['Артикул'] = oz__fbs_stocks['Артикул'].astype(str)

In [236]:
# oz__fbs_stocks['Остаток fbs'] = [oz__fbs_stocks.iloc[x, 1:].fillna(0).sum() for x in list(oz__fbs_stocks.index)]

In [237]:
# oz__fbs_stocks

In [238]:
# oz__fbs_stocks['Остаток fbs'].sum()

In [239]:
# oz__fbs_stocks = pd.DataFrame(columns=['Артикул'])
# for row in ['stock (8).xlsx', 'stock (7).xlsx']:
#     df_0 = pd.read_excel(path_downloads + row)
#     df_0 = df_0[['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 4']]
#     df_0.columns = ['Склад', 'Артикул', 'Остаток']
#     df_0 = df_0.rename(columns={'Остаток': df_0.loc[0]['Склад']})
#     df_0 = df_0.drop('Склад', axis=1)
#     df_0 = df_0.groupby('Артикул').sum().reset_index(drop=False)
#     wwm.unique_items(df_0)
#     oz__fbs_stocks = oz__fbs_stocks.merge(df_0, on='Артикул', how='outer')
#     wwm.unique_items(oz__fbs_stocks)
# oz__fbs_stocks['Остаток fbs'] = [oz__fbs_stocks.iloc[x, 1:].fillna(0).sum() for x in list(oz__fbs_stocks.index)]

In [240]:
# oz__fbs_stocks['Остаток fbs'].sum()

In [241]:

# oz__fbs_stocks['Артикул'] = oz__fbs_stocks['Артикул'].astype(str)
# if len(oz__fbs_stocks) - len(list(set(oz__fbs_stocks['Артикул']))) != 0:
#     print('error')
#     int('error')
# else:
#     None
# if 'Остаток fbs' in OZON_full_df.columns:
#     OZON_full_df = OZON_full_df.drop('Остаток fbs', axis=1)
# OZON_full_df = OZON_full_df.merge(oz__fbs_stocks, on='Артикул', how='left')
# oz__fbs_stocks = None
# OZON_full_df = OZON_full_df[list(OZON_full_df.columns)[:list(OZON_full_df.columns).index('Остаток')+1]\
# + ['Остаток fbs'] + list(OZON_full_df.columns)[list(OZON_full_df.columns).index('Остаток')+1:-1]]

In [242]:

oz_fbs_stocks['Артикул'] = oz_fbs_stocks['Артикул'].astype(str)
if len(oz_fbs_stocks) - len(list(set(oz_fbs_stocks['Артикул']))) != 0:
    print('error')
    int('error')
else:
    None
if 'Остаток fbs' in OZON_full_df.columns:
    OZON_full_df = OZON_full_df.drop('Остаток fbs', axis=1)
OZON_full_df = OZON_full_df.merge(oz_fbs_stocks, on='Артикул', how='left')
oz_fbs_stocks = None
OZON_full_df = OZON_full_df[list(OZON_full_df.columns)[:list(OZON_full_df.columns).index('Остаток')+1]\
+ ['Остаток fbs'] + list(OZON_full_df.columns)[list(OZON_full_df.columns).index('Остаток')+1:-1]]

In [243]:
list(OZON_full_df.columns)

['Артикул',
 'Название',
 'Номенклaтурa',
 'Баркод',
 'ТМ',
 'Статус',
 '20-February-2024',
 '21-February-2024',
 '22-February-2024',
 '23-February-2024',
 '24-February-2024',
 '25-February-2024',
 '26-February-2024',
 '27-February-2024',
 '28-February-2024',
 '29-February-2024',
 '01-March-2024',
 '02-March-2024',
 '03-March-2024',
 '26-February-03-March',
 '04-March-2024',
 '05-March-2024',
 '06-March-2024',
 '07-March-2024',
 '08-March-2024',
 '09-March-2024',
 '10-March-2024',
 '04-March-10-March',
 '11-March-2024',
 '12-March-2024',
 '13-March-2024',
 '14-March-2024',
 '15-March-2024',
 '16-March-2024',
 '17-March-2024',
 '11-March-17-March',
 '18-March-2024',
 '19-March-2024',
 '20-March-2024',
 '21-March-2024',
 '22-March-2024',
 '23-March-2024',
 '24-March-2024',
 '18-March-24-March',
 '25-March-2024',
 '26-March-2024',
 '27-March-2024',
 '28-March-2024',
 '29-March-2024',
 '30-March-2024',
 '31-March-2024',
 '25-March-31-March',
 '01-April-2024',
 '02-April-2024',
 '03-April-2

In [244]:
OZON_full_df.iloc[:, -2:].sum().sum()

11404.0

In [245]:
OZON_full_df['Остаток fbs'].sum()

31227.0

In [246]:
OZON_full_df

,Артикул,Название,Номенклaтурa,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,Шушары 8.11,Адыгейск 13.11,Пушкино 13.11,Софьино 13.11,Екат 16.11,Самара,Шушары 23.11,Склад Химки (FBS обычные товары),Склад Иркутск,ПСР
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,149314846,7432141949918,SS&Y Group,отказ,0,1,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,149314842,7432141949925,SS&Y Group,отказ,1,4,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,149314896,7432141949932,SS&Y Group,отказ,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,152870959,7432141550639,SS&Y Group,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,16920,Нож канцелярский металлический,676347909,4685453947735,Parchment,текущий,19,4,7,5,...,NaN,NaN,500.0,NaN,300.0,500.0,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Z0401,Ваза для цветов и сухоцветов решетка,877683869,4601663851785,Limpide Side,текущий Дарья,1,4,6,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,37.0,0.0
1228,Z0402,Ваза 5 колб,0,4665896931140,Limpide Side,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1229,Z0500,Декантер с бокалами,884222464,4602723427780,Limpide Side,текущий Дарья,24,19,10,5,...,NaN,NaN,NaN,68.0,NaN,NaN,NaN,0.0,1.0,0.0
1230,Z0501,Графин для воды,950979843,4630112870930,Limpide Side,текущий Дарья,6,6,3,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [247]:
# OZON_full_df = OZON_full_df.merge(oz__fbs_stocks, on='Артикул', how='left')

## Сохранение наценки

In [248]:
oz_markup_metrics = pd.read_excel(wwm.path_oz_markup_metrics,
                                  dtype={'Артикул': 'str'})
if len(oz_markup_metrics) - len(list(set(oz_markup_metrics['Артикул']))) != 0:
    print('Неуникальные артикулы в oz_markup_metrics')
    exit()
oz_markup_new = OZON_full_features_metric[['Артикул', 'Наценка']]\
                .rename(columns={'Наценка': const.dt_now.strftime('%d-%B-%Y')}).copy()
if len(oz_markup_new) - len(list(set(oz_markup_new['Артикул']))) != 0:
    print('Неуникальные артикулы в wb_markup_new')
    exit()
col = wwm.dt_col(oz_markup_new)[0]
for i in range(len(oz_markup_new)):
    art = oz_markup_new.loc[i]['Артикул']
    if art in list(set(ozon_price_metrics['Артикул'])):
        if ozon_price_metrics[ozon_price_metrics['Артикул']==art][col].values[0] == 0:
            oz_markup_new.loc[i, col] = None
if col in oz_markup_metrics.columns:
    oz_markup_metrics = oz_markup_metrics.drop(col, axis=1)
    oz_markup_metrics = oz_markup_metrics.merge(oz_markup_new, on='Артикул', how='outer')
else:
    oz_markup_metrics = oz_markup_metrics.merge(oz_markup_new, on='Артикул', how='outer')
if len(oz_markup_metrics) - len(list(set(oz_markup_metrics['Артикул']))) != 0:
    print('Неуникальные артикулы в oz_markup_metrics')
    exit()
oz_markup_metrics.to_excel(wwm.path_oz_markup_metrics, index=False)

In [249]:
ozon_fbs_metrics = wwm.rename_data_from_reestr(ozon_fbs_metrics, 
                                                  'Название', 
                                               reestr=reestr)
ozon_size_metrics = wwm.rename_data_from_reestr(ozon_size_metrics, 
                                                  'Название', 
                                                  reestr=reestr)
OZON_full_features_metric = wwm.rename_data_from_reestr(OZON_full_features_metric, 
                                                  'Название', 
                                                  reestr=reestr)
OZON_full_df = wwm.rename_data_from_reestr(OZON_full_df, 
                                                  'Название', 
                                                  reestr=reestr)

In [250]:
OZON_full_features_metric.to_csv(path_OZON_full_features_metric, index=False, sep=";", encoding='utf-8-sig')

In [251]:
ozon_size_metrics.to_csv(path_ozon_size_metrics, index=False, sep=";", encoding='utf-8-sig')

In [252]:
OZON_full_df.to_csv(path_OZON_full_df, index=False, sep=";", encoding='utf-8-sig')

In [253]:
ozon_fbs_metrics.to_csv(path_ozon_fbs_metrics, index=False, sep=";", encoding='utf-8-sig')

In [254]:
ozon_size_metrics

,Артикул,Название,Номенклaтурa,Баркод,ТМ,26-August-2022,29-August-2022,30-August-2022,31-August-2022,01-September-2022,...,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024,Diff
0,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,1.493149e+08,7.432142e+12,SS&Y Group,648.0,648.0,648.0,648.0,648.0,...,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.00
1,18309,"Чехол для airpods 2 с карабином, черный",1.493149e+08,7.432141e+12,SS&Y Group,166.0,166.0,166.0,166.0,166.0,...,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.00
2,18308,"Чехол для airpods 2 с карабином, голубой",1.493149e+08,7.432141e+12,SS&Y Group,198.0,198.0,198.0,198.0,198.0,...,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.00
3,18307,"Чехол для airpods 2 с карабином, сиреневый",1.492952e+08,7.432142e+12,SS&Y Group,198.0,198.0,198.0,198.0,198.0,...,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.21,0.08
4,18306,"Чехол для airpods 2 с карабином, коралловый",1.494596e+08,7.432142e+12,SS&Y Group,175.0,175.0,175.0,175.0,175.0,...,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,A5403,Хлебница большая,1.393961e+09,4.673766e+12,Shiny Kitchen,NaN,NaN,NaN,NaN,NaN,...,6.41,6.41,6.41,6.41,6.41,6.41,6.41,6.41,6.41,0.00
1227,28900,"Массажер гуаша для лица, розовый кварц",1.672768e+08,7.432142e+12,SSY,NaN,NaN,NaN,NaN,NaN,...,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.00
1228,50458,"Расческа с зеркальцем матовая, фиолетовая",1.469117e+09,4.674077e+12,REVOLUT,NaN,NaN,NaN,NaN,NaN,...,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.00
1229,50459,"Расческа с зеркальцем матовая, черная",1.469119e+09,4.674383e+12,REVOLUT,NaN,NaN,NaN,NaN,NaN,...,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.00


In [255]:
# OZON_full_df = OZON_full_df.merge(pd.read_excel(path_downloads + 'Софьино.xlsx', 
#               dtype={'Артикул': 'str'}, 
#               usecols=['Артикул', 'В поставке (шт.)']), on='Артикул', how='left')

In [256]:
# data = wb_markups.get_param_data()

# pr_n = wb_markups\
# .return_price(list(set(WB_full_df[WB_full_df['Артикул'].isin(list(data['Артикул']))]['Артикул'])), 
#               default_nc=80)

In [257]:
# import oz_markups
# data = oz_markups.get_param_data()

In [258]:
# pr_n = wb_markups\
# .return_price(list(set(WB_full_df[WB_full_df['Артикул'].isin(list(data['Артикул']))]['Артикул'])), 
#               default_nc=80)

In [259]:
# pr_n = oz_markups.return_price(list(data['Артикул']), default_nc=80)

In [260]:
# pr_n

In [261]:
# oz_markups.calc_features(['16801'], [834], flag_profit=True)[0][2]

In [262]:
# for i in range(len(OZON_full_df)):
#     art = OZON_full_df.loc[i]['Артикул']
#     if art in list(pr_n.keys()):
#         OZON_full_df.loc[i, 'Цена'] = pr_n[art]
#         OZON_full_df.loc[i, 'Наценка'] = oz_markups.calc_features([art], [pr_n[art]], flag_profit=True)[0][1]
#         OZON_full_df.loc[i, 'Прибыль'] = oz_markups.calc_features([art], [pr_n[art]], flag_profit=True)[0][2]
#     else:
#         OZON_full_df.loc[i, 'Цена'] = None
#         OZON_full_df.loc[i, 'Наценка'] = None
#         OZON_full_df.loc[i, 'Прибыль'] = None

In [263]:
# OZON_full_df

In [264]:
# OZON_full_df[OZON_full_df['Прибыль'].astype(str)!='None'][['Артикул', 'Прибыль']]\
# .reset_index(drop=True)

In [265]:
# df = data[['Артикул', 'Себестоимость']].merge(OZON_full_df[OZON_full_df['Прибыль'].astype(str)!='None'][['Артикул', 'Прибыль', '03-April-2024']]\
# .reset_index(drop=True)\
# .rename(columns = {'03-April-2024': 'Продажи'}),
# on='Артикул', 
#                                              how='inner').copy()

In [266]:
# df

In [267]:
# ITOG_NAC = round((((df['Прибыль'] * df['Продажи']).sum() / (df['Себестоимость'] *  df['Продажи']).sum()) * 100), 2)


In [268]:
# ITOG_NAC

In [269]:
# OZON_full_df.loc[len(OZON_full_df)-1, 'Итого наценка'] = 80.15

In [270]:
def unloading():
    
    with pd.ExcelWriter(path_algoritm + "Все файлы для ежедневной презентации/full_OZON_df2.0.xlsx") as writer:





        OZON_full_df.to_excel(writer, sheet_name="OZON_full_df", index=False)
        for column in OZON_full_df:
            column_width = max(OZON_full_df[column].astype(str).map(len).max() + 3, len(column))
            col_idx = OZON_full_df.columns.get_loc(column)
            writer.sheets['OZON_full_df'].set_column(col_idx, col_idx, column_width)



    writer.save()
    

## Выгрузка всех метрик в одном файле

In [271]:
def unloading():
    
    with pd.ExcelWriter(path_algoritm + "Все файлы для ежедневной презентации/full_OZON_df.xlsx") as writer:


        ozon_orders_metrics_non_orders.to_excel(writer, sheet_name="ozon_orders_metrics", index=False)
        for column in ozon_orders_metrics_non_orders:
            column_width = max(ozon_orders_metrics_non_orders[column].astype(str).map(len).max(), len(column))
            col_idx = ozon_orders_metrics_non_orders.columns.get_loc(column)
            writer.sheets['ozon_orders_metrics'].set_column(col_idx, col_idx, column_width)



        ozon_price_metrics.to_excel(writer, sheet_name="ozon_price_metrics", index=False)
        for column in ozon_price_metrics:
            column_width = max(ozon_price_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = ozon_price_metrics.columns.get_loc(column)
            writer.sheets['ozon_price_metrics'].set_column(col_idx, col_idx, column_width)


        ozon_fbs_metrics.to_excel(writer, sheet_name="ozon_fbs_metrics", index=False)
        for column in ozon_fbs_metrics:
            column_width = max(ozon_fbs_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = ozon_fbs_metrics.columns.get_loc(column)
            writer.sheets['ozon_fbs_metrics'].set_column(col_idx, col_idx, column_width)



        stocks_ozon_metrics.to_excel(writer, sheet_name="stocks_ozon_metrics", index=False)
        for column in stocks_ozon_metrics:
            column_width = max(stocks_ozon_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = stocks_ozon_metrics.columns.get_loc(column)
            writer.sheets['stocks_ozon_metrics'].set_column(col_idx, col_idx, column_width)



        ozon_size_metrics.to_excel(writer, sheet_name="ozon_size_metrics", index=False)
        for column in ozon_size_metrics:
            column_width = max(ozon_size_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = ozon_size_metrics.columns.get_loc(column)
            writer.sheets['ozon_size_metrics'].set_column(col_idx, col_idx, column_width)





        OZON_full_features_metric.to_excel(writer, sheet_name="OZON_full_features_metric", index=False)
        for column in OZON_full_features_metric:
            column_width = max(OZON_full_features_metric[column].astype(str).map(len).max() + 3, len(column))
            col_idx = OZON_full_features_metric.columns.get_loc(column)
            writer.sheets['OZON_full_features_metric'].set_column(col_idx, col_idx, column_width)



        OZON_full_df.to_excel(writer, sheet_name="OZON_full_df", index=False)
        for column in OZON_full_df:
            column_width = max(OZON_full_df[column].astype(str).map(len).max() + 3, len(column))
            col_idx = OZON_full_df.columns.get_loc(column)
            writer.sheets['OZON_full_df'].set_column(col_idx, col_idx, column_width)



    writer.save()
    

In [272]:
# dff = ozon_price_metrics[['Артикул']+[dt_col(ozon_price_metrics)[-2]]]\
# .rename(columns={'13-October-2023': 'Цена_13-October-2023'})\
# .merge(OZON_full_df[['Артикул']+dt_col(OZON_full_df)[-3:-1]], on='Артикул', how='inner')
# dff['Выручка_13-October-2023'] = dff['13-October-2023'] * dff['Цена_13-October-2023']
# dff['Выручка_14-October-2023'] = dff['14-October-2023'] * dff['Цена_13-October-2023']
# dff = dff[['Артикул', 'Цена_13-October-2023', '13-October-2023', 'Выручка_13-October-2023', '14-October-2023', 'Выручка_14-October-2023']]
# dff.to_excel(path_downloads + 'Выручка озон 13-14 сентября.xlsx', index=False)

In [273]:
try:
    unloading()
except:
    unloading()

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [274]:
OZON_full_df.to_excel(path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Метрики для презентации /OZON_full_df.xlsx', index=False)

## Обновление файла full_OZON_df на корпоративном гугл диске 

In [275]:
# Подключение к гугл диску, аутентефикация

gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)


# Находим "id" папки "Дима"

id_folder_Dima = [x['id'] for x in drive\
                 .ListFile({'q': "'root' in parents and trashed=false"}).GetList() if x['title'] == "Дима"]
if len(id_folder_Dima) != 1:
    print('error')
    int('error')
else:
    id_folder_Dima = id_folder_Dima[0]

    
# Находим "id" файла "full_OZON_df" в папке "Дима"

id_full_OZON_df = [x['id'] for x in drive.ListFile({'q': f"'{id_folder_Dima}' in parents and trashed=false"})\
.GetList() if 'full_OZON_df' in x['title']]

if len(id_full_OZON_df) != 1:
    print('error')
    int('error')
else:
    id_full_OZON_df = id_full_OZON_df[0]

    
# Путь к файлу "full_OZON_df" на локальном компьютере

path_file = path_algoritm + 'Все файлы для ежедневной презентации/full_OZON_df.xlsx'


# Обновление файла "full_WB_df" на корпоративном гугл диске 

file = drive.CreateFile({'id': id_full_OZON_df})
file.SetContentFile(path_file)
file.Upload()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=141351806184-lcpctltvgumjg7nedaiblgr9vtjpk98v.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [276]:
shutka_minutka = pyjokes.get_joke()
print(shutka_minutka)

Software developers like to solve problems. If there are no problems handily available, they will create their own.


In [277]:
list(OZON_full_df.columns)

['Артикул',
 'Название',
 'Номенклaтурa',
 'Баркод',
 'ТМ',
 'Статус',
 '20-February-2024',
 '21-February-2024',
 '22-February-2024',
 '23-February-2024',
 '24-February-2024',
 '25-February-2024',
 '26-February-2024',
 '27-February-2024',
 '28-February-2024',
 '29-February-2024',
 '01-March-2024',
 '02-March-2024',
 '03-March-2024',
 '26-February-03-March',
 '04-March-2024',
 '05-March-2024',
 '06-March-2024',
 '07-March-2024',
 '08-March-2024',
 '09-March-2024',
 '10-March-2024',
 '04-March-10-March',
 '11-March-2024',
 '12-March-2024',
 '13-March-2024',
 '14-March-2024',
 '15-March-2024',
 '16-March-2024',
 '17-March-2024',
 '11-March-17-March',
 '18-March-2024',
 '19-March-2024',
 '20-March-2024',
 '21-March-2024',
 '22-March-2024',
 '23-March-2024',
 '24-March-2024',
 '18-March-24-March',
 '25-March-2024',
 '26-March-2024',
 '27-March-2024',
 '28-March-2024',
 '29-March-2024',
 '30-March-2024',
 '31-March-2024',
 '25-March-31-March',
 '01-April-2024',
 '02-April-2024',
 '03-April-2

In [278]:
exit()

SystemExit: 

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
